In [1]:
# 一、数据读取

# mysql数据读取——构造一个数据字典
import pymysql
import pandas as pd
import numpy as np
def mysqlExe(sql):
    conn = pymysql.connect(host="localhost",
                          port=3306,
                          user="root",
                          password="jn580236",
                          database="baidu_data",
                          charset="utf8")
    cursor = conn.cursor(cursor=pymysql.cursors.DictCursor)
    cursor.execute(sql)
    data = cursor.fetchall()
    #对数据进行结构组织
    mydata = pd.DataFrame(data,columns = list(data[0].keys()))
    #pd.set_option('display.max_rows',None)
    #输出全部行结果，不省略
    #pd.set_option('display.max_columns',None)
    #输出全部列结果，不省略 
    #conn.commit()
    #在执行update、insert和delete命令时，需要增加一条conn.commit()命令
    cursor.close()
    conn.close()
    return mydata

#（一）数据sql —— 以星期为单位同时切开每个时辰的数据
week_datasql = "select c.原文摘要,c.c_time,c.H_time from \
(select b.原文摘要,dayname(b.date) as c_time,b.H_time from \
(select a.ID,a.书名,a.文档ID,a.用户ID,a.性别 as gender,a.年龄 as age,a.教育水平 as edu,a.原文摘要,a.time,\
a.date,EXTRACT(MONTH from a.time) as M_time,EXTRACT(DAY from a.time) as D_time,EXTRACT(HOUR from a.time) as H_time from \
(select ID,书名,文档ID,用户ID,性别,年龄,教育水平,原文摘要,FROM_UNIXTIME(`行为发生时间戳`,'%Y-%m-%d %H:%i:%s')as time,\
FROM_UNIXTIME(`行为发生时间戳`,'%Y-%m-%d')as date from reader_note where 书名 in \
('何以笙箫默','嫌疑人X的先生','小王子','边城','老人与海','傲慢与偏见','解忧杂货店','时间移民','谁动了我的奶酪')) a ) b ) c \
order by c.c_time,c.H_time"

# （二）数据字典 —— 构建以星期为key1，嵌套以时辰为key2，时辰内原文摘要为value的字典嵌套的字典数据
week_opt_data = mysqlExe(week_datasql) # 转化数据为dataframe
#week_list = list(set(week_opt_data['c_time'].tolist()))  # 获得日期列表
week_list = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']  # 获得星期列表

# 【全量week数据】
all_week_dict = {}
for week in week_list:
    h_week_dict = {}
    for i in range(0,24):
        data = week_opt_data.loc[(week_opt_data.c_time==week)&(week_opt_data.H_time==i)]['原文摘要'].tolist()
        #print("week：%s；i：%s；len：%s；"%(week,i,len(data)))
        h_week_dict['%s'%i]=data
    all_week_dict['%s'%week]=h_week_dict

# （三） 拆分数据子字典 —— 将all_week_dict按照工作日、周末、节假日、周末+节假日拆分成四个子字典
def get_dict_sentnum(data_dict):
    # 获取两层字典嵌套中，对应时间节点的评论条数
    sentnum = []
    for i in data_dict.keys():
        for j in data_dict[i].keys():
            num = len(data_dict[i][j])
            sentnum.append(num)
    return sentnum

# （1）原始week下的每个时刻评论条数
week_moral_sentnum = get_dict_sentnum(all_week_dict)
# （2）切分数据集使用config
workdays_list = week_list[0:5]
weekend_list = week_list[5:7]
holiday_list = ['2017-03-08','2017-04-04','2017-05-01','2017-05-02','2017-05-03']
weekend_holiday_list = weekend_list+holiday_list
# 注：2017-3-8(妇女节) 2017-04-04（清明节），2017-5-1,2017-5-2,2017-5-3(国庆节)

#**********************************************************************************************
# 补充数据集（用于构建节假日使用）
# 由于节假日是单独的日期，因此需要构建以date为单位的数据字典来引用

# B（一） 数据sql —— 按天为单位切开每个时辰的数据
date_datasql = "select b.原文摘要,b.date,b.H_time from \
(select a.ID,a.书名,a.文档ID,a.用户ID,a.性别 as gender,a.年龄 as age,a.教育水平 as edu,a.原文摘要,a.time,\
a.date,EXTRACT(MONTH from a.time) as M_time,EXTRACT(DAY from a.time) as D_time,EXTRACT(HOUR from a.time) as H_time from \
(select ID,书名,文档ID,用户ID,性别,年龄,教育水平,原文摘要,FROM_UNIXTIME(`行为发生时间戳`,'%Y-%m-%d %H:%i:%s')as time,\
FROM_UNIXTIME(`行为发生时间戳`,'%Y-%m-%d')as date from reader_note where 书名 in \
('何以笙箫默','嫌疑人X的先生','小王子','边城','老人与海','傲慢与偏见','解忧杂货店','时间移民','谁动了我的奶酪')) a ) b \
order by b.date,H_time"

# B（二）构建按date为key1，嵌套以时辰为key2，时辰内原文摘要为value的字典嵌套的字典数据
date_opt_data = mysqlExe(date_datasql) # 转化数据为dataframe
date_list = list(set(date_opt_data['date'].tolist()))  # 获得日期列表

# B（三）对date_list列表中的日期进行按日期排序
import datetime  
def get_timestamp(date):
    return datetime.datetime.strptime(date,"%Y-%m-%d").timestamp()
date_list_sort = sorted(date_list,key=lambda date:get_timestamp(date))

#print(date_opt_data[0:10])
#print(date_list)
#print(date_list_sort)

# 以天为单位并切分每一天中每个时辰的切分字典
all_date_dict = {} 
for date in date_list:
    h_data_dict = {}
    for i in range(0,24):
        data = date_opt_data.loc[(date_opt_data.date==date)&(date_opt_data.H_time==i)]['原文摘要'].tolist()
        h_data_dict['%s'%i]=data
    all_date_dict['%s'%date] = h_data_dict

#***********************************************************************************************
# 全量week，每个时刻下的评论条数
week_sentnum = get_dict_sentnum(all_week_dict)
# （3）工作日，节假日，周末，节假日&周末——每个时刻下的评论条数
#（3.1）工作日的子字典 & 每个时刻下的评论条数
workdays_dict = {key:value for key,value in all_week_dict.items() if key in workdays_list}
workdays_sentnum = get_dict_sentnum(workdays_dict)
print(workdays_sentnum[0:10])
#（3.2）周末的子字典 & 每个时刻下的评论条数
weekend_dict = {key:value for key,value in all_week_dict.items() if key in weekend_list}
weekend_sentnum = get_dict_sentnum(weekend_dict)
#（3.3）节假日的子字典（这个字典不是从all_week_dict中拆出来的，而是从all_date_dict中挑选出来）
holiday_dict = {key:value for key,value in all_date_dict.items() if key in holiday_list}
holiday_sentnum = get_dict_sentnum(holiday_dict)
print(holiday_dict.keys())
# （3.4）周末+节假日的字字典（字典合并）& 每个时刻下的评论条数
weekend_holiday_dict = {**weekend_dict,**holiday_dict}
weekend_holiday_sentnum = get_dict_sentnum(weekend_holiday_dict)

[263, 101, 44, 41, 14, 19, 90, 115, 209, 226]
dict_keys(['2017-03-08', '2017-05-02', '2017-04-04', '2017-05-03', '2017-05-01'])


In [2]:
# 二、情感词列表读取——构建情感词列表
# （一）构造情感词词典
import os
path1 = r'F:/姜崇博士文档/博士文本selfcorpus/SC-LIWC/new_LIWC/zl_LIWC/EMO_subpathconfig.txt'
path2 = r'F:/姜崇博士文档/博士文本selfcorpus/SC-LIWC/new_LIWC/zl_LIWC/LIWC_subpathconfig.txt'
path3 = r'F:/姜崇博士文档/博士文本selfcorpus/SC-LIWC/new_LIWC/zl_LIWC/emo/lex_degree.txt'
path4 = r'F:/姜崇博士文档/博士文本selfcorpus/SC-LIWC/new_LIWC/zl_LIWC/Emotions_CLWC_subpathconfig.txt'
with open(path1,'r',encoding='utf-8') as file:
    content1 = file.readlines()
with open(path2,'r',encoding='utf-8') as file:
    content2 = file.readlines()
with open(path3,'r',encoding='utf-8') as file:
    content3 = file.readlines()
with open(path4,'r',encoding='utf-8') as file:
    content4 = file.readlines()

# （1）构造emotion（包含agency,communion,lex_negation,lex_noscore_degree）的情感词典
emo_dict = {}    
for i in content1:
    emo_path = i.strip()
    name = os.path.split(emo_path)[1]
    with open(emo_path,'r',encoding='utf-8') as file:
        content = file.readlines()
        words = [i.strip() for i in content]
        emo_dict[name]=words

# （2）构建degree（程度词-lex_degree）的情感词典
# 注：程度词带有权重值，因此单独进行构造字典
degree_dict = {}
for i in content3:
    words = i.strip().split()[0]
    values = int(i.strip().split()[1])
    degree_dict[words] = values

#*********************************************************************
# 补充构造——LIWC的情感词典
liwc_dict = {}
for i in content2:
    liwc_path = i.strip()
    name = os.path.split(liwc_path)[1]
    with open(liwc_path,'r') as file:
        content = file.readlines()
        words = [i.strip() for i in content]
        liwc_dict[name]=words

# 构造C_liwc的情感词典(中文liwc情感词典)
#【注，这里的字典编码用的gbk，因此在打开解码时需要用utf-8解码】
cliwc_dict = {}
for i in content4:
    cliwc_path = i.strip()
    name = os.path.split(cliwc_path)[1]
    with open(cliwc_path,'r',encoding='utf-8') as file:
        content = file.readlines()
        words = [i.strip() for i in content]
        cliwc_dict[name]=words

#*********************************************************************
# （3）生成拥有全部情感词的文件，用于结巴分词的自定义字典（避免结巴分词将情感词切乱）
total_dict =dict(emo_dict,**liwc_dict,**cliwc_dict)   #含有全部情感词及分类名字典
total_words = []   #含有全部情感词列表
for i in total_dict.values():
    total_words = total_words+i

# （4）构造加入程度词，communion+agency+lex_negation+degree词表用来做否定词+程度词修饰的语句
lex_emo_degree_words = []
for i in emo_dict.values():
    lex_emo_degree_words = lex_emo_degree_words + i
print(len(lex_emo_degree_words))
jieba_dict_path = r'F:/姜崇博士文档/博士文本selfcorpus/SC-LIWC/new_LIWC/zl_LIWC/total_emotion_words.txt'

'''
# 用于构造并生成结巴分词的自定义字典（已完成）
jieba_dict_path = r'F:/姜崇博士文档/博士文本selfcorpus/SC-LIWC/new_LIWC/zl_LIWC/total_emotion_words.txt'
for i in total_words:
    with open(jieba_dict_path,'a+',encoding='utf-8')as file:
        file.writelines(i+'\n')
'''

1390


"\n# 用于构造并生成结巴分词的自定义字典（已完成）\njieba_dict_path = r'F:/姜崇博士文档/博士文本selfcorpus/SC-LIWC/new_LIWC/zl_LIWC/total_emotion_words.txt'\nfor i in total_words:\n    with open(jieba_dict_path,'a+',encoding='utf-8')as file:\n        file.writelines(i+'\n')\n"

In [3]:
# 三、结巴自定义字典与利用程度词规则进行结巴分词
import jieba
import re
import jieba.posseg as pseg
# 向结巴加载导入自定义的词典
jieba.load_userdict(jieba_dict_path)

# 函数（一）：输入包含句子的列表，输出为结巴分词后的词列表（采用自定义词典切割）
def get_segment(words):
    new_words = []
    for i in words:
        word = jieba.lcut(i)
        new_words = new_words + word
    return new_words

# 函数（二）：输入的是标注字典中的句子，输出为结巴分词后的词列表（采用自定义词典切割）
def get_sent_segment(sent):
    new_words = []
    word = jieba.lcut(sent)
    new_words = new_words + word
    return new_words

# 函数（三）：输入为两个包含词的列表，返回四个列表
# （1）两个列表的交集（不去重）
# （2）两个列表的交集（去重）
# （3）第一个返回的列表的长度（不去重单词数）
# （4）第二个返回的列表的长度（去重后单词数）
def compare_word(words1,words2):
    union = [word for word in words1 if word in words2]
    union_set = set(union)
    union_num = len(union)
    union_set_num = len(union_set)
    return union,union_set,union_num,union_set_num

# 函数（四）：输入为agency词表的统计次数列表和communion词表的统计次数列表
# 输出为道德分数，agency和communion的词表频率
def get_score_moral(data_agency,data_communion):
    data_tuple = list(zip(data_agency,data_communion))
    data_moral = []
    for ac in data_tuple:
        score_moral = ac[1]*722/(ac[0]*290)
        data_moral.append(score_moral)
    score_agency = []
    for ag in data_agency:
        sc_agency = ag/722
        score_agency.append(sc_agency)
    score_communion = []
    for co in data_communion:
        sc_communion = co/290
        score_communion.append(sc_communion)
    return data_moral,score_agency,score_communion

#********************************************************
# 程度词规则制定
# 函数（六）——依赖子函数（被用在子函数中）
# （1）第一层句子切分，利用双引号将对话内容抽取（“对话内容”）
def split_sentences_dialogue(line):
    line_split = re.split(r'[“”]',line.strip())
    line_split = [line.strip() for line in line_split if line.strip() not in ['“','”'] and len(line.strip())>1]
    return line_split

# （2）第二层切分，对旁白句子进行整句切分（非对话部分）
import re
def split_sentences_aside(line):
    line_split = re.split(r'[。！？；……]',line.strip())
    line_split = [line.strip() for line in line_split if line.strip() not in ['。','！','?','；','……'] and len(line.strip())>1]
    return line_split

# （3）第三层句子切分，增加（，）将整句进行切分（注：由于对话（dialogue）中仍然存在整句，因此需要增加（，）进行分句分割）
def split_half_sentences(line):
    line_split = re.split(r'[，。？！；……]',line.strip())
    line_split = [line.strip() for line in line_split if line.strip() not in ['，','。','？','！','；','……'] and len(line.strip())>1]
    return line_split

# （4）输入元组，输出元组内元素乘积结果
def product_tuple(tuple_example):
    from functools import reduce
    result = reduce(lambda x,y:x*y,tuple_example)
    return result

# 函数（六）——主函数
# 输入为一个批注句子，输出为按照程度词修饰后得到该句子中，communion和agency的词列表
def sent_degree_cal(data_sentence):
    # （一）句子级别拆分处理
    # 先利用（“”）把对话部分切割出来
    sentence_dialogue = split_sentences_dialogue(data_sentence)
    #print('第一层切割（“”）sentence:\n',sentence2)
    # 双引号之间的内容
    # （1）利用正则表达式将切割出来的对话部分构建一个单独的对话列表
    pattern = re.compile(r'“'+r'(.*?)'+r'”',re.S)
    dialogues = pattern.findall(data_sentence)
    #（2） 利用对话列表找到旁白部分的句子并按照整句进行切割，将整句和对话句子按照顺序重新组合成新句子列表
    # 这里之所以将对话句子单独拿出来，是当整体进行切割时，会将对话部分拆乱，无法组成一个单独句子
    new_split_sentences = []
    for i in sentence_dialogue:
        if i not in dialogues:
            sent_aside = split_sentences_aside(i)
            new_split_sentences = new_split_sentences + sent_aside
        if i in dialogues:
            sent_dialogues = r'“'+i+r'”'
            new_split_sentences.append(sent_dialogues)
    #print(new_split_sentences)
    # （3）增加逗号（，）将所有句子进行分句切分，每一个list里面是一个整句的分句
    # 型如：一条记录：{句子1：【[分句1]，[分句2]】，句子2：【[分句1]，[分句2]】}
    half_new_split_sentences = []
    for i in new_split_sentences:
        half_sentences = split_half_sentences(i)
        half_new_split_sentences.append(half_sentences)
    #print('分句列表：',half_new_split_sentences)
    # （二） 词级别拆分处理
    # （4）对新句子列表中的分句进行词切割
    mid_sent_list = []
    for i in half_new_split_sentences:
        mid_sent = []
        for j in i:
            words = get_sent_segment(j)
            #lex_emo_degree_words包括否定词，程度词，agency,communion四个词表的所有词列表
            new_words = [m for m in words[::-1] if m in lex_emo_degree_words]
            #将程度词替换为（程度词，权重）的元组
            degree_words = [m if m not in degree_dict.keys() else (m,degree_dict[m]) for m in new_words]
            # （4.1）处理特殊情况——一点也（程度1）+不（否定）
            new_degree_words = []
            # #########这里要补充特殊情况，程度词否定词的互相修饰
            for m,n in enumerate(degree_words):
                if type(n)==tuple and n[0] in ['一点','一点儿','一点点','一丁点','一丁点儿']:
                    if m>=1 and degree_words[m-1] in ['不会','也','都不','也不','不']:
                        new_degree = (n[0],3)
                        new_degree_words.append(new_degree)
                else:
                    new_degree_words.append(n)
            #（4.2）将否定词统一处理为'0'
            negation_words = [m if m not in emo_dict['lex_negation.txt'] else '0' for m in new_degree_words]
            # （4.3）将程度词改成统一字典形式
            # 这里改成统一形式是方便后续进行分组
            new_negation_words = []
            for i in negation_words:
                if type(i)==tuple:
                    new_i = {'degree':i[1]}
                    new_negation_words.append(new_i)
                else:
                    new_negation_words.append(i)
            # （4.4）将列表中的程度词进行itertools.groupby分组
            # 这里会将相邻的程度词，否定词分割成独立模块，最终剩下三种形式的组合
            from itertools import groupby
            from operator import itemgetter
            groupby_degree_words = [list(g) for k,g in groupby(new_negation_words,key=tuple)]
            # （4.5）将程度词列表构成元组
            new_groupby_degree_words = []
            for i in groupby_degree_words:
                if {'degree':1} in i or {'degree':2} in i or {'degree':3} in i:
                    degree_tuple = tuple([m['degree'] for m in i])
                    new_groupby_degree_words.append(degree_tuple)
                elif '0' in i:
                    lex_negation_tuple = tuple(i)
                    new_groupby_degree_words.append(lex_negation_tuple)
                else:
                    new_groupby_degree_words = new_groupby_degree_words + i
            #print(new_groupby_degree_words)
            # （4.6）将agency和communion与程度词和否定词进行捆绑
            union_degree_negation_words = []
            flag=0
            for m,n in enumerate(new_groupby_degree_words):
                if flag==0:
                    if m==0 and type(n)==tuple:
                        if '0' not in n:
                            if m<len(new_groupby_degree_words)-1:
                                if new_groupby_degree_words[m+1] in emo_dict['agency.txt'] or new_groupby_degree_words[m+1] in emo_dict['communion.txt']:
                                    # 程度词+com/agen+否定词
                                    if m<len(new_groupby_degree_words)-2 and '0' in new_groupby_degree_words[m+2]:
                                        union_tuple1 = (n,new_groupby_degree_words[m+1],new_groupby_degree_words[m+2])
                                        union_degree_negation_words.append(union_tuple1)
                                        flag = 2
                                    # 程度词+com/agen
                                    elif m<len(new_groupby_degree_words)-2 and '0' not in new_groupby_degree_words[m+2]:
                                        union_tuple2 = (n,new_groupby_degree_words[m+1])
                                        union_degree_negation_words.append(union_tuple2)
                                        flag=1
                                    # 其他情况
                                    else:
                                        union_tuple3 = (n,new_groupby_degree_words[m+1])
                                        union_degree_negation_words.append(union_tuple3)
                                        flag =1
                                else:union_degree_negation_words.append(n)
                            else:union_degree_negation_words.append(n)
                        else:union_degree_negation_words.append(n)    
                    elif n in emo_dict['agency.txt'] or n in emo_dict['communion.txt']:
                        if m<len(new_groupby_degree_words)-1 and type(new_groupby_degree_words[m+1])==tuple:
                            # com/agen+否定词或程度词+否定词或程度词
                            if m<len(new_groupby_degree_words)-2 and type(new_groupby_degree_words[m+2])==tuple:
                                union_tuple4 = (n,new_groupby_degree_words[m+1],new_groupby_degree_words[m+2])
                                union_degree_negation_words.append(union_tuple4)
                                flag=2
                            # com/agen+否定词或程度词
                            elif m<len(new_groupby_degree_words)-2 and type(new_groupby_degree_words[m+2])!=tuple:
                                union_tuple5 = (n,new_groupby_degree_words[m+1])
                                union_degree_negation_words.append(union_tuple5)
                                flag=1
                            # 其他情况
                            else:
                                union_tuple6 = (n,new_groupby_degree_words[m+1])
                                union_degree_negation_words.append(union_tuple6)
                                flag = 1                   
                        else:
                            union_degree_negation_words.append(n)
                    else:
                        union_degree_negation_words.append(n)
                else:
                    flag -= 1
            #print(union_degree_negation_words)
            #（4.7）将没有与communion和agency组合的程度词进行计算，将否定词进行舍弃
            # 保留分句中的连续程度词或单独程度词进行计算并生成数值。
            cal_degree_negation_words1 = []
            for i in union_degree_negation_words:
                if type(i)==tuple:
                    if '0' in i:
                        pass
                    elif 1 in i or 2 in i or 3 in i:
                        score = np.prod(i)
                        cal_degree_negation_words1.append(score)
                    else:
                        cal_degree_negation_words1.append(i)
                else:
                    cal_degree_negation_words1.append(i)
            #print(cal_degree_negation_words1)
            # （4.8）计算与communion和agency组合的程度词和否定词
            # 判断有否定词的计算是否需要改变词向，用1和-1表示
            # 判断有程度词的，计算程度词的乘积
            cal_degree_negation_words2 = []
            for i in cal_degree_negation_words1:
                if type(i)==tuple:
                    temp = []
                    for m in i:
                        if m in emo_dict['communion.txt'] or m in emo_dict['agency.txt']:
                            temp.append(m)
                        elif '0' in m:
                            if len(m)/2==0:
                                unchange = 1
                                temp.append(unchange)
                            else:
                                change = -1
                                temp.append(change)
                        elif 1 in m or 2 in m or 3 in m:
                            new_score = np.prod(m)
                            temp.append(new_score)
                    new_temp = tuple(temp)
                    cal_degree_negation_words2.append(new_temp)
                else:
                    cal_degree_negation_words2.append(i)
            #print(cal_degree_negation_words2)
            # （4.9）计算元组中的乘积
            # 将与communion和agency进行组合的程度词以及词向（1，-1）也进行计算，只保留最终组合计算结果
            cal_degree_negation_words3 = []
            for i in cal_degree_negation_words2:
                if type(i)==tuple:
                    word = [x for x in i if type(x)==str]
                    num = [x for x in i if type(x)!=str]
                    score = np.prod(num)
                    new_tuple = tuple(word+[score])
                    cal_degree_negation_words3.append(new_tuple)
                else:
                    cal_degree_negation_words3.append(i)
            #print(cal_degree_negation_words3)
            # 分句合成整句
            mid_sent.append(cal_degree_negation_words3)
        # 整句组成一条数据记录
        mid_sent_list.append(mid_sent)
    #print(mid_sent_list)
    # （5）将分句形成的多个列表融合进一个整句列表中。减少层级
    new_mid_sent_list=[]
    for i in mid_sent_list:
        operat_sent = []
        for j in i:
            operat_sent+=j
        new_mid_sent_list.append(operat_sent)
    #print(new_mid_sent_list)
    # （6）从整句中去除只有程度词不包含communion和agency的句子
    # 下面的代码可以判断列表中的元素是否都是统一type的简便判断。
    #len(list(groupby(yourlist,lambda i:type(i)))) == 1
    end_sent_list = []
    for i in new_mid_sent_list:
        if any(type(j) is str or type(j) is tuple for j in i):
            end_sent_list.append(i)
        else:
            pass
    #print(end_sent_list)
    # （7）计算分句中独立程度词对整个句子中communion和agency的词列表划分
    end_sent_words_list = []
    for i in end_sent_list:
        num = [j for j in i if not type(j)==str and not type(j)==tuple]
        words = [j for j in i if type(j)==str or type(j)==tuple]
        score = int(np.prod(num))
        new_words = score*words
        end_sent_words_list += new_words
    #print(end_sent_words_list)
    # （8）将元组中数值不为负的tuple组合拆分，保留数值为负的tuple组合。
    final_sent_words_list = []
    for i in end_sent_words_list:
        if type(i)==tuple:
            if i[1]>0:
                num = int(i[1])
                word = [i[0]]
                new_word = num*word
                final_sent_words_list += new_word
            else:
                final_sent_words_list.append(i)
        else:
            final_sent_words_list.append(i)
    #print(final_sent_words_list)
    # （9）最后保留的tuple类型是只有负值，需要变换词典的
    degree_communion = []
    degree_agency = []
    for i in final_sent_words_list:
        if type(i)==tuple:
            if i[0] in emo_dict['communion.txt']:
                num = int(abs(i[1]))
                word = [i[0]]
                new_word = num*word
                degree_agency += new_word
            if i[0] in emo_dict['agency.txt']:
                num1 = int(abs(i[1]))
                word1 = [i[0]]
                new_word1 = num1*word1
                degree_communion += new_word1
        else:
            if i in emo_dict['communion.txt']:
                degree_communion.append(i)
            elif i in emo_dict['agency.txt']:
                degree_agency.append(i)
    return degree_communion,degree_agency
#********************************************************
#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
# 函数（五）：使用否定词表，对每一个句子进行计算被否定词修饰的communion和agency词表
# 输入为一条标注句子，输出为该句子中的communion和agency的列表
# 函数依赖词表：lex_emo_words
# 函数依赖自定义函数：get_sent_segment
# 函数依赖官方包：itertools.groupby()
def getlex_communion_agency(sent):
    from itertools import groupby
    
    cal_sent = get_sent_segment(sent)
    new_cal_sent = [i for i in cal_sent[::-1] if i in lex_emo_words]
    change_new_cal_sent = [i if i not in emo_dict['lex_negation.txt'] else '0' for i in new_cal_sent]
    result = [list(g) for k,g in groupby(change_new_cal_sent)]
    new_result = []
    for i,j in enumerate(result):
        if i < len(result)-1:
            if '0' not in j and '0' not in result[i+1]:
                new_result = new_result + j
            if '0' not in j and '0' in result[i+1]:
                fale_tuple = tuple(j+result[i+1])
                new_result.append(fale_tuple)
        else:
            new_result = new_result + j
    end_communion = []
    end_agency = []
    for i in new_result:
        if '0' not in i:
            if i in emo_dict['communion.txt']:
                end_communion.append(i)
            if i in emo_dict['agency.txt']:
                end_agency.append(i)
        if '0' in i:
            num = len(i)-1
            if (num%2) == 0:
                if i[0] in emo_dict['communion.txt']:
                    end_communion.append(i)
                if i[0] in emo_dict['agency.txt']:
                    end_agency.append(i)
            else:
                if i[0] in emo_dict['communion.txt']:
                    end_agency.append(i)
                if i[0] in emo_dict['agency.txt']:
                    end_communion.append(i)
    return end_communion,end_agency


# （5）构造加入否定词的，communion+agency+lex_negation词表用来做后面的否定词修饰的语句(不包括degree程度词)
lex_emo_words = []
for i,j in emo_dict.items():
    if i != 'lex_noscore_degree.txt':
        for m in j:
            lex_emo_words.append(m)
print(len(lex_emo_words))
# （6）构造加入程度词的，communion+agency+lex_negation+degree词表用来做后面的否定词＋程度词修饰的语句
lex_emo_degree_words = []
for i in emo_dict.values():
    lex_emo_degree_words = lex_emo_degree_words + i
print(len(lex_emo_degree_words))
#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\83919\AppData\Local\Temp\jieba.cache
Loading model cost 0.622 seconds.
Prefix dict has been built successfully.


1039
1390


In [4]:
# 四、分词获取每小时communion和agency词列表，计算情境中的moral值
# 由于计算moral不能存在分母为0的情况，预处理将列表中0的值替换为1。
def change01(data):
    new_data = []
    for i in data:
        if i==0:
            i=i+1
            new_data.append(i)
        else:
            new_data.append(i)
    return new_data

'''
# week总量，workday_dict,weekend_dict,holiday_dict,weekend_holiday_dict
# 利用程度词进行分词获得communion和agency词表，和总切分词统计数
# 输入为数据集名称列表，和数据集词典，输出为communion，agency,all_words三个词统计列表
def degreecal_communion_agency_list(name_list,name_dict):
    week_communion = []
    week_agency = []
    cut_words_num = []
    for j in range(0,24):
        for i in name_list:
            segwords = get_segment(name_dict[i]['%s'%j])
            nums = len(segwords)
            cut_words_num.append(nums)
            temp_communion = []
            temp_agency = []
            for m in name_dict[i]['%s'%j]:
                c,a = sent_degree_cal(m)
                temp_communion += c
                temp_agency += a
            week_communion.append(len(temp_communion))
            week_agency.append(len(temp_agency))
    return week_communion,change01(week_agency),cut_words_num

'''
# 利用否定词进行分词获得communion和agency词表，和总切分词统计数
# 输入为数据集名称列表，和数据字典，输出为communion，agency，all_words三个词统计表
# 这里进行修改：不再将communion中的0值修改，只修改agency中的0值保证能除，从而使moral可以为0
def degreecal_communion_agency_list(name_list,name_dict):
    week_communion = []
    week_agency = []
    cut_words_num = []
    for j in range(0,24):
        for i in name_list:
            segwords = get_segment(name_dict[i]['%s'%j])
            nums = len(segwords)
            cut_words_num.append(nums)
            temp_communion = []
            temp_agency = []
            for m in name_dict[i]['%s'%j]:
                c,a = getlex_communion_agency(m)
                temp_communion += c
                temp_agency += a
            week_communion.append(len(temp_communion))
            week_agency.append(len(temp_agency))
    return week_communion,change01(week_agency),cut_words_num


# 注：下列数据已经完成将0值替换成1

# 利用程度词切分处理获得五种情境的communion、agency词列表,切分词总数量
#（1）week总量数据——168条【communion，agency，all_words】
week_communion,week_agency,week_all_words = degreecal_communion_agency_list(week_list,all_week_dict)
#print(week_communion[0:21])
#print(week_agency[0:21])
#（2）workdays数据——120条【communion，agency，all_workdays_words】
workdays_communion,workdays_agency,workdays_all_words = degreecal_communion_agency_list(workdays_list,workdays_dict)
#（3）weekend数据——48条【communion，agency，all_weekend_words】
weekend_communion,weekend_agency,weekend_all_words = degreecal_communion_agency_list(weekend_list,weekend_dict)
#（4）holiday数据——120条【communion，agency，all_holiday_words】
holiday_communion,holiday_agency,holiday_all_words = degreecal_communion_agency_list(holiday_list,holiday_dict)
#（5）weekend_holiday数据——168条【communion，agency，all_weekend_holiday_words】
weekend_holiday_communion,weekend_holiday_agency,weekend_holiday_all_words = degreecal_communion_agency_list(weekend_holiday_list,
                                                                                                             weekend_holiday_dict)

#print(holiday_communion)
#print(holiday_agency)
# 计算五种情境的moral值,rate_agency,rate_communion
# （1）week总量数据——168条【moral，rate_agency,rate_communion】
week_moral,week_score_agency,week_score_communion = get_score_moral(week_agency,week_communion)
#（2）workdays数据——120条【moral,rate_agency,rate_communion】
workdays_moral,workdays_score_agency,workdays_score_communion = get_score_moral(workdays_agency,workdays_communion)
#（3）weekend数据——48条【moral,rate_agency,rate_communion】
weekend_moral,weekend_score_agency,weekend_score_communion = get_score_moral(weekend_agency,weekend_communion)
#（4）holiday数据——120条【moral,rate_agency,rate_communion】
holiday_moral,holiday_score_agency,holiday_score_communion = get_score_moral(holiday_agency,holiday_communion)
#（5）weekend_holiday数据——168条【moral,rate_agency,rate_communion】
weekend_holiday_moral,weekend_holiday_score_agency,weekend_holiday_score_communion = get_score_moral(weekend_holiday_agency,
                                                                                                     weekend_holiday_communion)
#print(week_communion)
#print(week_agency)
#print(week_all_words)

In [5]:
# 五、建立八个情绪维度的统计列表（获取emotion维度数据）
# 情绪词表统计结果
# anger（249）
# anxi（111）
# expectation（91）
# hate（136）
# joy（109）
# love（144）
# sad（128）
# surprise（80）
import pandas as pd
from pyprocessmacro import Process

def pre_emodict(name_dict):
    special = []
    for i in name_dict:
        if '*' in i:
            m = i.replace('*','')
            special.append(m)
        elif '??' in i:
            n = i.replace('??','')
            special.append(n)
        elif '\ufeff' in i:
            p = i.replace('\ufeff','')
            special.append(p)
    return special 

def get_8emo_list(name_list,name_dict,emo_dict):
    emo_words = []
    emo_words_list = []
    cut_words_num = []
    for j in range(0,24):
        for i in name_list:
            segwords = get_segment(name_dict[i]['%s'%j])
            nums = len(segwords)
            cut_words_num.append(nums)
            special_words = pre_emodict(emo_dict)
            temp_words = []
            for m in segwords:
                if m in emo_dict:
                    temp_words.append(m)
                else:
                    if m[0] in special_words:
                        temp_words.append(m)
                    else:
                        if m[0:1] in special_words:
                            temp_words.append(m)
                        else:
                            if m[0:2] in special_words:
                                temp_words.append(m)
                            else:
                                if m[0:3] in special_words:
                                    temp_words.append(m)
            emo_words.append(len(temp_words))
            emo_words_list.append(temp_words)
    return emo_words,cut_words_num

In [6]:
# 不同情境分区情绪数据获取
# 获取情绪词统计列表
# （一）week
week_li_anger,_ = get_8emo_list(week_list,all_week_dict,cliwc_dict['LI_anger.txt'])
week_li_anxi,_ = get_8emo_list(week_list,all_week_dict,cliwc_dict['LI_anxi.txt'])
week_li_expectation,_ = get_8emo_list(week_list,all_week_dict,cliwc_dict['LI_expectation.txt'])
week_li_hate,_ = get_8emo_list(week_list,all_week_dict,cliwc_dict['LI_hate.txt'])
week_li_joy,_ = get_8emo_list(week_list,all_week_dict,cliwc_dict['LI_joy.txt'])
week_li_love,_ = get_8emo_list(week_list,all_week_dict,cliwc_dict['LI_love.txt'])
week_li_sad,_ = get_8emo_list(week_list,all_week_dict,cliwc_dict['LI_sad.txt'])
week_li_surprise,_ = get_8emo_list(week_list,all_week_dict,cliwc_dict['LI_surprise.txt'])

week_total_emo = [a+b+c+d+e+f+g+h for a,b,c,d,e,f,g,h in zip(week_li_anger,week_li_anxi,week_li_expectation,
                                                             week_li_hate,week_li_joy,week_li_love,
                                                             week_li_sad,week_li_surprise)]
week_positive_emo = [a+b+c for a,b,c in zip(week_li_expectation,week_li_joy,week_li_love)]
# negative情绪中去除surprise情绪，一般考虑为中性情绪
week_negative_emo = [a+b+c+d for a,b,c,d in zip(week_li_anger,week_li_anxi,week_li_hate,
                                                  week_li_sad)]

# （二）workdays
workdays_li_anger,_ = get_8emo_list(workdays_list,workdays_dict,cliwc_dict['LI_anger.txt'])
workdays_li_anxi,_ = get_8emo_list(workdays_list,workdays_dict,cliwc_dict['LI_anxi.txt'])
workdays_li_expectation,_ = get_8emo_list(workdays_list,workdays_dict,cliwc_dict['LI_expectation.txt'])
workdays_li_hate,_ = get_8emo_list(workdays_list,workdays_dict,cliwc_dict['LI_hate.txt'])
workdays_li_joy,_ = get_8emo_list(workdays_list,workdays_dict,cliwc_dict['LI_joy.txt'])
workdays_li_love,_ = get_8emo_list(workdays_list,workdays_dict,cliwc_dict['LI_love.txt'])
workdays_li_sad,_ = get_8emo_list(workdays_list,workdays_dict,cliwc_dict['LI_sad.txt'])
workdays_li_surprise,_ = get_8emo_list(workdays_list,workdays_dict,cliwc_dict['LI_surprise.txt'])

workdays_total_emo = [a+b+c+d+e+f+g+h for a,b,c,d,e,f,g,h in zip(workdays_li_anger,workdays_li_anxi,
                                                                 workdays_li_expectation,workdays_li_hate,
                                                                 workdays_li_joy,workdays_li_love,
                                                                 workdays_li_sad,workdays_li_surprise)]
workdays_positive_emo = [a+b+c for a,b,c in zip(workdays_li_expectation,workdays_li_joy,workdays_li_love)]
# negative情绪中去除surprise情绪，一般考虑为中性情绪
workdays_negative_emo = [a+b+c+d for a,b,c,d in zip(workdays_li_anger,workdays_li_anxi,workdays_li_hate,
                                                    workdays_li_sad)]

# （三）weekend
weekend_li_anger,_ = get_8emo_list(weekend_list,weekend_dict,cliwc_dict['LI_anger.txt'])
weekend_li_anxi,_ = get_8emo_list(weekend_list,weekend_dict,cliwc_dict['LI_anxi.txt'])
weekend_li_expectation,_ = get_8emo_list(weekend_list,weekend_dict,cliwc_dict['LI_expectation.txt'])
weekend_li_hate,_ = get_8emo_list(weekend_list,weekend_dict,cliwc_dict['LI_hate.txt'])
weekend_li_joy,_ = get_8emo_list(weekend_list,weekend_dict,cliwc_dict['LI_joy.txt'])
weekend_li_love,_ = get_8emo_list(weekend_list,weekend_dict,cliwc_dict['LI_love.txt'])
weekend_li_sad,_ = get_8emo_list(weekend_list,weekend_dict,cliwc_dict['LI_sad.txt'])
weekend_li_surprise,_ = get_8emo_list(weekend_list,weekend_dict,cliwc_dict['LI_surprise.txt'])

weekend_total_emo = [a+b+c+d+e+f+g+h for a,b,c,d,e,f,g,h in zip(weekend_li_anger,weekend_li_anxi,
                                                                 weekend_li_expectation,weekend_li_hate,
                                                                 weekend_li_joy,weekend_li_love,
                                                                 weekend_li_sad,weekend_li_surprise)]
weekend_positive_emo = [a+b+c for a,b,c in zip(weekend_li_expectation,weekend_li_joy,weekend_li_love)]
# negative情绪中去除surprise情绪，一般考虑为中性情绪
weekend_negative_emo = [a+b+c+d for a,b,c,d in zip(weekend_li_anger,weekend_li_anxi,weekend_li_hate,
                                                   weekend_li_sad)]

# （四）holiday
holiday_li_anger,_ = get_8emo_list(holiday_list,holiday_dict,cliwc_dict['LI_anger.txt'])
holiday_li_anxi,_ = get_8emo_list(holiday_list,holiday_dict,cliwc_dict['LI_anxi.txt'])
holiday_li_expectation,_ = get_8emo_list(holiday_list,holiday_dict,cliwc_dict['LI_expectation.txt'])
holiday_li_hate,_ = get_8emo_list(holiday_list,holiday_dict,cliwc_dict['LI_hate.txt'])
holiday_li_joy,_ = get_8emo_list(holiday_list,holiday_dict,cliwc_dict['LI_joy.txt'])
holiday_li_love,_ = get_8emo_list(holiday_list,holiday_dict,cliwc_dict['LI_love.txt'])
holiday_li_sad,_ = get_8emo_list(holiday_list,holiday_dict,cliwc_dict['LI_sad.txt'])
holiday_li_surprise,_ = get_8emo_list(holiday_list,holiday_dict,cliwc_dict['LI_surprise.txt'])

holiday_total_emo = [a+b+c+d+e+f+g+h for a,b,c,d,e,f,g,h in zip(holiday_li_anger,holiday_li_anxi,
                                                                 holiday_li_expectation,holiday_li_hate,
                                                                 holiday_li_joy,holiday_li_love,
                                                                 holiday_li_sad,holiday_li_surprise)]
holiday_positive_emo = [a+b+c for a,b,c in zip(holiday_li_expectation,holiday_li_joy,holiday_li_love)]
# negative情绪中去除surprise情绪，一般考虑为中性情绪
holiday_negative_emo = [a+b+c+d for a,b,c,d in zip(holiday_li_anger,holiday_li_anxi,holiday_li_hate,
                                                   holiday_li_sad)]

# （五）weekend & holiday
weekend_holiday_li_anger,_ = get_8emo_list(weekend_holiday_list,weekend_holiday_dict,cliwc_dict['LI_anger.txt'])
weekend_holiday_li_anxi,_ = get_8emo_list(weekend_holiday_list,weekend_holiday_dict,cliwc_dict['LI_anxi.txt'])
weekend_holiday_li_expectation,_ = get_8emo_list(weekend_holiday_list,weekend_holiday_dict,cliwc_dict['LI_expectation.txt'])
weekend_holiday_li_hate,_ = get_8emo_list(weekend_holiday_list,weekend_holiday_dict,cliwc_dict['LI_hate.txt'])
weekend_holiday_li_joy,_ = get_8emo_list(weekend_holiday_list,weekend_holiday_dict,cliwc_dict['LI_joy.txt'])
weekend_holiday_li_love,_ = get_8emo_list(weekend_holiday_list,weekend_holiday_dict,cliwc_dict['LI_love.txt'])
weekend_holiday_li_sad,_ = get_8emo_list(weekend_holiday_list,weekend_holiday_dict,cliwc_dict['LI_sad.txt'])
weekend_holiday_li_surprise,_ = get_8emo_list(weekend_holiday_list,weekend_holiday_dict,cliwc_dict['LI_surprise.txt'])

weekend_holiday_total_emo = [a+b+c+d+e+f+g+h for a,b,c,d,e,f,g,h in zip(weekend_holiday_li_anger,weekend_holiday_li_anxi,
                                                                 weekend_holiday_li_expectation,weekend_holiday_li_hate,
                                                                 weekend_holiday_li_joy,weekend_holiday_li_love,
                                                                 weekend_holiday_li_sad,weekend_holiday_li_surprise)]
weekend_holiday_positive_emo = [a+b+c for a,b,c in zip(weekend_holiday_li_expectation,weekend_holiday_li_joy,weekend_holiday_li_love)]
# negative情绪中去除surprise情绪，一般考虑为中性情绪
weekend_holiday_negative_emo = [a+b+c+d for a,b,c,d in zip(weekend_holiday_li_anger,weekend_holiday_li_anxi,weekend_holiday_li_hate,
                                                           weekend_holiday_li_sad)]

In [7]:
# （一）中介效应检测模型函数
# 采用model8，加入调节变量timezone
def test_media_process(time_group,moral_group,emo_group,timezone_group,mod,name):
    import pandas as pd
    from pyprocessmacro import Process
    # 构造数据字典
    dict_test = {'Time':time_group,'Moral':moral_group,
                 '%s'%name:emo_group,'Time_zone':timezone_group}
    # 将数据字典转变为dataframe结构
    test_data = pd.DataFrame.from_dict(dict_test)
    # 对转变数据dtype为category
    test_data['Time']=test_data['Time'].astype('category')
    test_data['Time_zone']=test_data['Time_zone'].astype('category')
    # 进行中介效应检测
    p = Process(data=test_data,model=mod,x='Time',y='Moral',w='Time_zone',m=['%s'%name])
    #p.summary()
    p.model_summary()

# 采用model4,不考虑timezone
def test_media_process2(time_group,moral_group,emo_group,mod,name):
    import pandas as pd
    from pyprocessmacro import Process
    # 构造数据字典
    dict_test = {'Time':time_group,'Moral':moral_group,
                 '%s'%name:emo_group}
    # 将数据字典转变为dataframe结构
    test_data = pd.DataFrame.from_dict(dict_test)
    # 对转变数据dtype为category
    test_data['Time']=test_data['Time'].astype('category')
    #test_data['Time_zone']=test_data['Time_zone'].astype('category')
    # 进行中介效应检测
    p = Process(data=test_data,model=mod,x='Time',y='Moral',m=['%s'%name])
    p.summary()
    


In [8]:
# 中介效应检测数据准备
# 一、week情境
# 有效道德区间
# （1）week_moral
# 3a （9-12hour，15-18hour，18-21hour，21-24hour）
week_moral_time3a_am4 = week_moral[63:84]
week_moral_time3a_pm2 = week_moral[105:126]
week_moral_time3a_pm3 = week_moral[126:147]
week_moral_time3a_pm4 = week_moral[147:168]
# 3b （10-13hour，16-19hour，22-1hour）
week_moral_time3b_am4 = week_moral[70:91]
week_moral_time3b_pm2 = week_moral[112:133]
week_moral_time3b_pm4 = week_moral[154:168]+week_moral[0:7]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour）
week_moral_time3c_am2 = week_moral[35:56]
week_moral_time3c_am3 = week_moral[56:77]
week_moral_time3c_am4 = week_moral[77:98]
week_moral_time3c_pm1 = week_moral[98:119]
week_moral_time3c_pm2 = week_moral[119:140]
week_moral_time3c_pm3 = week_moral[140:161]
# 4a （8-12hour，16-20hour，20-24hour）
week_moral_time4a_am3 = week_moral[56:84]
week_moral_time4a_pm2 = week_moral[112:140]
week_moral_time4a_pm3 = week_moral[140:168]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
week_moral_time4b_am2 = week_moral[35:63]
week_moral_time4b_am3 = week_moral[63:91]
week_moral_time4b_pm1 = week_moral[91:119]
week_moral_time4b_pm2 = week_moral[119:147]
week_moral_time4b_pm3 = week_moral[147:168] + week_moral[0:7]
# 4c （10-14hour，14-18hour，22-2hour）
week_moral_time4c_am3 = week_moral[70:98]
week_moral_time4c_pm1 = week_moral[98:126]
week_moral_time4c_pm3 = week_moral[154:168] + week_moral[0:14]
# 4d （11-15hour，15-19hour）
week_moral_time4d_am3 = week_moral[77:105]
week_moral_time4d_pm1 = week_moral[105:133]
# 6b （7-13hour，13-19hour，19-1hour）
week_moral_time6b_am2 = week_moral[49:91]
week_moral_time6b_pm1 = week_moral[91:133]
week_moral_time6b_pm2 = week_moral[133:168]+week_moral[0:7]
# 6c （8-14hour，14-20hour，20-2hour）
week_moral_time6c_am2 = week_moral[56:98]
week_moral_time6c_pm1 = week_moral[98:140]
week_moral_time6c_pm2 = week_moral[140:168]+week_moral[0:14]
# 6d （9-15hour，15-21hour）
week_moral_time6d_am2 = week_moral[63:105]
week_moral_time6d_pm1 = week_moral[105:147]

# （2）有效情绪区间（总情绪）
# 3a （9-12hour，15-18hour，18-21hour，21-24hour）
week_emo_time3a_am4 = week_total_emo[63:84]
week_emo_time3a_pm2 = week_total_emo[105:126]
week_emo_time3a_pm3 = week_total_emo[126:147]
week_emo_time3a_pm4 = week_total_emo[147:168]
# 3b （10-13hour，16-19hour，22-1hour）
week_emo_time3b_am4 = week_total_emo[70:91]
week_emo_time3b_pm2 = week_total_emo[112:133]
week_emo_time3b_pm4 = week_total_emo[154:168]+week_total_emo[0:7]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour）
week_emo_time3c_am2 = week_total_emo[35:56]
week_emo_time3c_am3 = week_total_emo[56:77]
week_emo_time3c_am4 = week_total_emo[77:98]
week_emo_time3c_pm1 = week_total_emo[98:119]
week_emo_time3c_pm2 = week_total_emo[119:140]
week_emo_time3c_pm3 = week_total_emo[140:161]
# 4a （8-12hour，16-20hour，20-24hour）
week_emo_time4a_am3 = week_total_emo[56:84]
week_emo_time4a_pm2 = week_total_emo[112:140]
week_emo_time4a_pm3 = week_total_emo[140:168]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
week_emo_time4b_am2 = week_total_emo[35:63]
week_emo_time4b_am3 = week_total_emo[63:91]
week_emo_time4b_pm1 = week_total_emo[91:119]
week_emo_time4b_pm2 = week_total_emo[119:147]
week_emo_time4b_pm3 = week_total_emo[147:168] + week_total_emo[0:7]
# 4c （10-14hour，14-18hour，22-2hour）
week_emo_time4c_am3 = week_total_emo[70:98]
week_emo_time4c_pm1 = week_total_emo[98:126]
week_emo_time4c_pm3 = week_total_emo[154:168] + week_total_emo[0:14]
# 4d （11-15hour，15-19hour）
week_emo_time4d_am3 = week_total_emo[77:105]
week_emo_time4d_pm1 = week_total_emo[105:133]
# 6b （7-13hour，13-19hour，19-1hour）
week_emo_time6b_am2 = week_total_emo[49:91]
week_emo_time6b_pm1 = week_total_emo[91:133]
week_emo_time6b_pm2 = week_total_emo[133:168]+week_total_emo[0:7]
# 6c （8-14hour，14-20hour，20-2hour）
week_emo_time6c_am2 = week_total_emo[56:98]
week_emo_time6c_pm1 = week_total_emo[98:140]
week_emo_time6c_pm2 = week_total_emo[140:168]+week_total_emo[0:14]
# 6d （9-15hour，15-21hour）
week_emo_time6d_am2 = week_total_emo[63:105]
week_emo_time6d_pm1 = week_total_emo[105:147]

# （3）有效积极情绪区间（positive）
# 3a （9-12hour，15-18hour，18-21hour，21-24hour）
week_positive_time3a_am4 = week_positive_emo[63:84]
week_positive_time3a_pm2 = week_positive_emo[105:126]
week_positive_time3a_pm3 = week_positive_emo[126:147]
week_positive_time3a_pm4 = week_positive_emo[147:168]
# 3b （10-13hour，16-19hour，22-1hour）
week_positive_time3b_am4 = week_positive_emo[70:91]
week_positive_time3b_pm2 = week_positive_emo[112:133]
week_positive_time3b_pm4 = week_positive_emo[154:168]+week_positive_emo[0:7]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour）
week_positive_time3c_am2 = week_positive_emo[35:56]
week_positive_time3c_am3 = week_positive_emo[56:77]
week_positive_time3c_am4 = week_positive_emo[77:98]
week_positive_time3c_pm1 = week_positive_emo[98:119]
week_positive_time3c_pm2 = week_positive_emo[119:140]
week_positive_time3c_pm3 = week_positive_emo[140:161]
# 4a （8-12hour，16-20hour，20-24hour）
week_positive_time4a_am3 = week_positive_emo[56:84]
week_positive_time4a_pm2 = week_positive_emo[112:140]
week_positive_time4a_pm3 = week_positive_emo[140:168]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
week_positive_time4b_am2 = week_positive_emo[35:63]
week_positive_time4b_am3 = week_positive_emo[63:91]
week_positive_time4b_pm1 = week_positive_emo[91:119]
week_positive_time4b_pm2 = week_positive_emo[119:147]
week_positive_time4b_pm3 = week_positive_emo[147:168] + week_positive_emo[0:7]
# 4c （10-14hour，14-18hour，22-2hour）
week_positive_time4c_am3 = week_positive_emo[70:98]
week_positive_time4c_pm1 = week_positive_emo[98:126]
week_positive_time4c_pm3 = week_positive_emo[154:168] + week_positive_emo[0:14]
# 4d （11-15hour，15-19hour）
week_positive_time4d_am3 = week_positive_emo[77:105]
week_positive_time4d_pm1 = week_positive_emo[105:133]
# 6b （7-13hour，13-19hour，19-1hour）
week_positive_time6b_am2 = week_positive_emo[49:91]
week_positive_time6b_pm1 = week_positive_emo[91:133]
week_positive_time6b_pm2 = week_positive_emo[133:168]+week_positive_emo[0:7]
# 6c （8-14hour，14-20hour，20-2hour）
week_positive_time6c_am2 = week_positive_emo[56:98]
week_positive_time6c_pm1 = week_positive_emo[98:140]
week_positive_time6c_pm2 = week_positive_emo[140:168]+week_positive_emo[0:14]
# 6d （9-15hour，15-21hour）
week_positive_time6d_am2 = week_positive_emo[63:105]
week_positive_time6d_pm1 = week_positive_emo[105:147]

# (4) 有效消极情绪区间（negative）
# 3a （9-12hour，15-18hour，18-21hour，21-24hour）
week_negative_time3a_am4 = week_negative_emo[63:84]
week_negative_time3a_pm2 = week_negative_emo[105:126]
week_negative_time3a_pm3 = week_negative_emo[126:147]
week_negative_time3a_pm4 = week_negative_emo[147:168]
# 3b （10-13hour，16-19hour，22-1hour）
week_negative_time3b_am4 = week_negative_emo[70:91]
week_negative_time3b_pm2 = week_negative_emo[112:133]
week_negative_time3b_pm4 = week_negative_emo[154:168]+week_negative_emo[0:7]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour）
week_negative_time3c_am2 = week_negative_emo[35:56]
week_negative_time3c_am3 = week_negative_emo[56:77]
week_negative_time3c_am4 = week_negative_emo[77:98]
week_negative_time3c_pm1 = week_negative_emo[98:119]
week_negative_time3c_pm2 = week_negative_emo[119:140]
week_negative_time3c_pm3 = week_negative_emo[140:161]
# 4a （8-12hour，16-20hour，20-24hour）
week_negative_time4a_am3 = week_negative_emo[56:84]
week_negative_time4a_pm2 = week_negative_emo[112:140]
week_negative_time4a_pm3 = week_negative_emo[140:168]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
week_negative_time4b_am2 = week_negative_emo[35:63]
week_negative_time4b_am3 = week_negative_emo[63:91]
week_negative_time4b_pm1 = week_negative_emo[91:119]
week_negative_time4b_pm2 = week_negative_emo[119:147]
week_negative_time4b_pm3 = week_negative_emo[147:168] + week_negative_emo[0:7]
# 4c （10-14hour，14-18hour，22-2hour）
week_negative_time4c_am3 = week_negative_emo[70:98]
week_negative_time4c_pm1 = week_negative_emo[98:126]
week_negative_time4c_pm3 = week_negative_emo[154:168] + week_negative_emo[0:14]
# 4d （11-15hour，15-19hour）
week_negative_time4d_am3 = week_negative_emo[77:105]
week_negative_time4d_pm1 = week_negative_emo[105:133]
# 6b （7-13hour，13-19hour，19-1hour）
week_negative_time6b_am2 = week_negative_emo[49:91]
week_negative_time6b_pm1 = week_negative_emo[91:133]
week_negative_time6b_pm2 = week_negative_emo[133:168]+week_negative_emo[0:7]
# 6c （8-14hour，14-20hour，20-2hour）
week_negative_time6c_am2 = week_negative_emo[56:98]
week_negative_time6c_pm1 = week_negative_emo[98:140]
week_negative_time6c_pm2 = week_negative_emo[140:168]+week_negative_emo[0:14]
# 6d （9-15hour，15-21hour）
week_negative_time6d_am2 = week_negative_emo[63:105]
week_negative_time6d_pm1 = week_negative_emo[105:147]

# (5) 有效anger情绪区间
# 3a （9-12hour，15-18hour，18-21hour，21-24hour）
week_anger_time3a_am4 = week_li_anger[63:84]
week_anger_time3a_pm2 = week_li_anger[105:126]
week_anger_time3a_pm3 = week_li_anger[126:147]
week_anger_time3a_pm4 = week_li_anger[147:168]
# 3b （10-13hour，16-19hour，22-1hour）
week_anger_time3b_am4 = week_li_anger[70:91]
week_anger_time3b_pm2 = week_li_anger[112:133]
week_anger_time3b_pm4 = week_li_anger[154:168]+week_li_anger[0:7]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour）
week_anger_time3c_am2 = week_li_anger[35:56]
week_anger_time3c_am3 = week_li_anger[56:77]
week_anger_time3c_am4 = week_li_anger[77:98]
week_anger_time3c_pm1 = week_li_anger[98:119]
week_anger_time3c_pm2 = week_li_anger[119:140]
week_anger_time3c_pm3 = week_li_anger[140:161]
# 4a （8-12hour，16-20hour，20-24hour）
week_anger_time4a_am3 = week_li_anger[56:84]
week_anger_time4a_pm2 = week_li_anger[112:140]
week_anger_time4a_pm3 = week_li_anger[140:168]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
week_anger_time4b_am2 = week_li_anger[35:63]
week_anger_time4b_am3 = week_li_anger[63:91]
week_anger_time4b_pm1 = week_li_anger[91:119]
week_anger_time4b_pm2 = week_li_anger[119:147]
week_anger_time4b_pm3 = week_li_anger[147:168] + week_li_anger[0:7]
# 4c （10-14hour，14-18hour，22-2hour）
week_anger_time4c_am3 = week_li_anger[70:98]
week_anger_time4c_pm1 = week_li_anger[98:126]
week_anger_time4c_pm3 = week_li_anger[154:168] + week_li_anger[0:14]
# 4d （11-15hour，15-19hour）
week_anger_time4d_am3 = week_li_anger[77:105]
week_anger_time4d_pm1 = week_li_anger[105:133]
# 6b （7-13hour，13-19hour，19-1hour）
week_anger_time6b_am2 = week_li_anger[49:91]
week_anger_time6b_pm1 = week_li_anger[91:133]
week_anger_time6b_pm2 = week_li_anger[133:168]+week_li_anger[0:7]
# 6c （8-14hour，14-20hour，20-2hour）
week_anger_time6c_am2 = week_li_anger[56:98]
week_anger_time6c_pm1 = week_li_anger[98:140]
week_anger_time6c_pm2 = week_li_anger[140:168]+week_li_anger[0:14]
# 6d （9-15hour，15-21hour）
week_anger_time6d_am2 = week_li_anger[63:105]
week_anger_time6d_pm1 = week_li_anger[105:147]

# (6) 有效anxi情绪区间
# 3a （9-12hour，15-18hour，18-21hour，21-24hour）
week_anxi_time3a_am4 = week_li_anxi[63:84]
week_anxi_time3a_pm2 = week_li_anxi[105:126]
week_anxi_time3a_pm3 = week_li_anxi[126:147]
week_anxi_time3a_pm4 = week_li_anxi[147:168]
# 3b （10-13hour，16-19hour，22-1hour）
week_anxi_time3b_am4 = week_li_anxi[70:91]
week_anxi_time3b_pm2 = week_li_anxi[112:133]
week_anxi_time3b_pm4 = week_li_anxi[154:168]+week_li_anxi[0:7]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour）
week_anxi_time3c_am2 = week_li_anxi[35:56]
week_anxi_time3c_am3 = week_li_anxi[56:77]
week_anxi_time3c_am4 = week_li_anxi[77:98]
week_anxi_time3c_pm1 = week_li_anxi[98:119]
week_anxi_time3c_pm2 = week_li_anxi[119:140]
week_anxi_time3c_pm3 = week_li_anxi[140:161]
# 4a （8-12hour，16-20hour，20-24hour）
week_anxi_time4a_am3 = week_li_anxi[56:84]
week_anxi_time4a_pm2 = week_li_anxi[112:140]
week_anxi_time4a_pm3 = week_li_anxi[140:168]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
week_anxi_time4b_am2 = week_li_anxi[35:63]
week_anxi_time4b_am3 = week_li_anxi[63:91]
week_anxi_time4b_pm1 = week_li_anxi[91:119]
week_anxi_time4b_pm2 = week_li_anxi[119:147]
week_anxi_time4b_pm3 = week_li_anxi[147:168] + week_li_anxi[0:7]
# 4c （10-14hour，14-18hour，22-2hour）
week_anxi_time4c_am3 = week_li_anxi[70:98]
week_anxi_time4c_pm1 = week_li_anxi[98:126]
week_anxi_time4c_pm3 = week_li_anxi[154:168] + week_li_anxi[0:14]
# 4d （11-15hour，15-19hour）
week_anxi_time4d_am3 = week_li_anxi[77:105]
week_anxi_time4d_pm1 = week_li_anxi[105:133]
# 6b （7-13hour，13-19hour，19-1hour）
week_anxi_time6b_am2 = week_li_anxi[49:91]
week_anxi_time6b_pm1 = week_li_anxi[91:133]
week_anxi_time6b_pm2 = week_li_anxi[133:168]+week_li_anxi[0:7]
# 6c （8-14hour，14-20hour，20-2hour）
week_anxi_time6c_am2 = week_li_anxi[56:98]
week_anxi_time6c_pm1 = week_li_anxi[98:140]
week_anxi_time6c_pm2 = week_li_anxi[140:168]+week_li_anxi[0:14]
# 6d （9-15hour，15-21hour）
week_anxi_time6d_am2 = week_li_anxi[63:105]
week_anxi_time6d_pm1 = week_li_anxi[105:147]

# (7) 有效hate情绪区间
# 3a （9-12hour，15-18hour，18-21hour，21-24hour）
week_hate_time3a_am4 = week_li_hate[63:84]
week_hate_time3a_pm2 = week_li_hate[105:126]
week_hate_time3a_pm3 = week_li_hate[126:147]
week_hate_time3a_pm4 = week_li_hate[147:168]
# 3b （10-13hour，16-19hour，22-1hour）
week_hate_time3b_am4 = week_li_hate[70:91]
week_hate_time3b_pm2 = week_li_hate[112:133]
week_hate_time3b_pm4 = week_li_hate[154:168]+week_li_hate[0:7]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour）
week_hate_time3c_am2 = week_li_hate[35:56]
week_hate_time3c_am3 = week_li_hate[56:77]
week_hate_time3c_am4 = week_li_hate[77:98]
week_hate_time3c_pm1 = week_li_hate[98:119]
week_hate_time3c_pm2 = week_li_hate[119:140]
week_hate_time3c_pm3 = week_li_hate[140:161]
# 4a （8-12hour，16-20hour，20-24hour）
week_hate_time4a_am3 = week_li_hate[56:84]
week_hate_time4a_pm2 = week_li_hate[112:140]
week_hate_time4a_pm3 = week_li_hate[140:168]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
week_hate_time4b_am2 = week_li_hate[35:63]
week_hate_time4b_am3 = week_li_hate[63:91]
week_hate_time4b_pm1 = week_li_hate[91:119]
week_hate_time4b_pm2 = week_li_hate[119:147]
week_hate_time4b_pm3 = week_li_hate[147:168] + week_li_hate[0:7]
# 4c （10-14hour，14-18hour，22-2hour）
week_hate_time4c_am3 = week_li_hate[70:98]
week_hate_time4c_pm1 = week_li_hate[98:126]
week_hate_time4c_pm3 = week_li_hate[154:168] + week_li_hate[0:14]
# 4d （11-15hour，15-19hour）
week_hate_time4d_am3 = week_li_hate[77:105]
week_hate_time4d_pm1 = week_li_hate[105:133]
# 6b （7-13hour，13-19hour，19-1hour）
week_hate_time6b_am2 = week_li_hate[49:91]
week_hate_time6b_pm1 = week_li_hate[91:133]
week_hate_time6b_pm2 = week_li_hate[133:168]+week_li_hate[0:7]
# 6c （8-14hour，14-20hour，20-2hour）
week_hate_time6c_am2 = week_li_hate[56:98]
week_hate_time6c_pm1 = week_li_hate[98:140]
week_hate_time6c_pm2 = week_li_hate[140:168]+week_li_hate[0:14]
# 6d （9-15hour，15-21hour）
week_hate_time6d_am2 = week_li_hate[63:105]
week_hate_time6d_pm1 = week_li_hate[105:147]

# (8) 有效sad情绪区间
# 3a （9-12hour，15-18hour，18-21hour，21-24hour）
week_sad_time3a_am4 = week_li_sad[63:84]
week_sad_time3a_pm2 = week_li_sad[105:126]
week_sad_time3a_pm3 = week_li_sad[126:147]
week_sad_time3a_pm4 = week_li_sad[147:168]
# 3b （10-13hour，16-19hour，22-1hour）
week_sad_time3b_am4 = week_li_sad[70:91]
week_sad_time3b_pm2 = week_li_sad[112:133]
week_sad_time3b_pm4 = week_li_sad[154:168]+week_li_sad[0:7]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour）
week_sad_time3c_am2 = week_li_sad[35:56]
week_sad_time3c_am3 = week_li_sad[56:77]
week_sad_time3c_am4 = week_li_sad[77:98]
week_sad_time3c_pm1 = week_li_sad[98:119]
week_sad_time3c_pm2 = week_li_sad[119:140]
week_sad_time3c_pm3 = week_li_sad[140:161]
# 4a （8-12hour，16-20hour，20-24hour）
week_sad_time4a_am3 = week_li_sad[56:84]
week_sad_time4a_pm2 = week_li_sad[112:140]
week_sad_time4a_pm3 = week_li_sad[140:168]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
week_sad_time4b_am2 = week_li_sad[35:63]
week_sad_time4b_am3 = week_li_sad[63:91]
week_sad_time4b_pm1 = week_li_sad[91:119]
week_sad_time4b_pm2 = week_li_sad[119:147]
week_sad_time4b_pm3 = week_li_sad[147:168] + week_li_sad[0:7]
# 4c （10-14hour，14-18hour，22-2hour）
week_sad_time4c_am3 = week_li_sad[70:98]
week_sad_time4c_pm1 = week_li_sad[98:126]
week_sad_time4c_pm3 = week_li_sad[154:168] + week_li_sad[0:14]
# 4d （11-15hour，15-19hour）
week_sad_time4d_am3 = week_li_sad[77:105]
week_sad_time4d_pm1 = week_li_sad[105:133]
# 6b （7-13hour，13-19hour，19-1hour）
week_sad_time6b_am2 = week_li_sad[49:91]
week_sad_time6b_pm1 = week_li_sad[91:133]
week_sad_time6b_pm2 = week_li_sad[133:168]+week_li_sad[0:7]
# 6c （8-14hour，14-20hour，20-2hour）
week_sad_time6c_am2 = week_li_sad[56:98]
week_sad_time6c_pm1 = week_li_sad[98:140]
week_sad_time6c_pm2 = week_li_sad[140:168]+week_li_sad[0:14]
# 6d （9-15hour，15-21hour）
week_sad_time6d_am2 = week_li_sad[63:105]
week_sad_time6d_pm1 = week_li_sad[105:147]


# (9) 有效surprise情绪区间
# 3a （9-12hour，15-18hour，18-21hour，21-24hour）
week_surprise_time3a_am4 = week_li_surprise[63:84]
week_surprise_time3a_pm2 = week_li_surprise[105:126]
week_surprise_time3a_pm3 = week_li_surprise[126:147]
week_surprise_time3a_pm4 = week_li_surprise[147:168]
# 3b （10-13hour，16-19hour，22-1hour）
week_surprise_time3b_am4 = week_li_surprise[70:91]
week_surprise_time3b_pm2 = week_li_surprise[112:133]
week_surprise_time3b_pm4 = week_li_surprise[154:168]+week_li_surprise[0:7]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour）
week_surprise_time3c_am2 = week_li_surprise[35:56]
week_surprise_time3c_am3 = week_li_surprise[56:77]
week_surprise_time3c_am4 = week_li_surprise[77:98]
week_surprise_time3c_pm1 = week_li_surprise[98:119]
week_surprise_time3c_pm2 = week_li_surprise[119:140]
week_surprise_time3c_pm3 = week_li_surprise[140:161]
# 4a （8-12hour，16-20hour，20-24hour）
week_surprise_time4a_am3 = week_li_surprise[56:84]
week_surprise_time4a_pm2 = week_li_surprise[112:140]
week_surprise_time4a_pm3 = week_li_surprise[140:168]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
week_surprise_time4b_am2 = week_li_surprise[35:63]
week_surprise_time4b_am3 = week_li_surprise[63:91]
week_surprise_time4b_pm1 = week_li_surprise[91:119]
week_surprise_time4b_pm2 = week_li_surprise[119:147]
week_surprise_time4b_pm3 = week_li_surprise[147:168] + week_li_surprise[0:7]
# 4c （10-14hour，14-18hour，22-2hour）
week_surprise_time4c_am3 = week_li_surprise[70:98]
week_surprise_time4c_pm1 = week_li_surprise[98:126]
week_surprise_time4c_pm3 = week_li_surprise[154:168] + week_li_surprise[0:14]
# 4d （11-15hour，15-19hour）
week_surprise_time4d_am3 = week_li_surprise[77:105]
week_surprise_time4d_pm1 = week_li_surprise[105:133]
# 6b （7-13hour，13-19hour，19-1hour）
week_surprise_time6b_am2 = week_li_surprise[49:91]
week_surprise_time6b_pm1 = week_li_surprise[91:133]
week_surprise_time6b_pm2 = week_li_surprise[133:168]+week_li_surprise[0:7]
# 6c （8-14hour，14-20hour，20-2hour）
week_surprise_time6c_am2 = week_li_surprise[56:98]
week_surprise_time6c_pm1 = week_li_surprise[98:140]
week_surprise_time6c_pm2 = week_li_surprise[140:168]+week_li_surprise[0:14]
# 6d （9-15hour，15-21hour）
week_surprise_time6d_am2 = week_li_surprise[63:105]
week_surprise_time6d_pm1 = week_li_surprise[105:147]


# (10) 有效love情绪区间
# 3a （9-12hour，15-18hour，18-21hour，21-24hour）
week_love_time3a_am4 = week_li_love[63:84]
week_love_time3a_pm2 = week_li_love[105:126]
week_love_time3a_pm3 = week_li_love[126:147]
week_love_time3a_pm4 = week_li_love[147:168]
# 3b （10-13hour，16-19hour，22-1hour）
week_love_time3b_am4 = week_li_love[70:91]
week_love_time3b_pm2 = week_li_love[112:133]
week_love_time3b_pm4 = week_li_love[154:168]+week_li_love[0:7]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour）
week_love_time3c_am2 = week_li_love[35:56]
week_love_time3c_am3 = week_li_love[56:77]
week_love_time3c_am4 = week_li_love[77:98]
week_love_time3c_pm1 = week_li_love[98:119]
week_love_time3c_pm2 = week_li_love[119:140]
week_love_time3c_pm3 = week_li_love[140:161]
# 4a （8-12hour，16-20hour，20-24hour）
week_love_time4a_am3 = week_li_love[56:84]
week_love_time4a_pm2 = week_li_love[112:140]
week_love_time4a_pm3 = week_li_love[140:168]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
week_love_time4b_am2 = week_li_love[35:63]
week_love_time4b_am3 = week_li_love[63:91]
week_love_time4b_pm1 = week_li_love[91:119]
week_love_time4b_pm2 = week_li_love[119:147]
week_love_time4b_pm3 = week_li_love[147:168] + week_li_love[0:7]
# 4c （10-14hour，14-18hour，22-2hour）
week_love_time4c_am3 = week_li_love[70:98]
week_love_time4c_pm1 = week_li_love[98:126]
week_love_time4c_pm3 = week_li_love[154:168] + week_li_love[0:14]
# 4d （11-15hour，15-19hour）
week_love_time4d_am3 = week_li_love[77:105]
week_love_time4d_pm1 = week_li_love[105:133]
# 6b （7-13hour，13-19hour，19-1hour）
week_love_time6b_am2 = week_li_love[49:91]
week_love_time6b_pm1 = week_li_love[91:133]
week_love_time6b_pm2 = week_li_love[133:168]+week_li_love[0:7]
# 6c （8-14hour，14-20hour，20-2hour）
week_love_time6c_am2 = week_li_love[56:98]
week_love_time6c_pm1 = week_li_love[98:140]
week_love_time6c_pm2 = week_li_love[140:168]+week_li_love[0:14]
# 6d （9-15hour，15-21hour）
week_love_time6d_am2 = week_li_love[63:105]
week_love_time6d_pm1 = week_li_love[105:147]


# (11) 有效joy情绪区间
# 3a （9-12hour，15-18hour，18-21hour，21-24hour）
week_joy_time3a_am4 = week_li_joy[63:84]
week_joy_time3a_pm2 = week_li_joy[105:126]
week_joy_time3a_pm3 = week_li_joy[126:147]
week_joy_time3a_pm4 = week_li_joy[147:168]
# 3b （10-13hour，16-19hour，22-1hour）
week_joy_time3b_am4 = week_li_joy[70:91]
week_joy_time3b_pm2 = week_li_joy[112:133]
week_joy_time3b_pm4 = week_li_joy[154:168]+week_li_joy[0:7]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour）
week_joy_time3c_am2 = week_li_joy[35:56]
week_joy_time3c_am3 = week_li_joy[56:77]
week_joy_time3c_am4 = week_li_joy[77:98]
week_joy_time3c_pm1 = week_li_joy[98:119]
week_joy_time3c_pm2 = week_li_joy[119:140]
week_joy_time3c_pm3 = week_li_joy[140:161]
# 4a （8-12hour，16-20hour，20-24hour）
week_joy_time4a_am3 = week_li_joy[56:84]
week_joy_time4a_pm2 = week_li_joy[112:140]
week_joy_time4a_pm3 = week_li_joy[140:168]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
week_joy_time4b_am2 = week_li_joy[35:63]
week_joy_time4b_am3 = week_li_joy[63:91]
week_joy_time4b_pm1 = week_li_joy[91:119]
week_joy_time4b_pm2 = week_li_joy[119:147]
week_joy_time4b_pm3 = week_li_joy[147:168] + week_li_joy[0:7]
# 4c （10-14hour，14-18hour，22-2hour）
week_joy_time4c_am3 = week_li_joy[70:98]
week_joy_time4c_pm1 = week_li_joy[98:126]
week_joy_time4c_pm3 = week_li_joy[154:168] + week_li_joy[0:14]
# 4d （11-15hour，15-19hour）
week_joy_time4d_am3 = week_li_joy[77:105]
week_joy_time4d_pm1 = week_li_joy[105:133]
# 6b （7-13hour，13-19hour，19-1hour）
week_joy_time6b_am2 = week_li_joy[49:91]
week_joy_time6b_pm1 = week_li_joy[91:133]
week_joy_time6b_pm2 = week_li_joy[133:168]+week_li_joy[0:7]
# 6c （8-14hour，14-20hour，20-2hour）
week_joy_time6c_am2 = week_li_joy[56:98]
week_joy_time6c_pm1 = week_li_joy[98:140]
week_joy_time6c_pm2 = week_li_joy[140:168]+week_li_joy[0:14]
# 6d （9-15hour，15-21hour）
week_joy_time6d_am2 = week_li_joy[63:105]
week_joy_time6d_pm1 = week_li_joy[105:147]

# (12) 有效expectation情绪区间
# 3a （9-12hour，15-18hour，18-21hour，21-24hour）
week_expectation_time3a_am4 = week_li_expectation[63:84]
week_expectation_time3a_pm2 = week_li_expectation[105:126]
week_expectation_time3a_pm3 = week_li_expectation[126:147]
week_expectation_time3a_pm4 = week_li_expectation[147:168]
# 3b （10-13hour，16-19hour，22-1hour）
week_expectation_time3b_am4 = week_li_expectation[70:91]
week_expectation_time3b_pm2 = week_li_expectation[112:133]
week_expectation_time3b_pm4 = week_li_expectation[154:168]+week_li_expectation[0:7]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour）
week_expectation_time3c_am2 = week_li_expectation[35:56]
week_expectation_time3c_am3 = week_li_expectation[56:77]
week_expectation_time3c_am4 = week_li_expectation[77:98]
week_expectation_time3c_pm1 = week_li_expectation[98:119]
week_expectation_time3c_pm2 = week_li_expectation[119:140]
week_expectation_time3c_pm3 = week_li_expectation[140:161]
# 4a （8-12hour，16-20hour，20-24hour）
week_expectation_time4a_am3 = week_li_expectation[56:84]
week_expectation_time4a_pm2 = week_li_expectation[112:140]
week_expectation_time4a_pm3 = week_li_expectation[140:168]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
week_expectation_time4b_am2 = week_li_expectation[35:63]
week_expectation_time4b_am3 = week_li_expectation[63:91]
week_expectation_time4b_pm1 = week_li_expectation[91:119]
week_expectation_time4b_pm2 = week_li_expectation[119:147]
week_expectation_time4b_pm3 = week_li_expectation[147:168] + week_li_expectation[0:7]
# 4c （10-14hour，14-18hour，22-2hour）
week_expectation_time4c_am3 = week_li_expectation[70:98]
week_expectation_time4c_pm1 = week_li_expectation[98:126]
week_expectation_time4c_pm3 = week_li_expectation[154:168] + week_li_expectation[0:14]
# 4d （11-15hour，15-19hour）
week_expectation_time4d_am3 = week_li_expectation[77:105]
week_expectation_time4d_pm1 = week_li_expectation[105:133]
# 6b （7-13hour，13-19hour，19-1hour）
week_expectation_time6b_am2 = week_li_expectation[49:91]
week_expectation_time6b_pm1 = week_li_expectation[91:133]
week_expectation_time6b_pm2 = week_li_expectation[133:168]+week_li_expectation[0:7]
# 6c （8-14hour，14-20hour，20-2hour）
week_expectation_time6c_am2 = week_li_expectation[56:98]
week_expectation_time6c_pm1 = week_li_expectation[98:140]
week_expectation_time6c_pm2 = week_li_expectation[140:168]+week_li_expectation[0:14]
# 6d （9-15hour，15-21hour）
week_expectation_time6d_am2 = week_li_expectation[63:105]
week_expectation_time6d_pm1 = week_li_expectation[105:147]


# 有效道德区间数据汇总
week_moral_group = [week_moral_time3a_am4+week_moral_time3a_pm2+week_moral_time3a_pm3+week_moral_time3a_pm4+
                     week_moral_time3b_am4+week_moral_time3b_pm2+week_moral_time3b_pm4+
                     week_moral_time3c_am2+week_moral_time3c_am3+week_moral_time3c_am4+
                     week_moral_time3c_pm1+week_moral_time3c_pm2+week_moral_time3c_pm3+
                     week_moral_time4a_am3+week_moral_time4a_pm2+week_moral_time4a_pm3+
                     week_moral_time4b_am2+week_moral_time4b_am3+week_moral_time4b_pm1+
                     week_moral_time4b_pm2+week_moral_time4b_pm3+week_moral_time4c_am3+
                     week_moral_time4c_pm1+week_moral_time4c_pm3+week_moral_time4d_am3+
                     week_moral_time4d_pm1+week_moral_time6b_am2+week_moral_time6b_pm1+
                     week_moral_time6b_pm2+week_moral_time6c_am2+week_moral_time6c_pm1+
                     week_moral_time6c_pm2+week_moral_time6d_am2+week_moral_time6d_pm1]

# 有效情绪区间数据汇总（总情绪）
week_emo_group = [week_emo_time3a_am4+week_emo_time3a_pm2+week_emo_time3a_pm3+week_emo_time3a_pm4+
                     week_emo_time3b_am4+week_emo_time3b_pm2+week_emo_time3b_pm4+
                     week_emo_time3c_am2+week_emo_time3c_am3+week_emo_time3c_am4+
                     week_emo_time3c_pm1+week_emo_time3c_pm2+week_emo_time3c_pm3+
                     week_emo_time4a_am3+week_emo_time4a_pm2+week_emo_time4a_pm3+
                     week_emo_time4b_am2+week_emo_time4b_am3+week_emo_time4b_pm1+
                     week_emo_time4b_pm2+week_emo_time4b_pm3+week_emo_time4c_am3+
                     week_emo_time4c_pm1+week_emo_time4c_pm3+week_emo_time4d_am3+
                     week_emo_time4d_pm1+week_emo_time6b_am2+week_emo_time6b_pm1+
                     week_emo_time6b_pm2+week_emo_time6c_am2+week_emo_time6c_pm1+
                     week_emo_time6c_pm2+week_emo_time6d_am2+week_emo_time6d_pm1]

# 有效情绪区间数据汇总（积极情绪positive）
week_positive_group = [week_positive_time3a_am4+week_positive_time3a_pm2+week_positive_time3a_pm3+
                        week_positive_time3a_pm4+week_positive_time3b_am4+week_positive_time3b_pm2+
                        week_positive_time3b_pm4+week_positive_time3c_am2+week_positive_time3c_am3+
                        week_positive_time3c_am4+week_positive_time3c_pm1+week_positive_time3c_pm2+
                        week_positive_time3c_pm3+week_positive_time4a_am3+week_positive_time4a_pm2+
                        week_positive_time4a_pm3+week_positive_time4b_am2+week_positive_time4b_am3+
                        week_positive_time4b_pm1+week_positive_time4b_pm2+week_positive_time4b_pm3+
                        week_positive_time4c_am3+week_positive_time4c_pm1+week_positive_time4c_pm3+
                        week_positive_time4d_am3+week_positive_time4d_pm1+week_positive_time6b_am2+
                        week_positive_time6b_pm1+week_positive_time6b_pm2+week_positive_time6c_am2+
                        week_positive_time6c_pm1+week_positive_time6c_pm2+week_positive_time6d_am2+
                        week_positive_time6d_pm1]

# 有效情绪区间数据汇总（消极情绪negative）
week_negative_group = [week_negative_time3a_am4+week_negative_time3a_pm2+week_negative_time3a_pm3+
                        week_negative_time3a_pm4+week_negative_time3b_am4+week_negative_time3b_pm2+
                        week_negative_time3b_pm4+week_negative_time3c_am2+week_negative_time3c_am3+
                        week_negative_time3c_am4+week_negative_time3c_pm1+week_negative_time3c_pm2+
                        week_negative_time3c_pm3+week_negative_time4a_am3+week_negative_time4a_pm2+
                        week_negative_time4a_pm3+week_negative_time4b_am2+week_negative_time4b_am3+
                        week_negative_time4b_pm1+week_negative_time4b_pm2+week_negative_time4b_pm3+
                        week_negative_time4c_am3+week_negative_time4c_pm1+week_negative_time4c_pm3+
                        week_negative_time4d_am3+week_negative_time4d_pm1+week_negative_time6b_am2+
                        week_negative_time6b_pm1+week_negative_time6b_pm2+week_negative_time6c_am2+
                        week_negative_time6c_pm1+week_negative_time6c_pm2+week_negative_time6d_am2+
                        week_negative_time6d_pm1]

# 有效anger情绪区间数据汇总
week_anger_group = [week_anger_time3a_am4+week_anger_time3a_pm2+week_anger_time3a_pm3+
                        week_anger_time3a_pm4+week_anger_time3b_am4+week_anger_time3b_pm2+
                        week_anger_time3b_pm4+week_anger_time3c_am2+week_anger_time3c_am3+
                        week_anger_time3c_am4+week_anger_time3c_pm1+week_anger_time3c_pm2+
                        week_anger_time3c_pm3+week_anger_time4a_am3+week_anger_time4a_pm2+
                        week_anger_time4a_pm3+week_anger_time4b_am2+week_anger_time4b_am3+
                        week_anger_time4b_pm1+week_anger_time4b_pm2+week_anger_time4b_pm3+
                        week_anger_time4c_am3+week_anger_time4c_pm1+week_anger_time4c_pm3+
                        week_anger_time4d_am3+week_anger_time4d_pm1+week_anger_time6b_am2+
                        week_anger_time6b_pm1+week_anger_time6b_pm2+week_anger_time6c_am2+
                        week_anger_time6c_pm1+week_anger_time6c_pm2+week_anger_time6d_am2+
                        week_anger_time6d_pm1]

# 有效anxi情绪区间数据汇总
week_anxi_group = [week_anxi_time3a_am4+week_anxi_time3a_pm2+week_anxi_time3a_pm3+
                        week_anxi_time3a_pm4+week_anxi_time3b_am4+week_anxi_time3b_pm2+
                        week_anxi_time3b_pm4+week_anxi_time3c_am2+week_anxi_time3c_am3+
                        week_anxi_time3c_am4+week_anxi_time3c_pm1+week_anxi_time3c_pm2+
                        week_anxi_time3c_pm3+week_anxi_time4a_am3+week_anxi_time4a_pm2+
                        week_anxi_time4a_pm3+week_anxi_time4b_am2+week_anxi_time4b_am3+
                        week_anxi_time4b_pm1+week_anxi_time4b_pm2+week_anxi_time4b_pm3+
                        week_anxi_time4c_am3+week_anxi_time4c_pm1+week_anxi_time4c_pm3+
                        week_anxi_time4d_am3+week_anxi_time4d_pm1+week_anxi_time6b_am2+
                        week_anxi_time6b_pm1+week_anxi_time6b_pm2+week_anxi_time6c_am2+
                        week_anxi_time6c_pm1+week_anxi_time6c_pm2+week_anxi_time6d_am2+
                        week_anxi_time6d_pm1]

# 有效hate情绪区间数据汇总
week_hate_group = [week_hate_time3a_am4+week_hate_time3a_pm2+week_hate_time3a_pm3+
                        week_hate_time3a_pm4+week_hate_time3b_am4+week_hate_time3b_pm2+
                        week_hate_time3b_pm4+week_hate_time3c_am2+week_hate_time3c_am3+
                        week_hate_time3c_am4+week_hate_time3c_pm1+week_hate_time3c_pm2+
                        week_hate_time3c_pm3+week_hate_time4a_am3+week_hate_time4a_pm2+
                        week_hate_time4a_pm3+week_hate_time4b_am2+week_hate_time4b_am3+
                        week_hate_time4b_pm1+week_hate_time4b_pm2+week_hate_time4b_pm3+
                        week_hate_time4c_am3+week_hate_time4c_pm1+week_hate_time4c_pm3+
                        week_hate_time4d_am3+week_hate_time4d_pm1+week_hate_time6b_am2+
                        week_hate_time6b_pm1+week_hate_time6b_pm2+week_hate_time6c_am2+
                        week_hate_time6c_pm1+week_hate_time6c_pm2+week_hate_time6d_am2+
                        week_hate_time6d_pm1]

# 有效sad情绪区间数据汇总
week_sad_group = [week_sad_time3a_am4+week_sad_time3a_pm2+week_sad_time3a_pm3+
                        week_sad_time3a_pm4+week_sad_time3b_am4+week_sad_time3b_pm2+
                        week_sad_time3b_pm4+week_sad_time3c_am2+week_sad_time3c_am3+
                        week_sad_time3c_am4+week_sad_time3c_pm1+week_sad_time3c_pm2+
                        week_sad_time3c_pm3+week_sad_time4a_am3+week_sad_time4a_pm2+
                        week_sad_time4a_pm3+week_sad_time4b_am2+week_sad_time4b_am3+
                        week_sad_time4b_pm1+week_sad_time4b_pm2+week_sad_time4b_pm3+
                        week_sad_time4c_am3+week_sad_time4c_pm1+week_sad_time4c_pm3+
                        week_sad_time4d_am3+week_sad_time4d_pm1+week_sad_time6b_am2+
                        week_sad_time6b_pm1+week_sad_time6b_pm2+week_sad_time6c_am2+
                        week_sad_time6c_pm1+week_sad_time6c_pm2+week_sad_time6d_am2+
                        week_sad_time6d_pm1]

# 有效sad情绪区间数据汇总
week_surprise_group = [week_surprise_time3a_am4+week_surprise_time3a_pm2+week_surprise_time3a_pm3+
                        week_surprise_time3a_pm4+week_surprise_time3b_am4+week_surprise_time3b_pm2+
                        week_surprise_time3b_pm4+week_surprise_time3c_am2+week_surprise_time3c_am3+
                        week_surprise_time3c_am4+week_surprise_time3c_pm1+week_surprise_time3c_pm2+
                        week_surprise_time3c_pm3+week_surprise_time4a_am3+week_surprise_time4a_pm2+
                        week_surprise_time4a_pm3+week_surprise_time4b_am2+week_surprise_time4b_am3+
                        week_surprise_time4b_pm1+week_surprise_time4b_pm2+week_surprise_time4b_pm3+
                        week_surprise_time4c_am3+week_surprise_time4c_pm1+week_surprise_time4c_pm3+
                        week_surprise_time4d_am3+week_surprise_time4d_pm1+week_surprise_time6b_am2+
                        week_surprise_time6b_pm1+week_surprise_time6b_pm2+week_surprise_time6c_am2+
                        week_surprise_time6c_pm1+week_surprise_time6c_pm2+week_surprise_time6d_am2+
                        week_surprise_time6d_pm1]


# 有效love情绪区间数据汇总
week_love_group = [week_love_time3a_am4+week_love_time3a_pm2+week_love_time3a_pm3+
                        week_love_time3a_pm4+week_love_time3b_am4+week_love_time3b_pm2+
                        week_love_time3b_pm4+week_love_time3c_am2+week_love_time3c_am3+
                        week_love_time3c_am4+week_love_time3c_pm1+week_love_time3c_pm2+
                        week_love_time3c_pm3+week_love_time4a_am3+week_love_time4a_pm2+
                        week_love_time4a_pm3+week_love_time4b_am2+week_love_time4b_am3+
                        week_love_time4b_pm1+week_love_time4b_pm2+week_love_time4b_pm3+
                        week_love_time4c_am3+week_love_time4c_pm1+week_love_time4c_pm3+
                        week_love_time4d_am3+week_love_time4d_pm1+week_love_time6b_am2+
                        week_love_time6b_pm1+week_love_time6b_pm2+week_love_time6c_am2+
                        week_love_time6c_pm1+week_love_time6c_pm2+week_love_time6d_am2+
                        week_love_time6d_pm1]

# 有效joy情绪区间数据汇总
week_joy_group = [week_joy_time3a_am4+week_joy_time3a_pm2+week_joy_time3a_pm3+
                        week_joy_time3a_pm4+week_joy_time3b_am4+week_joy_time3b_pm2+
                        week_joy_time3b_pm4+week_joy_time3c_am2+week_joy_time3c_am3+
                        week_joy_time3c_am4+week_joy_time3c_pm1+week_joy_time3c_pm2+
                        week_joy_time3c_pm3+week_joy_time4a_am3+week_joy_time4a_pm2+
                        week_joy_time4a_pm3+week_joy_time4b_am2+week_joy_time4b_am3+
                        week_joy_time4b_pm1+week_joy_time4b_pm2+week_joy_time4b_pm3+
                        week_joy_time4c_am3+week_joy_time4c_pm1+week_joy_time4c_pm3+
                        week_joy_time4d_am3+week_joy_time4d_pm1+week_joy_time6b_am2+
                        week_joy_time6b_pm1+week_joy_time6b_pm2+week_joy_time6c_am2+
                        week_joy_time6c_pm1+week_joy_time6c_pm2+week_joy_time6d_am2+
                        week_joy_time6d_pm1]

# 有效expectation情绪区间数据汇总
week_expectation_group = [week_expectation_time3a_am4+week_expectation_time3a_pm2+week_expectation_time3a_pm3+
                        week_expectation_time3a_pm4+week_expectation_time3b_am4+week_expectation_time3b_pm2+
                        week_expectation_time3b_pm4+week_expectation_time3c_am2+week_expectation_time3c_am3+
                        week_expectation_time3c_am4+week_expectation_time3c_pm1+week_expectation_time3c_pm2+
                        week_expectation_time3c_pm3+week_expectation_time4a_am3+week_expectation_time4a_pm2+
                        week_expectation_time4a_pm3+week_expectation_time4b_am2+week_expectation_time4b_am3+
                        week_expectation_time4b_pm1+week_expectation_time4b_pm2+week_expectation_time4b_pm3+
                        week_expectation_time4c_am3+week_expectation_time4c_pm1+week_expectation_time4c_pm3+
                        week_expectation_time4d_am3+week_expectation_time4d_pm1+week_expectation_time6b_am2+
                        week_expectation_time6b_pm1+week_expectation_time6b_pm2+week_expectation_time6c_am2+
                        week_expectation_time6c_pm1+week_expectation_time6c_pm2+week_expectation_time6d_am2+
                        week_expectation_time6d_pm1]

# 有效时间标签构造（Morning=1,Afternoon=0,Night=-1）
# 注，需要在数据结构中改为category类型
week_time_group = [21*[1]+21*[0]+21*[0]+21*[-1]+
                    21*[1]+21*[0]+21*[-1]+
                    21*[1]+21*[1]+21*[1]+21*[0]+21*[0]+21*[-1]+
                    28*[1]+28*[0]+28*[-1]+
                    28*[1]+28*[1]+28*[0]+28*[0]+28*[-1]+
                    28*[1]+28*[0]+28*[-1]+
                    28*[1]+28*[0]+
                    42*[1]+42*[0]+42*[0]+
                    42*[1]+42*[0]+42*[-1]+
                    42*[1]+42*[0]]

# 有效时区长度标签构造（三小时时区=3，四小时时区=4，六小时时区=6）
# 注，需要在数据结构中改为category类型
week_zone_group = [273*[3]+364*[4]+336*[6]]


In [9]:
# week中介效应测试结果
# 总体情绪——道德动机
week_emo_to_moral = test_media_process2(week_time_group[0],week_moral_group[0],
                                   week_emo_group[0],4,'Total_emotion(week)')
# 积极情绪——道德动机
week_positive_to_moral = test_media_process2(week_time_group[0],week_moral_group[0],
                                        week_positive_group[0],4,'Positive(week)')
# 消极情绪——道德动机
week_negative_to_moral = test_media_process2(week_time_group[0],week_moral_group[0],
                                        week_negative_group[0],4,'Negative(week)')

#*******************************************************************************
# anger——道德动机
week_anger_to_moral = test_media_process2(week_time_group[0],week_moral_group[0],
                                     week_anger_group[0],4,'Anger(week)')
# anxi——道德动机
week_anxi_to_moral = test_media_process2(week_time_group[0],week_moral_group[0],
                                     week_anxi_group[0],4,'Anxi(week)')
# hate——道德动机
week_hate_to_moral = test_media_process2(week_time_group[0],week_moral_group[0],
                                     week_hate_group[0],4,'Hate(week)')
# sad——道德动机
week_sad_to_moral = test_media_process2(week_time_group[0],week_moral_group[0],
                                     week_sad_group[0],4,'Sad(week)')
# surprise——道德动机
week_surprise_to_moral = test_media_process2(week_time_group[0],week_moral_group[0],
                                     week_surprise_group[0],4,'Surprise(week)')

#*******************************************************************************
# love——道德动机
week_love_to_moral = test_media_process2(week_time_group[0],week_moral_group[0],
                                     week_love_group[0],4,'Love(week)')
# joy——道德动机
week_joy_to_moral = test_media_process2(week_time_group[0],week_moral_group[0],
                                     week_joy_group[0],4,'Joy(week)')
# expectation——道德动机
week_expectation_to_moral = test_media_process2(week_time_group[0],week_moral_group[0],
                                     week_expectation_group[0],4,'Expectation(week)')

Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 4

Variables:
    Cons = Cons
    x = Time
    y = Moral
    m1 = Total_emotion(week)

Sample size:
973

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = Moral 
OLS Regression Summary

     R²  Adj. R²    MSE       F  df1  df2  p-value
 0.0239   0.0209 0.0531 11.8813    2  970   0.0000

Coefficients

                      coeff     se       t      p    LLCI    ULCI
Cons                 0.9737 0.0198 49.2694 0.0000  0.9349  1.0124
Time                -0.0213 0.0106 -2.0121 0.0445 -0.0420 -0.0006
Total_emotion(week)  0.0012 0.0003  3.5903 0.0003  0.0005  0.0019

-------------------------------------------------------------

Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 4

Variables:
    Cons = Cons
    x = Time
    y = Moral
    m1 = Hate(week)

Sample size:
973

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = Moral 
OLS Regression Summary

     R²  Adj. R²    MSE      F  df1  df2  p-value
 0.0154   0.0123 0.0536 7.5680    2  970   0.0005

Coefficients

             coeff     se        t      p    LLCI    ULCI
Cons        1.0250 0.0102 100.4969 0.0000  1.0051  1.0450
Time       -0.0280 0.0104  -2.6947 0.0072 -0.0484 -0.0076
Hate(week)  0.0167 0.0080   2.0876 0.0371  0.0010  0.0324

-------------------------------------------------------------------------

Outcome = Hate(week) 
OLS Reg

Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 4

Variables:
    Cons = Cons
    x = Time
    y = Moral
    m1 = Expectation(week)

Sample size:
973

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = Moral 
OLS Regression Summary

     R²  Adj. R²    MSE      F  df1  df2  p-value
 0.0110   0.0079 0.0538 5.3981    2  970   0.0047

Coefficients

                    coeff     se       t      p    LLCI    ULCI
Cons               1.0418 0.0129 80.5428 0.0000  1.0164  1.0671
Time              -0.0335 0.0102 -3.2779 0.0011 -0.0536 -0.0135
Expectation(week) -0.0010 0.0038 -0.2563 0.7978 -0.0083  0.0064

-------------------------------------------------------------------------

In [10]:
# 中介效应检测数据准备
# 二、workdays情境
# 有效道德区间
# （1）workdays_moral
# 3a （9-12hour，15-18hour，21-24hour）
workdays_moral_time3a_am4 = workdays_moral[45:60]
workdays_moral_time3a_pm2 = workdays_moral[75:90]
workdays_moral_time3a_pm4 = workdays_moral[105:120]
# 3b （7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
workdays_moral_time3b_am3 = workdays_moral[35:50]
workdays_moral_time3b_am4 = workdays_moral[50:65]
workdays_moral_time3b_pm1 = workdays_moral[65:80]
workdays_moral_time3b_pm2 = workdays_moral[80:95]
workdays_moral_time3b_pm3 = workdays_moral[95:110]
workdays_moral_time3b_pm4 = workdays_moral[110:120]+workdays_moral[0:5]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
workdays_moral_time3c_am2 = workdays_moral[25:40]
workdays_moral_time3c_am3 = workdays_moral[40:55]
workdays_moral_time3c_am4 = workdays_moral[55:70]
workdays_moral_time3c_pm1 = workdays_moral[70:85]
workdays_moral_time3c_pm2 = workdays_moral[85:100]
workdays_moral_time3c_pm3 = workdays_moral[100:115]
workdays_moral_time3c_pm4 = workdays_moral[115:120]+workdays_moral[0:10]
# 4a （8-12hour，16-20hour，20-24hour）
workdays_moral_time4a_am3 = workdays_moral[40:60]
workdays_moral_time4a_pm2 = workdays_moral[80:100]
workdays_moral_time4a_pm3 = workdays_moral[100:120]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
workdays_moral_time4b_am2 = workdays_moral[25:45]
workdays_moral_time4b_am3 = workdays_moral[45:65]
workdays_moral_time4b_pm1 = workdays_moral[65:85]
workdays_moral_time4b_pm2 = workdays_moral[85:105]
workdays_moral_time4b_pm3 = workdays_moral[105:120] + workdays_moral[0:5]
# 4c （6-10hour，10-14hour，14-18hour，22-2hour）
workdays_moral_time4c_am2 = workdays_moral[30:50]
workdays_moral_time4c_am3 = workdays_moral[50:70]
workdays_moral_time4c_pm1 = workdays_moral[70:90]
workdays_moral_time4c_pm3 = workdays_moral[110:120] + workdays_moral[0:10]
# 4d （7-11hour，11-15hour，15-19hour，19-23hour）
workdays_moral_time4d_am2 = workdays_moral[35:55]
workdays_moral_time4d_am3 = workdays_moral[55:75]
workdays_moral_time4d_pm1 = workdays_moral[75:95]
workdays_moral_time4d_pm2 = workdays_moral[95:115]
# 6a （6-12hour，12-18hour）
workdays_moral_time6a_am2 = workdays_moral[30:60]
workdays_moral_time6a_pm1 = workdays_moral[60:90]
# 6b （7-13hour，13-19hour，19-1hour）
workdays_moral_time6b_am2 = workdays_moral[35:65]
workdays_moral_time6b_pm1 = workdays_moral[65:95]
workdays_moral_time6b_pm2 = workdays_moral[95:120]+workdays_moral[0:5]
# 6c （8-14hour，14-20hour，20-2hour）
workdays_moral_time6c_am2 = workdays_moral[40:70]
workdays_moral_time6c_pm1 = workdays_moral[70:100]
workdays_moral_time6c_pm2 = workdays_moral[100:120]+workdays_moral[0:10]
# 6e （4-10hour，16-22hour）
workdays_moral_time6e_am1 = workdays_moral[20:50]
workdays_moral_time6e_pm1 = workdays_moral[80:110]
# 6f （5-11hour，11-17hour，17-23hour）
workdays_moral_time6f_am1 = workdays_moral[25:55]
workdays_moral_time6f_am2 = workdays_moral[55:85]
workdays_moral_time6f_pm1 = workdays_moral[85:115]

# （2）有效情绪区间（总情绪）
# 3a （9-12hour，15-18hour，21-24hour）
workdays_emo_time3a_am4 = workdays_total_emo[45:60]
workdays_emo_time3a_pm2 = workdays_total_emo[75:90]
workdays_emo_time3a_pm4 = workdays_total_emo[105:120]
# 3b （7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
workdays_emo_time3b_am3 = workdays_total_emo[35:50]
workdays_emo_time3b_am4 = workdays_total_emo[50:65]
workdays_emo_time3b_pm1 = workdays_total_emo[65:80]
workdays_emo_time3b_pm2 = workdays_total_emo[80:95]
workdays_emo_time3b_pm3 = workdays_total_emo[95:110]
workdays_emo_time3b_pm4 = workdays_total_emo[110:120]+workdays_total_emo[0:5]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
workdays_emo_time3c_am2 = workdays_total_emo[25:40]
workdays_emo_time3c_am3 = workdays_total_emo[40:55]
workdays_emo_time3c_am4 = workdays_total_emo[55:70]
workdays_emo_time3c_pm1 = workdays_total_emo[70:85]
workdays_emo_time3c_pm2 = workdays_total_emo[85:100]
workdays_emo_time3c_pm3 = workdays_total_emo[100:115]
workdays_emo_time3c_pm4 = workdays_total_emo[115:120]+workdays_total_emo[0:10]
# 4a （8-12hour，16-20hour，20-24hour）
workdays_emo_time4a_am3 = workdays_total_emo[40:60]
workdays_emo_time4a_pm2 = workdays_total_emo[80:100]
workdays_emo_time4a_pm3 = workdays_total_emo[100:120]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
workdays_emo_time4b_am2 = workdays_total_emo[25:45]
workdays_emo_time4b_am3 = workdays_total_emo[45:65]
workdays_emo_time4b_pm1 = workdays_total_emo[65:85]
workdays_emo_time4b_pm2 = workdays_total_emo[85:105]
workdays_emo_time4b_pm3 = workdays_total_emo[105:120] + workdays_total_emo[0:5]
# 4c （6-10hour，10-14hour，14-18hour，22-2hour）
workdays_emo_time4c_am2 = workdays_total_emo[30:50]
workdays_emo_time4c_am3 = workdays_total_emo[50:70]
workdays_emo_time4c_pm1 = workdays_total_emo[70:90]
workdays_emo_time4c_pm3 = workdays_total_emo[110:120] + workdays_total_emo[0:10]
# 4d （7-11hour，11-15hour，15-19hour，19-23hour）
workdays_emo_time4d_am2 = workdays_total_emo[35:55]
workdays_emo_time4d_am3 = workdays_total_emo[55:75]
workdays_emo_time4d_pm1 = workdays_total_emo[75:95]
workdays_emo_time4d_pm2 = workdays_total_emo[95:115]
# 6a （6-12hour，12-18hour）
workdays_emo_time6a_am2 = workdays_total_emo[30:60]
workdays_emo_time6a_pm1 = workdays_total_emo[60:90]
# 6b （7-13hour，13-19hour，19-1hour）
workdays_emo_time6b_am2 = workdays_total_emo[35:65]
workdays_emo_time6b_pm1 = workdays_total_emo[65:95]
workdays_emo_time6b_pm2 = workdays_total_emo[95:120]+workdays_total_emo[0:5]
# 6c （8-14hour，14-20hour，20-2hour）
workdays_emo_time6c_am2 = workdays_total_emo[40:70]
workdays_emo_time6c_pm1 = workdays_total_emo[70:100]
workdays_emo_time6c_pm2 = workdays_total_emo[100:120]+workdays_total_emo[0:10]
# 6e （4-10hour，16-22hour）
workdays_emo_time6e_am1 = workdays_total_emo[20:50]
workdays_emo_time6e_pm1 = workdays_total_emo[80:110]
# 6f （5-11hour，11-17hour，17-23hour）
workdays_emo_time6f_am1 = workdays_total_emo[25:55]
workdays_emo_time6f_am2 = workdays_total_emo[55:85]
workdays_emo_time6f_pm1 = workdays_total_emo[85:115]

# （3）有效积极情绪区间（positive）
# 3a （9-12hour，15-18hour，21-24hour）
workdays_positive_time3a_am4 = workdays_positive_emo[45:60]
workdays_positive_time3a_pm2 = workdays_positive_emo[75:90]
workdays_positive_time3a_pm4 = workdays_positive_emo[105:120]
# 3b （7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
workdays_positive_time3b_am3 = workdays_positive_emo[35:50]
workdays_positive_time3b_am4 = workdays_positive_emo[50:65]
workdays_positive_time3b_pm1 = workdays_positive_emo[65:80]
workdays_positive_time3b_pm2 = workdays_positive_emo[80:95]
workdays_positive_time3b_pm3 = workdays_positive_emo[95:110]
workdays_positive_time3b_pm4 = workdays_positive_emo[110:120]+workdays_positive_emo[0:5]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
workdays_positive_time3c_am2 = workdays_positive_emo[25:40]
workdays_positive_time3c_am3 = workdays_positive_emo[40:55]
workdays_positive_time3c_am4 = workdays_positive_emo[55:70]
workdays_positive_time3c_pm1 = workdays_positive_emo[70:85]
workdays_positive_time3c_pm2 = workdays_positive_emo[85:100]
workdays_positive_time3c_pm3 = workdays_positive_emo[100:115]
workdays_positive_time3c_pm4 = workdays_positive_emo[115:120]+workdays_positive_emo[0:10]
# 4a （8-12hour，16-20hour，20-24hour）
workdays_positive_time4a_am3 = workdays_positive_emo[40:60]
workdays_positive_time4a_pm2 = workdays_positive_emo[80:100]
workdays_positive_time4a_pm3 = workdays_positive_emo[100:120]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
workdays_positive_time4b_am2 = workdays_positive_emo[25:45]
workdays_positive_time4b_am3 = workdays_positive_emo[45:65]
workdays_positive_time4b_pm1 = workdays_positive_emo[65:85]
workdays_positive_time4b_pm2 = workdays_positive_emo[85:105]
workdays_positive_time4b_pm3 = workdays_positive_emo[105:120] + workdays_positive_emo[0:5]
# 4c （6-10hour，10-14hour，14-18hour，22-2hour）
workdays_positive_time4c_am2 = workdays_positive_emo[30:50]
workdays_positive_time4c_am3 = workdays_positive_emo[50:70]
workdays_positive_time4c_pm1 = workdays_positive_emo[70:90]
workdays_positive_time4c_pm3 = workdays_positive_emo[110:120] + workdays_positive_emo[0:10]
# 4d （7-11hour，11-15hour，15-19hour，19-23hour）
workdays_positive_time4d_am2 = workdays_positive_emo[35:55]
workdays_positive_time4d_am3 = workdays_positive_emo[55:75]
workdays_positive_time4d_pm1 = workdays_positive_emo[75:95]
workdays_positive_time4d_pm2 = workdays_positive_emo[95:115]
# 6a （6-12hour，12-18hour）
workdays_positive_time6a_am2 = workdays_positive_emo[30:60]
workdays_positive_time6a_pm1 = workdays_positive_emo[60:90]
# 6b （7-13hour，13-19hour，19-1hour）
workdays_positive_time6b_am2 = workdays_positive_emo[35:65]
workdays_positive_time6b_pm1 = workdays_positive_emo[65:95]
workdays_positive_time6b_pm2 = workdays_positive_emo[95:120]+workdays_positive_emo[0:5]
# 6c （8-14hour，14-20hour，20-2hour）
workdays_positive_time6c_am2 = workdays_positive_emo[40:70]
workdays_positive_time6c_pm1 = workdays_positive_emo[70:100]
workdays_positive_time6c_pm2 = workdays_positive_emo[100:120]+workdays_positive_emo[0:10]
# 6e （4-10hour，16-22hour）
workdays_positive_time6e_am1 = workdays_positive_emo[20:50]
workdays_positive_time6e_pm1 = workdays_positive_emo[80:110]
# 6f （5-11hour，11-17hour，17-23hour）
workdays_positive_time6f_am1 = workdays_positive_emo[25:55]
workdays_positive_time6f_am2 = workdays_positive_emo[55:85]
workdays_positive_time6f_pm1 = workdays_positive_emo[85:115]


# （4）有效消极情绪区间（negative）
# 3a （9-12hour，15-18hour，21-24hour）
workdays_negative_time3a_am4 = workdays_negative_emo[45:60]
workdays_negative_time3a_pm2 = workdays_negative_emo[75:90]
workdays_negative_time3a_pm4 = workdays_negative_emo[105:120]
# 3b （7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
workdays_negative_time3b_am3 = workdays_negative_emo[35:50]
workdays_negative_time3b_am4 = workdays_negative_emo[50:65]
workdays_negative_time3b_pm1 = workdays_negative_emo[65:80]
workdays_negative_time3b_pm2 = workdays_negative_emo[80:95]
workdays_negative_time3b_pm3 = workdays_negative_emo[95:110]
workdays_negative_time3b_pm4 = workdays_negative_emo[110:120]+workdays_negative_emo[0:5]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
workdays_negative_time3c_am2 = workdays_negative_emo[25:40]
workdays_negative_time3c_am3 = workdays_negative_emo[40:55]
workdays_negative_time3c_am4 = workdays_negative_emo[55:70]
workdays_negative_time3c_pm1 = workdays_negative_emo[70:85]
workdays_negative_time3c_pm2 = workdays_negative_emo[85:100]
workdays_negative_time3c_pm3 = workdays_negative_emo[100:115]
workdays_negative_time3c_pm4 = workdays_negative_emo[115:120]+workdays_negative_emo[0:10]
# 4a （8-12hour，16-20hour，20-24hour）
workdays_negative_time4a_am3 = workdays_negative_emo[40:60]
workdays_negative_time4a_pm2 = workdays_negative_emo[80:100]
workdays_negative_time4a_pm3 = workdays_negative_emo[100:120]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
workdays_negative_time4b_am2 = workdays_negative_emo[25:45]
workdays_negative_time4b_am3 = workdays_negative_emo[45:65]
workdays_negative_time4b_pm1 = workdays_negative_emo[65:85]
workdays_negative_time4b_pm2 = workdays_negative_emo[85:105]
workdays_negative_time4b_pm3 = workdays_negative_emo[105:120] + workdays_negative_emo[0:5]
# 4c （6-10hour，10-14hour，14-18hour，22-2hour）
workdays_negative_time4c_am2 = workdays_negative_emo[30:50]
workdays_negative_time4c_am3 = workdays_negative_emo[50:70]
workdays_negative_time4c_pm1 = workdays_negative_emo[70:90]
workdays_negative_time4c_pm3 = workdays_negative_emo[110:120] + workdays_negative_emo[0:10]
# 4d （7-11hour，11-15hour，15-19hour，19-23hour）
workdays_negative_time4d_am2 = workdays_negative_emo[35:55]
workdays_negative_time4d_am3 = workdays_negative_emo[55:75]
workdays_negative_time4d_pm1 = workdays_negative_emo[75:95]
workdays_negative_time4d_pm2 = workdays_negative_emo[95:115]
# 6a （6-12hour，12-18hour）
workdays_negative_time6a_am2 = workdays_negative_emo[30:60]
workdays_negative_time6a_pm1 = workdays_negative_emo[60:90]
# 6b （7-13hour，13-19hour，19-1hour）
workdays_negative_time6b_am2 = workdays_negative_emo[35:65]
workdays_negative_time6b_pm1 = workdays_negative_emo[65:95]
workdays_negative_time6b_pm2 = workdays_negative_emo[95:120]+workdays_negative_emo[0:5]
# 6c （8-14hour，14-20hour，20-2hour）
workdays_negative_time6c_am2 = workdays_negative_emo[40:70]
workdays_negative_time6c_pm1 = workdays_negative_emo[70:100]
workdays_negative_time6c_pm2 = workdays_negative_emo[100:120]+workdays_negative_emo[0:10]
# 6e （4-10hour，16-22hour）
workdays_negative_time6e_am1 = workdays_negative_emo[20:50]
workdays_negative_time6e_pm1 = workdays_negative_emo[80:110]
# 6f （5-11hour，11-17hour，17-23hour）
workdays_negative_time6f_am1 = workdays_negative_emo[25:55]
workdays_negative_time6f_am2 = workdays_negative_emo[55:85]
workdays_negative_time6f_pm1 = workdays_negative_emo[85:115]

# (5) 有效anger情绪区间
# 3a （9-12hour，15-18hour，21-24hour）
workdays_anger_time3a_am4 = workdays_li_anger[45:60]
workdays_anger_time3a_pm2 = workdays_li_anger[75:90]
workdays_anger_time3a_pm4 = workdays_li_anger[105:120]
# 3b （7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
workdays_anger_time3b_am3 = workdays_li_anger[35:50]
workdays_anger_time3b_am4 = workdays_li_anger[50:65]
workdays_anger_time3b_pm1 = workdays_li_anger[65:80]
workdays_anger_time3b_pm2 = workdays_li_anger[80:95]
workdays_anger_time3b_pm3 = workdays_li_anger[95:110]
workdays_anger_time3b_pm4 = workdays_li_anger[110:120]+workdays_li_anger[0:5]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
workdays_anger_time3c_am2 = workdays_li_anger[25:40]
workdays_anger_time3c_am3 = workdays_li_anger[40:55]
workdays_anger_time3c_am4 = workdays_li_anger[55:70]
workdays_anger_time3c_pm1 = workdays_li_anger[70:85]
workdays_anger_time3c_pm2 = workdays_li_anger[85:100]
workdays_anger_time3c_pm3 = workdays_li_anger[100:115]
workdays_anger_time3c_pm4 = workdays_li_anger[115:120]+workdays_li_anger[0:10]
# 4a （8-12hour，16-20hour，20-24hour）
workdays_anger_time4a_am3 = workdays_li_anger[40:60]
workdays_anger_time4a_pm2 = workdays_li_anger[80:100]
workdays_anger_time4a_pm3 = workdays_li_anger[100:120]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
workdays_anger_time4b_am2 = workdays_li_anger[25:45]
workdays_anger_time4b_am3 = workdays_li_anger[45:65]
workdays_anger_time4b_pm1 = workdays_li_anger[65:85]
workdays_anger_time4b_pm2 = workdays_li_anger[85:105]
workdays_anger_time4b_pm3 = workdays_li_anger[105:120] + workdays_li_anger[0:5]
# 4c （6-10hour，10-14hour，14-18hour，22-2hour）
workdays_anger_time4c_am2 = workdays_li_anger[30:50]
workdays_anger_time4c_am3 = workdays_li_anger[50:70]
workdays_anger_time4c_pm1 = workdays_li_anger[70:90]
workdays_anger_time4c_pm3 = workdays_li_anger[110:120] + workdays_li_anger[0:10]
# 4d （7-11hour，11-15hour，15-19hour，19-23hour）
workdays_anger_time4d_am2 = workdays_li_anger[35:55]
workdays_anger_time4d_am3 = workdays_li_anger[55:75]
workdays_anger_time4d_pm1 = workdays_li_anger[75:95]
workdays_anger_time4d_pm2 = workdays_li_anger[95:115]
# 6a （6-12hour，12-18hour）
workdays_anger_time6a_am2 = workdays_li_anger[30:60]
workdays_anger_time6a_pm1 = workdays_li_anger[60:90]
# 6b （7-13hour，13-19hour，19-1hour）
workdays_anger_time6b_am2 = workdays_li_anger[35:65]
workdays_anger_time6b_pm1 = workdays_li_anger[65:95]
workdays_anger_time6b_pm2 = workdays_li_anger[95:120]+workdays_li_anger[0:5]
# 6c （8-14hour，14-20hour，20-2hour）
workdays_anger_time6c_am2 = workdays_li_anger[40:70]
workdays_anger_time6c_pm1 = workdays_li_anger[70:100]
workdays_anger_time6c_pm2 = workdays_li_anger[100:120]+workdays_li_anger[0:10]
# 6e （4-10hour，16-22hour）
workdays_anger_time6e_am1 = workdays_li_anger[20:50]
workdays_anger_time6e_pm1 = workdays_li_anger[80:110]
# 6f （5-11hour，11-17hour，17-23hour）
workdays_anger_time6f_am1 = workdays_li_anger[25:55]
workdays_anger_time6f_am2 = workdays_li_anger[55:85]
workdays_anger_time6f_pm1 = workdays_li_anger[85:115]

# (6) 有效anxi情绪区间
# 3a （9-12hour，15-18hour，21-24hour）
workdays_anxi_time3a_am4 = workdays_li_anxi[45:60]
workdays_anxi_time3a_pm2 = workdays_li_anxi[75:90]
workdays_anxi_time3a_pm4 = workdays_li_anxi[105:120]
# 3b （7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
workdays_anxi_time3b_am3 = workdays_li_anxi[35:50]
workdays_anxi_time3b_am4 = workdays_li_anxi[50:65]
workdays_anxi_time3b_pm1 = workdays_li_anxi[65:80]
workdays_anxi_time3b_pm2 = workdays_li_anxi[80:95]
workdays_anxi_time3b_pm3 = workdays_li_anxi[95:110]
workdays_anxi_time3b_pm4 = workdays_li_anxi[110:120]+workdays_li_anxi[0:5]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
workdays_anxi_time3c_am2 = workdays_li_anxi[25:40]
workdays_anxi_time3c_am3 = workdays_li_anxi[40:55]
workdays_anxi_time3c_am4 = workdays_li_anxi[55:70]
workdays_anxi_time3c_pm1 = workdays_li_anxi[70:85]
workdays_anxi_time3c_pm2 = workdays_li_anxi[85:100]
workdays_anxi_time3c_pm3 = workdays_li_anxi[100:115]
workdays_anxi_time3c_pm4 = workdays_li_anxi[115:120]+workdays_li_anxi[0:10]
# 4a （8-12hour，16-20hour，20-24hour）
workdays_anxi_time4a_am3 = workdays_li_anxi[40:60]
workdays_anxi_time4a_pm2 = workdays_li_anxi[80:100]
workdays_anxi_time4a_pm3 = workdays_li_anxi[100:120]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
workdays_anxi_time4b_am2 = workdays_li_anxi[25:45]
workdays_anxi_time4b_am3 = workdays_li_anxi[45:65]
workdays_anxi_time4b_pm1 = workdays_li_anxi[65:85]
workdays_anxi_time4b_pm2 = workdays_li_anxi[85:105]
workdays_anxi_time4b_pm3 = workdays_li_anxi[105:120] + workdays_li_anxi[0:5]
# 4c （6-10hour，10-14hour，14-18hour，22-2hour）
workdays_anxi_time4c_am2 = workdays_li_anxi[30:50]
workdays_anxi_time4c_am3 = workdays_li_anxi[50:70]
workdays_anxi_time4c_pm1 = workdays_li_anxi[70:90]
workdays_anxi_time4c_pm3 = workdays_li_anxi[110:120] + workdays_li_anxi[0:10]
# 4d （7-11hour，11-15hour，15-19hour，19-23hour）
workdays_anxi_time4d_am2 = workdays_li_anxi[35:55]
workdays_anxi_time4d_am3 = workdays_li_anxi[55:75]
workdays_anxi_time4d_pm1 = workdays_li_anxi[75:95]
workdays_anxi_time4d_pm2 = workdays_li_anxi[95:115]
# 6a （6-12hour，12-18hour）
workdays_anxi_time6a_am2 = workdays_li_anxi[30:60]
workdays_anxi_time6a_pm1 = workdays_li_anxi[60:90]
# 6b （7-13hour，13-19hour，19-1hour）
workdays_anxi_time6b_am2 = workdays_li_anxi[35:65]
workdays_anxi_time6b_pm1 = workdays_li_anxi[65:95]
workdays_anxi_time6b_pm2 = workdays_li_anxi[95:120]+workdays_li_anxi[0:5]
# 6c （8-14hour，14-20hour，20-2hour）
workdays_anxi_time6c_am2 = workdays_li_anxi[40:70]
workdays_anxi_time6c_pm1 = workdays_li_anxi[70:100]
workdays_anxi_time6c_pm2 = workdays_li_anxi[100:120]+workdays_li_anxi[0:10]
# 6e （4-10hour，16-22hour）
workdays_anxi_time6e_am1 = workdays_li_anxi[20:50]
workdays_anxi_time6e_pm1 = workdays_li_anxi[80:110]
# 6f （5-11hour，11-17hour，17-23hour）
workdays_anxi_time6f_am1 = workdays_li_anxi[25:55]
workdays_anxi_time6f_am2 = workdays_li_anxi[55:85]
workdays_anxi_time6f_pm1 = workdays_li_anxi[85:115]


# (7) 有效hate情绪区间
# 3a （9-12hour，15-18hour，21-24hour）
workdays_hate_time3a_am4 = workdays_li_hate[45:60]
workdays_hate_time3a_pm2 = workdays_li_hate[75:90]
workdays_hate_time3a_pm4 = workdays_li_hate[105:120]
# 3b （7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
workdays_hate_time3b_am3 = workdays_li_hate[35:50]
workdays_hate_time3b_am4 = workdays_li_hate[50:65]
workdays_hate_time3b_pm1 = workdays_li_hate[65:80]
workdays_hate_time3b_pm2 = workdays_li_hate[80:95]
workdays_hate_time3b_pm3 = workdays_li_hate[95:110]
workdays_hate_time3b_pm4 = workdays_li_hate[110:120]+workdays_li_hate[0:5]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
workdays_hate_time3c_am2 = workdays_li_hate[25:40]
workdays_hate_time3c_am3 = workdays_li_hate[40:55]
workdays_hate_time3c_am4 = workdays_li_hate[55:70]
workdays_hate_time3c_pm1 = workdays_li_hate[70:85]
workdays_hate_time3c_pm2 = workdays_li_hate[85:100]
workdays_hate_time3c_pm3 = workdays_li_hate[100:115]
workdays_hate_time3c_pm4 = workdays_li_hate[115:120]+workdays_li_hate[0:10]
# 4a （8-12hour，16-20hour，20-24hour）
workdays_hate_time4a_am3 = workdays_li_hate[40:60]
workdays_hate_time4a_pm2 = workdays_li_hate[80:100]
workdays_hate_time4a_pm3 = workdays_li_hate[100:120]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
workdays_hate_time4b_am2 = workdays_li_hate[25:45]
workdays_hate_time4b_am3 = workdays_li_hate[45:65]
workdays_hate_time4b_pm1 = workdays_li_hate[65:85]
workdays_hate_time4b_pm2 = workdays_li_hate[85:105]
workdays_hate_time4b_pm3 = workdays_li_hate[105:120] + workdays_li_hate[0:5]
# 4c （6-10hour，10-14hour，14-18hour，22-2hour）
workdays_hate_time4c_am2 = workdays_li_hate[30:50]
workdays_hate_time4c_am3 = workdays_li_hate[50:70]
workdays_hate_time4c_pm1 = workdays_li_hate[70:90]
workdays_hate_time4c_pm3 = workdays_li_hate[110:120] + workdays_li_hate[0:10]
# 4d （7-11hour，11-15hour，15-19hour，19-23hour）
workdays_hate_time4d_am2 = workdays_li_hate[35:55]
workdays_hate_time4d_am3 = workdays_li_hate[55:75]
workdays_hate_time4d_pm1 = workdays_li_hate[75:95]
workdays_hate_time4d_pm2 = workdays_li_hate[95:115]
# 6a （6-12hour，12-18hour）
workdays_hate_time6a_am2 = workdays_li_hate[30:60]
workdays_hate_time6a_pm1 = workdays_li_hate[60:90]
# 6b （7-13hour，13-19hour，19-1hour）
workdays_hate_time6b_am2 = workdays_li_hate[35:65]
workdays_hate_time6b_pm1 = workdays_li_hate[65:95]
workdays_hate_time6b_pm2 = workdays_li_hate[95:120]+workdays_li_hate[0:5]
# 6c （8-14hour，14-20hour，20-2hour）
workdays_hate_time6c_am2 = workdays_li_hate[40:70]
workdays_hate_time6c_pm1 = workdays_li_hate[70:100]
workdays_hate_time6c_pm2 = workdays_li_hate[100:120]+workdays_li_hate[0:10]
# 6e （4-10hour，16-22hour）
workdays_hate_time6e_am1 = workdays_li_hate[20:50]
workdays_hate_time6e_pm1 = workdays_li_hate[80:110]
# 6f （5-11hour，11-17hour，17-23hour）
workdays_hate_time6f_am1 = workdays_li_hate[25:55]
workdays_hate_time6f_am2 = workdays_li_hate[55:85]
workdays_hate_time6f_pm1 = workdays_li_hate[85:115]

# (8) 有效sad情绪区间
# 3a （9-12hour，15-18hour，21-24hour）
workdays_sad_time3a_am4 = workdays_li_sad[45:60]
workdays_sad_time3a_pm2 = workdays_li_sad[75:90]
workdays_sad_time3a_pm4 = workdays_li_sad[105:120]
# 3b （7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
workdays_sad_time3b_am3 = workdays_li_sad[35:50]
workdays_sad_time3b_am4 = workdays_li_sad[50:65]
workdays_sad_time3b_pm1 = workdays_li_sad[65:80]
workdays_sad_time3b_pm2 = workdays_li_sad[80:95]
workdays_sad_time3b_pm3 = workdays_li_sad[95:110]
workdays_sad_time3b_pm4 = workdays_li_sad[110:120]+workdays_li_sad[0:5]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
workdays_sad_time3c_am2 = workdays_li_sad[25:40]
workdays_sad_time3c_am3 = workdays_li_sad[40:55]
workdays_sad_time3c_am4 = workdays_li_sad[55:70]
workdays_sad_time3c_pm1 = workdays_li_sad[70:85]
workdays_sad_time3c_pm2 = workdays_li_sad[85:100]
workdays_sad_time3c_pm3 = workdays_li_sad[100:115]
workdays_sad_time3c_pm4 = workdays_li_sad[115:120]+workdays_li_sad[0:10]
# 4a （8-12hour，16-20hour，20-24hour）
workdays_sad_time4a_am3 = workdays_li_sad[40:60]
workdays_sad_time4a_pm2 = workdays_li_sad[80:100]
workdays_sad_time4a_pm3 = workdays_li_sad[100:120]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
workdays_sad_time4b_am2 = workdays_li_sad[25:45]
workdays_sad_time4b_am3 = workdays_li_sad[45:65]
workdays_sad_time4b_pm1 = workdays_li_sad[65:85]
workdays_sad_time4b_pm2 = workdays_li_sad[85:105]
workdays_sad_time4b_pm3 = workdays_li_sad[105:120] + workdays_li_sad[0:5]
# 4c （6-10hour，10-14hour，14-18hour，22-2hour）
workdays_sad_time4c_am2 = workdays_li_sad[30:50]
workdays_sad_time4c_am3 = workdays_li_sad[50:70]
workdays_sad_time4c_pm1 = workdays_li_sad[70:90]
workdays_sad_time4c_pm3 = workdays_li_sad[110:120] + workdays_li_sad[0:10]
# 4d （7-11hour，11-15hour，15-19hour，19-23hour）
workdays_sad_time4d_am2 = workdays_li_sad[35:55]
workdays_sad_time4d_am3 = workdays_li_sad[55:75]
workdays_sad_time4d_pm1 = workdays_li_sad[75:95]
workdays_sad_time4d_pm2 = workdays_li_sad[95:115]
# 6a （6-12hour，12-18hour）
workdays_sad_time6a_am2 = workdays_li_sad[30:60]
workdays_sad_time6a_pm1 = workdays_li_sad[60:90]
# 6b （7-13hour，13-19hour，19-1hour）
workdays_sad_time6b_am2 = workdays_li_sad[35:65]
workdays_sad_time6b_pm1 = workdays_li_sad[65:95]
workdays_sad_time6b_pm2 = workdays_li_sad[95:120]+workdays_li_sad[0:5]
# 6c （8-14hour，14-20hour，20-2hour）
workdays_sad_time6c_am2 = workdays_li_sad[40:70]
workdays_sad_time6c_pm1 = workdays_li_sad[70:100]
workdays_sad_time6c_pm2 = workdays_li_sad[100:120]+workdays_li_sad[0:10]
# 6e （4-10hour，16-22hour）
workdays_sad_time6e_am1 = workdays_li_sad[20:50]
workdays_sad_time6e_pm1 = workdays_li_sad[80:110]
# 6f （5-11hour，11-17hour，17-23hour）
workdays_sad_time6f_am1 = workdays_li_sad[25:55]
workdays_sad_time6f_am2 = workdays_li_sad[55:85]
workdays_sad_time6f_pm1 = workdays_li_sad[85:115]


# (9) 有效surprise情绪区间
# 3a （9-12hour，15-18hour，21-24hour）
workdays_surprise_time3a_am4 = workdays_li_surprise[45:60]
workdays_surprise_time3a_pm2 = workdays_li_surprise[75:90]
workdays_surprise_time3a_pm4 = workdays_li_surprise[105:120]
# 3b （7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
workdays_surprise_time3b_am3 = workdays_li_surprise[35:50]
workdays_surprise_time3b_am4 = workdays_li_surprise[50:65]
workdays_surprise_time3b_pm1 = workdays_li_surprise[65:80]
workdays_surprise_time3b_pm2 = workdays_li_surprise[80:95]
workdays_surprise_time3b_pm3 = workdays_li_surprise[95:110]
workdays_surprise_time3b_pm4 = workdays_li_surprise[110:120]+workdays_li_surprise[0:5]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
workdays_surprise_time3c_am2 = workdays_li_surprise[25:40]
workdays_surprise_time3c_am3 = workdays_li_surprise[40:55]
workdays_surprise_time3c_am4 = workdays_li_surprise[55:70]
workdays_surprise_time3c_pm1 = workdays_li_surprise[70:85]
workdays_surprise_time3c_pm2 = workdays_li_surprise[85:100]
workdays_surprise_time3c_pm3 = workdays_li_surprise[100:115]
workdays_surprise_time3c_pm4 = workdays_li_surprise[115:120]+workdays_li_surprise[0:10]
# 4a （8-12hour，16-20hour，20-24hour）
workdays_surprise_time4a_am3 = workdays_li_surprise[40:60]
workdays_surprise_time4a_pm2 = workdays_li_surprise[80:100]
workdays_surprise_time4a_pm3 = workdays_li_surprise[100:120]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
workdays_surprise_time4b_am2 = workdays_li_surprise[25:45]
workdays_surprise_time4b_am3 = workdays_li_surprise[45:65]
workdays_surprise_time4b_pm1 = workdays_li_surprise[65:85]
workdays_surprise_time4b_pm2 = workdays_li_surprise[85:105]
workdays_surprise_time4b_pm3 = workdays_li_surprise[105:120] + workdays_li_surprise[0:5]
# 4c （6-10hour，10-14hour，14-18hour，22-2hour）
workdays_surprise_time4c_am2 = workdays_li_surprise[30:50]
workdays_surprise_time4c_am3 = workdays_li_surprise[50:70]
workdays_surprise_time4c_pm1 = workdays_li_surprise[70:90]
workdays_surprise_time4c_pm3 = workdays_li_surprise[110:120] + workdays_li_surprise[0:10]
# 4d （7-11hour，11-15hour，15-19hour，19-23hour）
workdays_surprise_time4d_am2 = workdays_li_surprise[35:55]
workdays_surprise_time4d_am3 = workdays_li_surprise[55:75]
workdays_surprise_time4d_pm1 = workdays_li_surprise[75:95]
workdays_surprise_time4d_pm2 = workdays_li_surprise[95:115]
# 6a （6-12hour，12-18hour）
workdays_surprise_time6a_am2 = workdays_li_surprise[30:60]
workdays_surprise_time6a_pm1 = workdays_li_surprise[60:90]
# 6b （7-13hour，13-19hour，19-1hour）
workdays_surprise_time6b_am2 = workdays_li_surprise[35:65]
workdays_surprise_time6b_pm1 = workdays_li_surprise[65:95]
workdays_surprise_time6b_pm2 = workdays_li_surprise[95:120]+workdays_li_surprise[0:5]
# 6c （8-14hour，14-20hour，20-2hour）
workdays_surprise_time6c_am2 = workdays_li_surprise[40:70]
workdays_surprise_time6c_pm1 = workdays_li_surprise[70:100]
workdays_surprise_time6c_pm2 = workdays_li_surprise[100:120]+workdays_li_surprise[0:10]
# 6e （4-10hour，16-22hour）
workdays_surprise_time6e_am1 = workdays_li_surprise[20:50]
workdays_surprise_time6e_pm1 = workdays_li_surprise[80:110]
# 6f （5-11hour，11-17hour，17-23hour）
workdays_surprise_time6f_am1 = workdays_li_surprise[25:55]
workdays_surprise_time6f_am2 = workdays_li_surprise[55:85]
workdays_surprise_time6f_pm1 = workdays_li_surprise[85:115]

# (10) 有效love情绪区间
# 3a （9-12hour，15-18hour，21-24hour）
workdays_love_time3a_am4 = workdays_li_love[45:60]
workdays_love_time3a_pm2 = workdays_li_love[75:90]
workdays_love_time3a_pm4 = workdays_li_love[105:120]
# 3b （7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
workdays_love_time3b_am3 = workdays_li_love[35:50]
workdays_love_time3b_am4 = workdays_li_love[50:65]
workdays_love_time3b_pm1 = workdays_li_love[65:80]
workdays_love_time3b_pm2 = workdays_li_love[80:95]
workdays_love_time3b_pm3 = workdays_li_love[95:110]
workdays_love_time3b_pm4 = workdays_li_love[110:120]+workdays_li_love[0:5]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
workdays_love_time3c_am2 = workdays_li_love[25:40]
workdays_love_time3c_am3 = workdays_li_love[40:55]
workdays_love_time3c_am4 = workdays_li_love[55:70]
workdays_love_time3c_pm1 = workdays_li_love[70:85]
workdays_love_time3c_pm2 = workdays_li_love[85:100]
workdays_love_time3c_pm3 = workdays_li_love[100:115]
workdays_love_time3c_pm4 = workdays_li_love[115:120]+workdays_li_love[0:10]
# 4a （8-12hour，16-20hour，20-24hour）
workdays_love_time4a_am3 = workdays_li_love[40:60]
workdays_love_time4a_pm2 = workdays_li_love[80:100]
workdays_love_time4a_pm3 = workdays_li_love[100:120]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
workdays_love_time4b_am2 = workdays_li_love[25:45]
workdays_love_time4b_am3 = workdays_li_love[45:65]
workdays_love_time4b_pm1 = workdays_li_love[65:85]
workdays_love_time4b_pm2 = workdays_li_love[85:105]
workdays_love_time4b_pm3 = workdays_li_love[105:120] + workdays_li_love[0:5]
# 4c （6-10hour，10-14hour，14-18hour，22-2hour）
workdays_love_time4c_am2 = workdays_li_love[30:50]
workdays_love_time4c_am3 = workdays_li_love[50:70]
workdays_love_time4c_pm1 = workdays_li_love[70:90]
workdays_love_time4c_pm3 = workdays_li_love[110:120] + workdays_li_love[0:10]
# 4d （7-11hour，11-15hour，15-19hour，19-23hour）
workdays_love_time4d_am2 = workdays_li_love[35:55]
workdays_love_time4d_am3 = workdays_li_love[55:75]
workdays_love_time4d_pm1 = workdays_li_love[75:95]
workdays_love_time4d_pm2 = workdays_li_love[95:115]
# 6a （6-12hour，12-18hour）
workdays_love_time6a_am2 = workdays_li_love[30:60]
workdays_love_time6a_pm1 = workdays_li_love[60:90]
# 6b （7-13hour，13-19hour，19-1hour）
workdays_love_time6b_am2 = workdays_li_love[35:65]
workdays_love_time6b_pm1 = workdays_li_love[65:95]
workdays_love_time6b_pm2 = workdays_li_love[95:120]+workdays_li_love[0:5]
# 6c （8-14hour，14-20hour，20-2hour）
workdays_love_time6c_am2 = workdays_li_love[40:70]
workdays_love_time6c_pm1 = workdays_li_love[70:100]
workdays_love_time6c_pm2 = workdays_li_love[100:120]+workdays_li_love[0:10]
# 6e （4-10hour，16-22hour）
workdays_love_time6e_am1 = workdays_li_love[20:50]
workdays_love_time6e_pm1 = workdays_li_love[80:110]
# 6f （5-11hour，11-17hour，17-23hour）
workdays_love_time6f_am1 = workdays_li_love[25:55]
workdays_love_time6f_am2 = workdays_li_love[55:85]
workdays_love_time6f_pm1 = workdays_li_love[85:115]

# (11) 有效joy情绪区间
# 3a （9-12hour，15-18hour，21-24hour）
workdays_joy_time3a_am4 = workdays_li_joy[45:60]
workdays_joy_time3a_pm2 = workdays_li_joy[75:90]
workdays_joy_time3a_pm4 = workdays_li_joy[105:120]
# 3b （7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
workdays_joy_time3b_am3 = workdays_li_joy[35:50]
workdays_joy_time3b_am4 = workdays_li_joy[50:65]
workdays_joy_time3b_pm1 = workdays_li_joy[65:80]
workdays_joy_time3b_pm2 = workdays_li_joy[80:95]
workdays_joy_time3b_pm3 = workdays_li_joy[95:110]
workdays_joy_time3b_pm4 = workdays_li_joy[110:120]+workdays_li_joy[0:5]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
workdays_joy_time3c_am2 = workdays_li_joy[25:40]
workdays_joy_time3c_am3 = workdays_li_joy[40:55]
workdays_joy_time3c_am4 = workdays_li_joy[55:70]
workdays_joy_time3c_pm1 = workdays_li_joy[70:85]
workdays_joy_time3c_pm2 = workdays_li_joy[85:100]
workdays_joy_time3c_pm3 = workdays_li_joy[100:115]
workdays_joy_time3c_pm4 = workdays_li_joy[115:120]+workdays_li_joy[0:10]
# 4a （8-12hour，16-20hour，20-24hour）
workdays_joy_time4a_am3 = workdays_li_joy[40:60]
workdays_joy_time4a_pm2 = workdays_li_joy[80:100]
workdays_joy_time4a_pm3 = workdays_li_joy[100:120]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
workdays_joy_time4b_am2 = workdays_li_joy[25:45]
workdays_joy_time4b_am3 = workdays_li_joy[45:65]
workdays_joy_time4b_pm1 = workdays_li_joy[65:85]
workdays_joy_time4b_pm2 = workdays_li_joy[85:105]
workdays_joy_time4b_pm3 = workdays_li_joy[105:120] + workdays_li_joy[0:5]
# 4c （6-10hour，10-14hour，14-18hour，22-2hour）
workdays_joy_time4c_am2 = workdays_li_joy[30:50]
workdays_joy_time4c_am3 = workdays_li_joy[50:70]
workdays_joy_time4c_pm1 = workdays_li_joy[70:90]
workdays_joy_time4c_pm3 = workdays_li_joy[110:120] + workdays_li_joy[0:10]
# 4d （7-11hour，11-15hour，15-19hour，19-23hour）
workdays_joy_time4d_am2 = workdays_li_joy[35:55]
workdays_joy_time4d_am3 = workdays_li_joy[55:75]
workdays_joy_time4d_pm1 = workdays_li_joy[75:95]
workdays_joy_time4d_pm2 = workdays_li_joy[95:115]
# 6a （6-12hour，12-18hour）
workdays_joy_time6a_am2 = workdays_li_joy[30:60]
workdays_joy_time6a_pm1 = workdays_li_joy[60:90]
# 6b （7-13hour，13-19hour，19-1hour）
workdays_joy_time6b_am2 = workdays_li_joy[35:65]
workdays_joy_time6b_pm1 = workdays_li_joy[65:95]
workdays_joy_time6b_pm2 = workdays_li_joy[95:120]+workdays_li_joy[0:5]
# 6c （8-14hour，14-20hour，20-2hour）
workdays_joy_time6c_am2 = workdays_li_joy[40:70]
workdays_joy_time6c_pm1 = workdays_li_joy[70:100]
workdays_joy_time6c_pm2 = workdays_li_joy[100:120]+workdays_li_joy[0:10]
# 6e （4-10hour，16-22hour）
workdays_joy_time6e_am1 = workdays_li_joy[20:50]
workdays_joy_time6e_pm1 = workdays_li_joy[80:110]
# 6f （5-11hour，11-17hour，17-23hour）
workdays_joy_time6f_am1 = workdays_li_joy[25:55]
workdays_joy_time6f_am2 = workdays_li_joy[55:85]
workdays_joy_time6f_pm1 = workdays_li_joy[85:115]

# (12) 有效expectation情绪区间
# 3a （9-12hour，15-18hour，21-24hour）
workdays_expectation_time3a_am4 = workdays_li_expectation[45:60]
workdays_expectation_time3a_pm2 = workdays_li_expectation[75:90]
workdays_expectation_time3a_pm4 = workdays_li_expectation[105:120]
# 3b （7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
workdays_expectation_time3b_am3 = workdays_li_expectation[35:50]
workdays_expectation_time3b_am4 = workdays_li_expectation[50:65]
workdays_expectation_time3b_pm1 = workdays_li_expectation[65:80]
workdays_expectation_time3b_pm2 = workdays_li_expectation[80:95]
workdays_expectation_time3b_pm3 = workdays_li_expectation[95:110]
workdays_expectation_time3b_pm4 = workdays_li_expectation[110:120]+workdays_li_expectation[0:5]
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
workdays_expectation_time3c_am2 = workdays_li_expectation[25:40]
workdays_expectation_time3c_am3 = workdays_li_expectation[40:55]
workdays_expectation_time3c_am4 = workdays_li_expectation[55:70]
workdays_expectation_time3c_pm1 = workdays_li_expectation[70:85]
workdays_expectation_time3c_pm2 = workdays_li_expectation[85:100]
workdays_expectation_time3c_pm3 = workdays_li_expectation[100:115]
workdays_expectation_time3c_pm4 = workdays_li_expectation[115:120]+workdays_li_expectation[0:10]
# 4a （8-12hour，16-20hour，20-24hour）
workdays_expectation_time4a_am3 = workdays_li_expectation[40:60]
workdays_expectation_time4a_pm2 = workdays_li_expectation[80:100]
workdays_expectation_time4a_pm3 = workdays_li_expectation[100:120]
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
workdays_expectation_time4b_am2 = workdays_li_expectation[25:45]
workdays_expectation_time4b_am3 = workdays_li_expectation[45:65]
workdays_expectation_time4b_pm1 = workdays_li_expectation[65:85]
workdays_expectation_time4b_pm2 = workdays_li_expectation[85:105]
workdays_expectation_time4b_pm3 = workdays_li_expectation[105:120] + workdays_li_expectation[0:5]
# 4c （6-10hour，10-14hour，14-18hour，22-2hour）
workdays_expectation_time4c_am2 = workdays_li_expectation[30:50]
workdays_expectation_time4c_am3 = workdays_li_expectation[50:70]
workdays_expectation_time4c_pm1 = workdays_li_expectation[70:90]
workdays_expectation_time4c_pm3 = workdays_li_expectation[110:120] + workdays_li_expectation[0:10]
# 4d （7-11hour，11-15hour，15-19hour，19-23hour）
workdays_expectation_time4d_am2 = workdays_li_expectation[35:55]
workdays_expectation_time4d_am3 = workdays_li_expectation[55:75]
workdays_expectation_time4d_pm1 = workdays_li_expectation[75:95]
workdays_expectation_time4d_pm2 = workdays_li_expectation[95:115]
# 6a （6-12hour，12-18hour）
workdays_expectation_time6a_am2 = workdays_li_expectation[30:60]
workdays_expectation_time6a_pm1 = workdays_li_expectation[60:90]
# 6b （7-13hour，13-19hour，19-1hour）
workdays_expectation_time6b_am2 = workdays_li_expectation[35:65]
workdays_expectation_time6b_pm1 = workdays_li_expectation[65:95]
workdays_expectation_time6b_pm2 = workdays_li_expectation[95:120]+workdays_li_expectation[0:5]
# 6c （8-14hour，14-20hour，20-2hour）
workdays_expectation_time6c_am2 = workdays_li_expectation[40:70]
workdays_expectation_time6c_pm1 = workdays_li_expectation[70:100]
workdays_expectation_time6c_pm2 = workdays_li_expectation[100:120]+workdays_li_expectation[0:10]
# 6e （4-10hour，16-22hour）
workdays_expectation_time6e_am1 = workdays_li_expectation[20:50]
workdays_expectation_time6e_pm1 = workdays_li_expectation[80:110]
# 6f （5-11hour，11-17hour，17-23hour）
workdays_expectation_time6f_am1 = workdays_li_expectation[25:55]
workdays_expectation_time6f_am2 = workdays_li_expectation[55:85]
workdays_expectation_time6f_pm1 = workdays_li_expectation[85:115]



# 有效道德区间数据汇总
workdays_moral_group = [workdays_moral_time3a_am4+workdays_moral_time3a_pm2+workdays_moral_time3a_pm4+
                        workdays_moral_time3b_am3+workdays_moral_time3b_am4+workdays_moral_time3b_pm1+
                        workdays_moral_time3b_pm2+workdays_moral_time3b_pm3+workdays_moral_time3b_pm4+
                        workdays_moral_time3c_am2+workdays_moral_time3c_am3+workdays_moral_time3c_am4+
                        workdays_moral_time3c_pm1+workdays_moral_time3c_pm2+workdays_moral_time3c_pm3+
                        workdays_moral_time3c_pm4+workdays_moral_time4a_am3+workdays_moral_time4a_pm2+
                        workdays_moral_time4a_pm3+workdays_moral_time4b_am2+workdays_moral_time4b_am3+
                        workdays_moral_time4b_pm1+workdays_moral_time4b_pm2+workdays_moral_time4b_pm3+
                        workdays_moral_time4c_am2+workdays_moral_time4c_am3+workdays_moral_time4c_pm1+
                        workdays_moral_time4c_pm3+workdays_moral_time4d_am2+workdays_moral_time4d_am3+
                        workdays_moral_time4d_pm1+workdays_moral_time4d_pm2+workdays_moral_time6a_am2+
                        workdays_moral_time6a_pm1+workdays_moral_time6b_am2+workdays_moral_time6b_pm1+
                        workdays_moral_time6b_pm2+workdays_moral_time6c_am2+workdays_moral_time6c_pm1+
                        workdays_moral_time6c_pm2+workdays_moral_time6e_am1+workdays_moral_time6e_pm1+
                        workdays_moral_time6f_am1+workdays_moral_time6f_am2+workdays_moral_time6f_pm1]

# 有效情绪区间数据汇总（总情绪）
workdays_emo_group = [workdays_emo_time3a_am4+workdays_emo_time3a_pm2+workdays_emo_time3a_pm4+
                        workdays_emo_time3b_am3+workdays_emo_time3b_am4+workdays_emo_time3b_pm1+
                        workdays_emo_time3b_pm2+workdays_emo_time3b_pm3+workdays_emo_time3b_pm4+
                        workdays_emo_time3c_am2+workdays_emo_time3c_am3+workdays_emo_time3c_am4+
                        workdays_emo_time3c_pm1+workdays_emo_time3c_pm2+workdays_emo_time3c_pm3+
                        workdays_emo_time3c_pm4+workdays_emo_time4a_am3+workdays_emo_time4a_pm2+
                        workdays_emo_time4a_pm3+workdays_emo_time4b_am2+workdays_emo_time4b_am3+
                        workdays_emo_time4b_pm1+workdays_emo_time4b_pm2+workdays_emo_time4b_pm3+
                        workdays_emo_time4c_am2+workdays_emo_time4c_am3+workdays_emo_time4c_pm1+
                        workdays_emo_time4c_pm3+workdays_emo_time4d_am2+workdays_emo_time4d_am3+
                        workdays_emo_time4d_pm1+workdays_emo_time4d_pm2+workdays_emo_time6a_am2+
                        workdays_emo_time6a_pm1+workdays_emo_time6b_am2+workdays_emo_time6b_pm1+
                        workdays_emo_time6b_pm2+workdays_emo_time6c_am2+workdays_emo_time6c_pm1+
                        workdays_emo_time6c_pm2+workdays_emo_time6e_am1+workdays_emo_time6e_pm1+
                        workdays_emo_time6f_am1+workdays_emo_time6f_am2+workdays_emo_time6f_pm1]

# 有效情绪区间数据汇总（积极情绪positive）
workdays_positive_group = [workdays_positive_time3a_am4+workdays_positive_time3a_pm2+workdays_positive_time3a_pm4+
                        workdays_positive_time3b_am3+workdays_positive_time3b_am4+workdays_positive_time3b_pm1+
                        workdays_positive_time3b_pm2+workdays_positive_time3b_pm3+workdays_positive_time3b_pm4+
                        workdays_positive_time3c_am2+workdays_positive_time3c_am3+workdays_positive_time3c_am4+
                        workdays_positive_time3c_pm1+workdays_positive_time3c_pm2+workdays_positive_time3c_pm3+
                        workdays_positive_time3c_pm4+workdays_positive_time4a_am3+workdays_positive_time4a_pm2+
                        workdays_positive_time4a_pm3+workdays_positive_time4b_am2+workdays_positive_time4b_am3+
                        workdays_positive_time4b_pm1+workdays_positive_time4b_pm2+workdays_positive_time4b_pm3+
                        workdays_positive_time4c_am2+workdays_positive_time4c_am3+workdays_positive_time4c_pm1+
                        workdays_positive_time4c_pm3+workdays_positive_time4d_am2+workdays_positive_time4d_am3+
                        workdays_positive_time4d_pm1+workdays_positive_time4d_pm2+workdays_positive_time6a_am2+
                        workdays_positive_time6a_pm1+workdays_positive_time6b_am2+workdays_positive_time6b_pm1+
                        workdays_positive_time6b_pm2+workdays_positive_time6c_am2+workdays_positive_time6c_pm1+
                        workdays_positive_time6c_pm2+workdays_positive_time6e_am1+workdays_positive_time6e_pm1+
                        workdays_positive_time6f_am1+workdays_positive_time6f_am2+workdays_positive_time6f_pm1]

# 有效情绪区间数据汇总（消极情绪negative）
workdays_negative_group = [workdays_negative_time3a_am4+workdays_negative_time3a_pm2+workdays_negative_time3a_pm4+
                        workdays_negative_time3b_am3+workdays_negative_time3b_am4+workdays_negative_time3b_pm1+
                        workdays_negative_time3b_pm2+workdays_negative_time3b_pm3+workdays_negative_time3b_pm4+
                        workdays_negative_time3c_am2+workdays_negative_time3c_am3+workdays_negative_time3c_am4+
                        workdays_negative_time3c_pm1+workdays_negative_time3c_pm2+workdays_negative_time3c_pm3+
                        workdays_negative_time3c_pm4+workdays_negative_time4a_am3+workdays_negative_time4a_pm2+
                        workdays_negative_time4a_pm3+workdays_negative_time4b_am2+workdays_negative_time4b_am3+
                        workdays_negative_time4b_pm1+workdays_negative_time4b_pm2+workdays_negative_time4b_pm3+
                        workdays_negative_time4c_am2+workdays_negative_time4c_am3+workdays_negative_time4c_pm1+
                        workdays_negative_time4c_pm3+workdays_negative_time4d_am2+workdays_negative_time4d_am3+
                        workdays_negative_time4d_pm1+workdays_negative_time4d_pm2+workdays_negative_time6a_am2+
                        workdays_negative_time6a_pm1+workdays_negative_time6b_am2+workdays_negative_time6b_pm1+
                        workdays_negative_time6b_pm2+workdays_negative_time6c_am2+workdays_negative_time6c_pm1+
                        workdays_negative_time6c_pm2+workdays_negative_time6e_am1+workdays_negative_time6e_pm1+
                        workdays_negative_time6f_am1+workdays_negative_time6f_am2+workdays_negative_time6f_pm1]

# 有效anger情绪区间数据汇总
workdays_anger_group = [workdays_anger_time3a_am4+workdays_anger_time3a_pm2+workdays_anger_time3a_pm4+
                        workdays_anger_time3b_am3+workdays_anger_time3b_am4+workdays_anger_time3b_pm1+
                        workdays_anger_time3b_pm2+workdays_anger_time3b_pm3+workdays_anger_time3b_pm4+
                        workdays_anger_time3c_am2+workdays_anger_time3c_am3+workdays_anger_time3c_am4+
                        workdays_anger_time3c_pm1+workdays_anger_time3c_pm2+workdays_anger_time3c_pm3+
                        workdays_anger_time3c_pm4+workdays_anger_time4a_am3+workdays_anger_time4a_pm2+
                        workdays_anger_time4a_pm3+workdays_anger_time4b_am2+workdays_anger_time4b_am3+
                        workdays_anger_time4b_pm1+workdays_anger_time4b_pm2+workdays_anger_time4b_pm3+
                        workdays_anger_time4c_am2+workdays_anger_time4c_am3+workdays_anger_time4c_pm1+
                        workdays_anger_time4c_pm3+workdays_anger_time4d_am2+workdays_anger_time4d_am3+
                        workdays_anger_time4d_pm1+workdays_anger_time4d_pm2+workdays_anger_time6a_am2+
                        workdays_anger_time6a_pm1+workdays_anger_time6b_am2+workdays_anger_time6b_pm1+
                        workdays_anger_time6b_pm2+workdays_anger_time6c_am2+workdays_anger_time6c_pm1+
                        workdays_anger_time6c_pm2+workdays_anger_time6e_am1+workdays_anger_time6e_pm1+
                        workdays_anger_time6f_am1+workdays_anger_time6f_am2+workdays_anger_time6f_pm1]

# 有效anxi情绪区间数据汇总
workdays_anxi_group = [workdays_anxi_time3a_am4+workdays_anxi_time3a_pm2+workdays_anxi_time3a_pm4+
                        workdays_anxi_time3b_am3+workdays_anxi_time3b_am4+workdays_anxi_time3b_pm1+
                        workdays_anxi_time3b_pm2+workdays_anxi_time3b_pm3+workdays_anxi_time3b_pm4+
                        workdays_anxi_time3c_am2+workdays_anxi_time3c_am3+workdays_anxi_time3c_am4+
                        workdays_anxi_time3c_pm1+workdays_anxi_time3c_pm2+workdays_anxi_time3c_pm3+
                        workdays_anxi_time3c_pm4+workdays_anxi_time4a_am3+workdays_anxi_time4a_pm2+
                        workdays_anxi_time4a_pm3+workdays_anxi_time4b_am2+workdays_anxi_time4b_am3+
                        workdays_anxi_time4b_pm1+workdays_anxi_time4b_pm2+workdays_anxi_time4b_pm3+
                        workdays_anxi_time4c_am2+workdays_anxi_time4c_am3+workdays_anxi_time4c_pm1+
                        workdays_anxi_time4c_pm3+workdays_anxi_time4d_am2+workdays_anxi_time4d_am3+
                        workdays_anxi_time4d_pm1+workdays_anxi_time4d_pm2+workdays_anxi_time6a_am2+
                        workdays_anxi_time6a_pm1+workdays_anxi_time6b_am2+workdays_anxi_time6b_pm1+
                        workdays_anxi_time6b_pm2+workdays_anxi_time6c_am2+workdays_anxi_time6c_pm1+
                        workdays_anxi_time6c_pm2+workdays_anxi_time6e_am1+workdays_anxi_time6e_pm1+
                        workdays_anxi_time6f_am1+workdays_anxi_time6f_am2+workdays_anxi_time6f_pm1]

# 有效hate情绪区间数据汇总
workdays_hate_group = [workdays_hate_time3a_am4+workdays_hate_time3a_pm2+workdays_hate_time3a_pm4+
                        workdays_hate_time3b_am3+workdays_hate_time3b_am4+workdays_hate_time3b_pm1+
                        workdays_hate_time3b_pm2+workdays_hate_time3b_pm3+workdays_hate_time3b_pm4+
                        workdays_hate_time3c_am2+workdays_hate_time3c_am3+workdays_hate_time3c_am4+
                        workdays_hate_time3c_pm1+workdays_hate_time3c_pm2+workdays_hate_time3c_pm3+
                        workdays_hate_time3c_pm4+workdays_hate_time4a_am3+workdays_hate_time4a_pm2+
                        workdays_hate_time4a_pm3+workdays_hate_time4b_am2+workdays_hate_time4b_am3+
                        workdays_hate_time4b_pm1+workdays_hate_time4b_pm2+workdays_hate_time4b_pm3+
                        workdays_hate_time4c_am2+workdays_hate_time4c_am3+workdays_hate_time4c_pm1+
                        workdays_hate_time4c_pm3+workdays_hate_time4d_am2+workdays_hate_time4d_am3+
                        workdays_hate_time4d_pm1+workdays_hate_time4d_pm2+workdays_hate_time6a_am2+
                        workdays_hate_time6a_pm1+workdays_hate_time6b_am2+workdays_hate_time6b_pm1+
                        workdays_hate_time6b_pm2+workdays_hate_time6c_am2+workdays_hate_time6c_pm1+
                        workdays_hate_time6c_pm2+workdays_hate_time6e_am1+workdays_hate_time6e_pm1+
                        workdays_hate_time6f_am1+workdays_hate_time6f_am2+workdays_hate_time6f_pm1]

# 有效sad情绪区间数据汇总
workdays_sad_group = [workdays_sad_time3a_am4+workdays_sad_time3a_pm2+workdays_sad_time3a_pm4+
                        workdays_sad_time3b_am3+workdays_sad_time3b_am4+workdays_sad_time3b_pm1+
                        workdays_sad_time3b_pm2+workdays_sad_time3b_pm3+workdays_sad_time3b_pm4+
                        workdays_sad_time3c_am2+workdays_sad_time3c_am3+workdays_sad_time3c_am4+
                        workdays_sad_time3c_pm1+workdays_sad_time3c_pm2+workdays_sad_time3c_pm3+
                        workdays_sad_time3c_pm4+workdays_sad_time4a_am3+workdays_sad_time4a_pm2+
                        workdays_sad_time4a_pm3+workdays_sad_time4b_am2+workdays_sad_time4b_am3+
                        workdays_sad_time4b_pm1+workdays_sad_time4b_pm2+workdays_sad_time4b_pm3+
                        workdays_sad_time4c_am2+workdays_sad_time4c_am3+workdays_sad_time4c_pm1+
                        workdays_sad_time4c_pm3+workdays_sad_time4d_am2+workdays_sad_time4d_am3+
                        workdays_sad_time4d_pm1+workdays_sad_time4d_pm2+workdays_sad_time6a_am2+
                        workdays_sad_time6a_pm1+workdays_sad_time6b_am2+workdays_sad_time6b_pm1+
                        workdays_sad_time6b_pm2+workdays_sad_time6c_am2+workdays_sad_time6c_pm1+
                        workdays_sad_time6c_pm2+workdays_sad_time6e_am1+workdays_sad_time6e_pm1+
                        workdays_sad_time6f_am1+workdays_sad_time6f_am2+workdays_sad_time6f_pm1]

# 有效surprise情绪区间数据汇总
workdays_surprise_group = [workdays_surprise_time3a_am4+workdays_surprise_time3a_pm2+workdays_surprise_time3a_pm4+
                        workdays_surprise_time3b_am3+workdays_surprise_time3b_am4+workdays_surprise_time3b_pm1+
                        workdays_surprise_time3b_pm2+workdays_surprise_time3b_pm3+workdays_surprise_time3b_pm4+
                        workdays_surprise_time3c_am2+workdays_surprise_time3c_am3+workdays_surprise_time3c_am4+
                        workdays_surprise_time3c_pm1+workdays_surprise_time3c_pm2+workdays_surprise_time3c_pm3+
                        workdays_surprise_time3c_pm4+workdays_surprise_time4a_am3+workdays_surprise_time4a_pm2+
                        workdays_surprise_time4a_pm3+workdays_surprise_time4b_am2+workdays_surprise_time4b_am3+
                        workdays_surprise_time4b_pm1+workdays_surprise_time4b_pm2+workdays_surprise_time4b_pm3+
                        workdays_surprise_time4c_am2+workdays_surprise_time4c_am3+workdays_surprise_time4c_pm1+
                        workdays_surprise_time4c_pm3+workdays_surprise_time4d_am2+workdays_surprise_time4d_am3+
                        workdays_surprise_time4d_pm1+workdays_surprise_time4d_pm2+workdays_surprise_time6a_am2+
                        workdays_surprise_time6a_pm1+workdays_surprise_time6b_am2+workdays_surprise_time6b_pm1+
                        workdays_surprise_time6b_pm2+workdays_surprise_time6c_am2+workdays_surprise_time6c_pm1+
                        workdays_surprise_time6c_pm2+workdays_surprise_time6e_am1+workdays_surprise_time6e_pm1+
                        workdays_surprise_time6f_am1+workdays_surprise_time6f_am2+workdays_surprise_time6f_pm1]

# 有效love情绪区间数据汇总
workdays_love_group = [workdays_love_time3a_am4+workdays_love_time3a_pm2+workdays_love_time3a_pm4+
                        workdays_love_time3b_am3+workdays_love_time3b_am4+workdays_love_time3b_pm1+
                        workdays_love_time3b_pm2+workdays_love_time3b_pm3+workdays_love_time3b_pm4+
                        workdays_love_time3c_am2+workdays_love_time3c_am3+workdays_love_time3c_am4+
                        workdays_love_time3c_pm1+workdays_love_time3c_pm2+workdays_love_time3c_pm3+
                        workdays_love_time3c_pm4+workdays_love_time4a_am3+workdays_love_time4a_pm2+
                        workdays_love_time4a_pm3+workdays_love_time4b_am2+workdays_love_time4b_am3+
                        workdays_love_time4b_pm1+workdays_love_time4b_pm2+workdays_love_time4b_pm3+
                        workdays_love_time4c_am2+workdays_love_time4c_am3+workdays_love_time4c_pm1+
                        workdays_love_time4c_pm3+workdays_love_time4d_am2+workdays_love_time4d_am3+
                        workdays_love_time4d_pm1+workdays_love_time4d_pm2+workdays_love_time6a_am2+
                        workdays_love_time6a_pm1+workdays_love_time6b_am2+workdays_love_time6b_pm1+
                        workdays_love_time6b_pm2+workdays_love_time6c_am2+workdays_love_time6c_pm1+
                        workdays_love_time6c_pm2+workdays_love_time6e_am1+workdays_love_time6e_pm1+
                        workdays_love_time6f_am1+workdays_love_time6f_am2+workdays_love_time6f_pm1]

# 有效joy情绪区间数据汇总
workdays_joy_group = [workdays_joy_time3a_am4+workdays_joy_time3a_pm2+workdays_joy_time3a_pm4+
                        workdays_joy_time3b_am3+workdays_joy_time3b_am4+workdays_joy_time3b_pm1+
                        workdays_joy_time3b_pm2+workdays_joy_time3b_pm3+workdays_joy_time3b_pm4+
                        workdays_joy_time3c_am2+workdays_joy_time3c_am3+workdays_joy_time3c_am4+
                        workdays_joy_time3c_pm1+workdays_joy_time3c_pm2+workdays_joy_time3c_pm3+
                        workdays_joy_time3c_pm4+workdays_joy_time4a_am3+workdays_joy_time4a_pm2+
                        workdays_joy_time4a_pm3+workdays_joy_time4b_am2+workdays_joy_time4b_am3+
                        workdays_joy_time4b_pm1+workdays_joy_time4b_pm2+workdays_joy_time4b_pm3+
                        workdays_joy_time4c_am2+workdays_joy_time4c_am3+workdays_joy_time4c_pm1+
                        workdays_joy_time4c_pm3+workdays_joy_time4d_am2+workdays_joy_time4d_am3+
                        workdays_joy_time4d_pm1+workdays_joy_time4d_pm2+workdays_joy_time6a_am2+
                        workdays_joy_time6a_pm1+workdays_joy_time6b_am2+workdays_joy_time6b_pm1+
                        workdays_joy_time6b_pm2+workdays_joy_time6c_am2+workdays_joy_time6c_pm1+
                        workdays_joy_time6c_pm2+workdays_joy_time6e_am1+workdays_joy_time6e_pm1+
                        workdays_joy_time6f_am1+workdays_joy_time6f_am2+workdays_joy_time6f_pm1]

# 有效expectation情绪区间数据汇总
workdays_expectation_group = [workdays_expectation_time3a_am4+workdays_expectation_time3a_pm2+workdays_expectation_time3a_pm4+
                        workdays_expectation_time3b_am3+workdays_expectation_time3b_am4+workdays_expectation_time3b_pm1+
                        workdays_expectation_time3b_pm2+workdays_expectation_time3b_pm3+workdays_expectation_time3b_pm4+
                        workdays_expectation_time3c_am2+workdays_expectation_time3c_am3+workdays_expectation_time3c_am4+
                        workdays_expectation_time3c_pm1+workdays_expectation_time3c_pm2+workdays_expectation_time3c_pm3+
                        workdays_expectation_time3c_pm4+workdays_expectation_time4a_am3+workdays_expectation_time4a_pm2+
                        workdays_expectation_time4a_pm3+workdays_expectation_time4b_am2+workdays_expectation_time4b_am3+
                        workdays_expectation_time4b_pm1+workdays_expectation_time4b_pm2+workdays_expectation_time4b_pm3+
                        workdays_expectation_time4c_am2+workdays_expectation_time4c_am3+workdays_expectation_time4c_pm1+
                        workdays_expectation_time4c_pm3+workdays_expectation_time4d_am2+workdays_expectation_time4d_am3+
                        workdays_expectation_time4d_pm1+workdays_expectation_time4d_pm2+workdays_expectation_time6a_am2+
                        workdays_expectation_time6a_pm1+workdays_expectation_time6b_am2+workdays_expectation_time6b_pm1+
                        workdays_expectation_time6b_pm2+workdays_expectation_time6c_am2+workdays_expectation_time6c_pm1+
                        workdays_expectation_time6c_pm2+workdays_expectation_time6e_am1+workdays_expectation_time6e_pm1+
                        workdays_expectation_time6f_am1+workdays_expectation_time6f_am2+workdays_expectation_time6f_pm1]

# 有效时间标签构造（Morning=1,Afternoon=0,Night=-1）
# 注，需要在数据结构中改为category类型
workdays_time_group = [15*[1]+15*[0]+15*[-1]+
                       15*[1]+15*[1]+15*[0]+15*[0]+15*[0]+15*[-1]+
                       15*[1]+15*[1]+15*[1]+15*[0]+15*[0]+15*[-1]+15*[-1]+
                       20*[1]+20*[0]+20*[-1]+
                       20*[1]+20*[1]+20*[0]+20*[0]+20*[-1]+
                       20*[1]+20*[1]+20*[0]+20*[-1]+
                       20*[1]+20*[1]+20*[0]+20*[0]+
                       30*[1]+30*[0]+
                       30*[1]+30*[0]+30*[0]+
                       30*[1]+30*[0]+30*[-1]+
                       30*[1]+30*[0]+
                       30*[1]+30*[1]+30*[0]]

# 有效时区长度标签构造（三小时时区=3，四小时时区=4，六小时时区=6）
# 注，需要在数据结构中改为category类型
workdays_zone_group = [240*[3]+320*[4]+390*[6]]

In [11]:
# workdays中介效应测试结果
# 总体情绪——道德动机
workdays_emo_to_moral = test_media_process2(workdays_time_group[0],workdays_moral_group[0],
                                   workdays_emo_group[0],4,'Total_emotion(workdays)')
# 积极情绪——道德动机
workdays_positive_to_moral = test_media_process2(workdays_time_group[0],workdays_moral_group[0],
                                   workdays_positive_group[0],4,'Positive(workdays)')
# 消极情绪——道德动机
workdays_negative_to_moral = test_media_process2(workdays_time_group[0],workdays_moral_group[0],
                                   workdays_negative_group[0],4,'Negative(workdays)')
#*******************************************************************************
# anger——道德动机
workdays_anger_to_moral = test_media_process2(workdays_time_group[0],workdays_moral_group[0],
                                   workdays_anger_group[0],4,'Anger(workdays)')
# anxi——道德动机
workdays_anxi_to_moral = test_media_process2(workdays_time_group[0],workdays_moral_group[0],
                                   workdays_anxi_group[0],4,'Anxi(workdays)')
# hate——道德动机
workdays_hate_to_moral = test_media_process2(workdays_time_group[0],workdays_moral_group[0],
                                   workdays_hate_group[0],4,'Hate(workdays)')
# sad——道德动机
workdays_sad_to_moral = test_media_process2(workdays_time_group[0],workdays_moral_group[0],
                                   workdays_sad_group[0],4,'Sad(workdays)')

# surprise——道德动机
workdays_surprise_to_moral = test_media_process2(workdays_time_group[0],workdays_moral_group[0],
                                   workdays_surprise_group[0],4,'Surprise(workdays)')
#*******************************************************************************
# love——道德动机
workdays_love_to_moral = test_media_process2(workdays_time_group[0],workdays_moral_group[0],
                                   workdays_love_group[0],4,'Love(workdays)')
# joy——道德动机
workdays_joy_to_moral = test_media_process2(workdays_time_group[0],workdays_moral_group[0],
                                   workdays_joy_group[0],4,'Joy(workdays)')
# expectation——道德动机
workdays_expectation_to_moral = test_media_process2(workdays_time_group[0],workdays_moral_group[0],
                                   workdays_expectation_group[0],4,'Expectation(workdays)')

Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 4

Variables:
    Cons = Cons
    x = Time
    y = Moral
    m1 = Total_emotion(workdays)

Sample size:
950

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = Moral 
OLS Regression Summary

     R²  Adj. R²    MSE       F  df1  df2  p-value
 0.0358   0.0328 0.0576 17.6007    2  947   0.0000

Coefficients

                          coeff     se       t      p    LLCI   ULCI
Cons                     0.9370 0.0215 43.5639 0.0000  0.8948 0.9792
Time                    -0.0201 0.0118 -1.6991 0.0896 -0.0432 0.0031
Total_emotion(workdays)  0.0018 0.0004  4.5502 0.0000  0.0010 0.0026

---------------------------------------------

Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 4

Variables:
    Cons = Cons
    x = Time
    y = Moral
    m1 = Hate(workdays)

Sample size:
950

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = Moral 
OLS Regression Summary

     R²  Adj. R²    MSE      F  df1  df2  p-value
 0.0196   0.0165 0.0586 9.4750    2  947   0.0001

Coefficients

                 coeff     se       t      p    LLCI    ULCI
Cons            1.0115 0.0111 91.1807 0.0000  0.9898  1.0333
Time           -0.0344 0.0114 -3.0208 0.0026 -0.0568 -0.0121
Hate(workdays)  0.0180 0.0083  2.1662 0.0305  0.0017  0.0342

-------------------------------------------------------------------------

Outcome = Hat

Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 4

Variables:
    Cons = Cons
    x = Time
    y = Moral
    m1 = Expectation(workdays)

Sample size:
950

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = Moral 
OLS Regression Summary

     R²  Adj. R²    MSE      F  df1  df2  p-value
 0.0149   0.0118 0.0589 7.1500    2  947   0.0008

Coefficients

                        coeff     se       t      p    LLCI    ULCI
Cons                   1.0305 0.0130 79.3006 0.0000  1.0051  1.0560
Time                  -0.0417 0.0110 -3.7807 0.0002 -0.0634 -0.0201
Expectation(workdays) -0.0013 0.0040 -0.3335 0.7388 -0.0091  0.0065

-----------------------------------------------------

In [12]:
# 中介效应检测数据准备
# 三、weekend情境
# 有效道德区间
# （1）weekend_moral
# 3a （0-3hour，6-9hour，12-15hour，21-24hour）
weekend_moral_time3a_am1 = weekend_moral[0:6]
weekend_moral_time3a_am3 = weekend_moral[12:18]
weekend_moral_time3a_pm1 = weekend_moral[24:30]
weekend_moral_time3a_pm4 = weekend_moral[42:48]
# 3b （1-4hour，10-13hour，16-19hour，22-1hour）
weekend_moral_time3b_am1 = weekend_moral[2:8]
weekend_moral_time3b_am4 = weekend_moral[20:26]
weekend_moral_time3b_pm2 = weekend_moral[32:38]
weekend_moral_time3b_pm4 = weekend_moral[44:48]+weekend_moral[0:2]
# 4a （0-4hour，8-12hour，12-16hour）
weekend_moral_time4a_am1 = weekend_moral[0:8]
weekend_moral_time4a_am3 = weekend_moral[16:24]
weekend_moral_time4a_pm1 = weekend_moral[24:32]
# 4d （7-11hour，11-15hour，23-3hour）
weekend_moral_time4d_am2 = weekend_moral[14:22]
weekend_moral_time4d_am3 = weekend_moral[22:30]
weekend_moral_time4d_pm3 = weekend_moral[46:48] + weekend_moral[0:6]

# （2）有效情绪区间（总情绪）
# 3a （0-3hour，6-9hour，12-15hour，21-24hour）
weekend_emo_time3a_am1 = weekend_total_emo[0:6]
weekend_emo_time3a_am3 = weekend_total_emo[12:18]
weekend_emo_time3a_pm1 = weekend_total_emo[24:30]
weekend_emo_time3a_pm4 = weekend_total_emo[42:48]
# 3b （1-4hour，10-13hour，16-19hour，22-1hour）
weekend_emo_time3b_am1 = weekend_total_emo[2:8]
weekend_emo_time3b_am4 = weekend_total_emo[20:26]
weekend_emo_time3b_pm2 = weekend_total_emo[32:38]
weekend_emo_time3b_pm4 = weekend_total_emo[44:48]+weekend_total_emo[0:2]
# 4a （0-4hour，8-12hour，12-16hour）
weekend_emo_time4a_am1 = weekend_total_emo[0:8]
weekend_emo_time4a_am3 = weekend_total_emo[16:24]
weekend_emo_time4a_pm1 = weekend_total_emo[24:32]
# 4d （7-11hour，11-15hour，23-3hour）
weekend_emo_time4d_am2 = weekend_total_emo[14:22]
weekend_emo_time4d_am3 = weekend_total_emo[22:30]
weekend_emo_time4d_pm3 = weekend_total_emo[46:48] + weekend_total_emo[0:6]

# （3）有效积极情绪区间（positive）
# 3a （0-3hour，6-9hour，12-15hour，21-24hour）
weekend_positive_time3a_am1 = weekend_positive_emo[0:6]
weekend_positive_time3a_am3 = weekend_positive_emo[12:18]
weekend_positive_time3a_pm1 = weekend_positive_emo[24:30]
weekend_positive_time3a_pm4 = weekend_positive_emo[42:48]
# 3b （1-4hour，10-13hour，16-19hour，22-1hour）
weekend_positive_time3b_am1 = weekend_positive_emo[2:8]
weekend_positive_time3b_am4 = weekend_positive_emo[20:26]
weekend_positive_time3b_pm2 = weekend_positive_emo[32:38]
weekend_positive_time3b_pm4 = weekend_positive_emo[44:48]+weekend_positive_emo[0:2]
# 4a （0-4hour，8-12hour，12-16hour）
weekend_positive_time4a_am1 = weekend_positive_emo[0:8]
weekend_positive_time4a_am3 = weekend_positive_emo[16:24]
weekend_positive_time4a_pm1 = weekend_positive_emo[24:32]
# 4d （7-11hour，11-15hour，23-3hour）
weekend_positive_time4d_am2 = weekend_positive_emo[14:22]
weekend_positive_time4d_am3 = weekend_positive_emo[22:30]
weekend_positive_time4d_pm3 = weekend_positive_emo[46:48] + weekend_positive_emo[0:6]

# （4）有效消极情绪区间（negative）
# 3a （0-3hour，6-9hour，12-15hour，21-24hour）
weekend_negative_time3a_am1 = weekend_negative_emo[0:6]
weekend_negative_time3a_am3 = weekend_negative_emo[12:18]
weekend_negative_time3a_pm1 = weekend_negative_emo[24:30]
weekend_negative_time3a_pm4 = weekend_negative_emo[42:48]
# 3b （1-4hour，10-13hour，16-19hour，22-1hour）
weekend_negative_time3b_am1 = weekend_negative_emo[2:8]
weekend_negative_time3b_am4 = weekend_negative_emo[20:26]
weekend_negative_time3b_pm2 = weekend_negative_emo[32:38]
weekend_negative_time3b_pm4 = weekend_negative_emo[44:48]+weekend_negative_emo[0:2]
# 4a （0-4hour，8-12hour，12-16hour）
weekend_negative_time4a_am1 = weekend_negative_emo[0:8]
weekend_negative_time4a_am3 = weekend_negative_emo[16:24]
weekend_negative_time4a_pm1 = weekend_negative_emo[24:32]
# 4d （7-11hour，11-15hour，23-3hour）
weekend_negative_time4d_am2 = weekend_negative_emo[14:22]
weekend_negative_time4d_am3 = weekend_negative_emo[22:30]
weekend_negative_time4d_pm3 = weekend_negative_emo[46:48] + weekend_negative_emo[0:6]

# (5) 有效anger情绪区间
# 3a （0-3hour，6-9hour，12-15hour，21-24hour）
weekend_anger_time3a_am1 = weekend_li_anger[0:6]
weekend_anger_time3a_am3 = weekend_li_anger[12:18]
weekend_anger_time3a_pm1 = weekend_li_anger[24:30]
weekend_anger_time3a_pm4 = weekend_li_anger[42:48]
# 3b （1-4hour，10-13hour，16-19hour，22-1hour）
weekend_anger_time3b_am1 = weekend_li_anger[2:8]
weekend_anger_time3b_am4 = weekend_li_anger[20:26]
weekend_anger_time3b_pm2 = weekend_li_anger[32:38]
weekend_anger_time3b_pm4 = weekend_li_anger[44:48]+weekend_li_anger[0:2]
# 4a （0-4hour，8-12hour，12-16hour）
weekend_anger_time4a_am1 = weekend_li_anger[0:8]
weekend_anger_time4a_am3 = weekend_li_anger[16:24]
weekend_anger_time4a_pm1 = weekend_li_anger[24:32]
# 4d （7-11hour，11-15hour，23-3hour）
weekend_anger_time4d_am2 = weekend_li_anger[14:22]
weekend_anger_time4d_am3 = weekend_li_anger[22:30]
weekend_anger_time4d_pm3 = weekend_li_anger[46:48] + weekend_li_anger[0:6]

# (6) 有效anxi情绪区间
# 3a （0-3hour，6-9hour，12-15hour，21-24hour）
weekend_anxi_time3a_am1 = weekend_li_anxi[0:6]
weekend_anxi_time3a_am3 = weekend_li_anxi[12:18]
weekend_anxi_time3a_pm1 = weekend_li_anxi[24:30]
weekend_anxi_time3a_pm4 = weekend_li_anxi[42:48]
# 3b （1-4hour，10-13hour，16-19hour，22-1hour）
weekend_anxi_time3b_am1 = weekend_li_anxi[2:8]
weekend_anxi_time3b_am4 = weekend_li_anxi[20:26]
weekend_anxi_time3b_pm2 = weekend_li_anxi[32:38]
weekend_anxi_time3b_pm4 = weekend_li_anxi[44:48]+weekend_li_anxi[0:2]
# 4a （0-4hour，8-12hour，12-16hour）
weekend_anxi_time4a_am1 = weekend_li_anxi[0:8]
weekend_anxi_time4a_am3 = weekend_li_anxi[16:24]
weekend_anxi_time4a_pm1 = weekend_li_anxi[24:32]
# 4d （7-11hour，11-15hour，23-3hour）
weekend_anxi_time4d_am2 = weekend_li_anxi[14:22]
weekend_anxi_time4d_am3 = weekend_li_anxi[22:30]
weekend_anxi_time4d_pm3 = weekend_li_anxi[46:48] + weekend_li_anxi[0:6]

# (7) 有效hate情绪区间
# 3a （0-3hour，6-9hour，12-15hour，21-24hour）
weekend_hate_time3a_am1 = weekend_li_hate[0:6]
weekend_hate_time3a_am3 = weekend_li_hate[12:18]
weekend_hate_time3a_pm1 = weekend_li_hate[24:30]
weekend_hate_time3a_pm4 = weekend_li_hate[42:48]
# 3b （1-4hour，10-13hour，16-19hour，22-1hour）
weekend_hate_time3b_am1 = weekend_li_hate[2:8]
weekend_hate_time3b_am4 = weekend_li_hate[20:26]
weekend_hate_time3b_pm2 = weekend_li_hate[32:38]
weekend_hate_time3b_pm4 = weekend_li_hate[44:48]+weekend_li_hate[0:2]
# 4a （0-4hour，8-12hour，12-16hour）
weekend_hate_time4a_am1 = weekend_li_hate[0:8]
weekend_hate_time4a_am3 = weekend_li_hate[16:24]
weekend_hate_time4a_pm1 = weekend_li_hate[24:32]
# 4d （7-11hour，11-15hour，23-3hour）
weekend_hate_time4d_am2 = weekend_li_hate[14:22]
weekend_hate_time4d_am3 = weekend_li_hate[22:30]
weekend_hate_time4d_pm3 = weekend_li_hate[46:48] + weekend_li_hate[0:6]

# (8) 有效sad情绪区间
# 3a （0-3hour，6-9hour，12-15hour，21-24hour）
weekend_sad_time3a_am1 = weekend_li_sad[0:6]
weekend_sad_time3a_am3 = weekend_li_sad[12:18]
weekend_sad_time3a_pm1 = weekend_li_sad[24:30]
weekend_sad_time3a_pm4 = weekend_li_sad[42:48]
# 3b （1-4hour，10-13hour，16-19hour，22-1hour）
weekend_sad_time3b_am1 = weekend_li_sad[2:8]
weekend_sad_time3b_am4 = weekend_li_sad[20:26]
weekend_sad_time3b_pm2 = weekend_li_sad[32:38]
weekend_sad_time3b_pm4 = weekend_li_sad[44:48]+weekend_li_sad[0:2]
# 4a （0-4hour，8-12hour，12-16hour）
weekend_sad_time4a_am1 = weekend_li_sad[0:8]
weekend_sad_time4a_am3 = weekend_li_sad[16:24]
weekend_sad_time4a_pm1 = weekend_li_sad[24:32]
# 4d （7-11hour，11-15hour，23-3hour）
weekend_sad_time4d_am2 = weekend_li_sad[14:22]
weekend_sad_time4d_am3 = weekend_li_sad[22:30]
weekend_sad_time4d_pm3 = weekend_li_sad[46:48] + weekend_li_sad[0:6]

# (9) 有效surprise情绪区间
# 3a （0-3hour，6-9hour，12-15hour，21-24hour）
weekend_surprise_time3a_am1 = weekend_li_surprise[0:6]
weekend_surprise_time3a_am3 = weekend_li_surprise[12:18]
weekend_surprise_time3a_pm1 = weekend_li_surprise[24:30]
weekend_surprise_time3a_pm4 = weekend_li_surprise[42:48]
# 3b （1-4hour，10-13hour，16-19hour，22-1hour）
weekend_surprise_time3b_am1 = weekend_li_surprise[2:8]
weekend_surprise_time3b_am4 = weekend_li_surprise[20:26]
weekend_surprise_time3b_pm2 = weekend_li_surprise[32:38]
weekend_surprise_time3b_pm4 = weekend_li_surprise[44:48]+weekend_li_surprise[0:2]
# 4a （0-4hour，8-12hour，12-16hour）
weekend_surprise_time4a_am1 = weekend_li_surprise[0:8]
weekend_surprise_time4a_am3 = weekend_li_surprise[16:24]
weekend_surprise_time4a_pm1 = weekend_li_surprise[24:32]
# 4d （7-11hour，11-15hour，23-3hour）
weekend_surprise_time4d_am2 = weekend_li_surprise[14:22]
weekend_surprise_time4d_am3 = weekend_li_surprise[22:30]
weekend_surprise_time4d_pm3 = weekend_li_surprise[46:48] + weekend_li_surprise[0:6]

# (10) 有效love情绪区间
# 3a （0-3hour，6-9hour，12-15hour，21-24hour）
weekend_love_time3a_am1 = weekend_li_love[0:6]
weekend_love_time3a_am3 = weekend_li_love[12:18]
weekend_love_time3a_pm1 = weekend_li_love[24:30]
weekend_love_time3a_pm4 = weekend_li_love[42:48]
# 3b （1-4hour，10-13hour，16-19hour，22-1hour）
weekend_love_time3b_am1 = weekend_li_love[2:8]
weekend_love_time3b_am4 = weekend_li_love[20:26]
weekend_love_time3b_pm2 = weekend_li_love[32:38]
weekend_love_time3b_pm4 = weekend_li_love[44:48]+weekend_li_love[0:2]
# 4a （0-4hour，8-12hour，12-16hour）
weekend_love_time4a_am1 = weekend_li_love[0:8]
weekend_love_time4a_am3 = weekend_li_love[16:24]
weekend_love_time4a_pm1 = weekend_li_love[24:32]
# 4d （7-11hour，11-15hour，23-3hour）
weekend_love_time4d_am2 = weekend_li_love[14:22]
weekend_love_time4d_am3 = weekend_li_love[22:30]
weekend_love_time4d_pm3 = weekend_li_love[46:48] + weekend_li_love[0:6]

# (11) 有效joy情绪区间
# 3a （0-3hour，6-9hour，12-15hour，21-24hour）
weekend_joy_time3a_am1 = weekend_li_joy[0:6]
weekend_joy_time3a_am3 = weekend_li_joy[12:18]
weekend_joy_time3a_pm1 = weekend_li_joy[24:30]
weekend_joy_time3a_pm4 = weekend_li_joy[42:48]
# 3b （1-4hour，10-13hour，16-19hour，22-1hour）
weekend_joy_time3b_am1 = weekend_li_joy[2:8]
weekend_joy_time3b_am4 = weekend_li_joy[20:26]
weekend_joy_time3b_pm2 = weekend_li_joy[32:38]
weekend_joy_time3b_pm4 = weekend_li_joy[44:48]+weekend_li_joy[0:2]
# 4a （0-4hour，8-12hour，12-16hour）
weekend_joy_time4a_am1 = weekend_li_joy[0:8]
weekend_joy_time4a_am3 = weekend_li_joy[16:24]
weekend_joy_time4a_pm1 = weekend_li_joy[24:32]
# 4d （7-11hour，11-15hour，23-3hour）
weekend_joy_time4d_am2 = weekend_li_joy[14:22]
weekend_joy_time4d_am3 = weekend_li_joy[22:30]
weekend_joy_time4d_pm3 = weekend_li_joy[46:48] + weekend_li_joy[0:6]

# (12) 有效expectation情绪区间
# 3a （0-3hour，6-9hour，12-15hour，21-24hour）
weekend_expectation_time3a_am1 = weekend_li_expectation[0:6]
weekend_expectation_time3a_am3 = weekend_li_expectation[12:18]
weekend_expectation_time3a_pm1 = weekend_li_expectation[24:30]
weekend_expectation_time3a_pm4 = weekend_li_expectation[42:48]
# 3b （1-4hour，10-13hour，16-19hour，22-1hour）
weekend_expectation_time3b_am1 = weekend_li_expectation[2:8]
weekend_expectation_time3b_am4 = weekend_li_expectation[20:26]
weekend_expectation_time3b_pm2 = weekend_li_expectation[32:38]
weekend_expectation_time3b_pm4 = weekend_li_expectation[44:48]+weekend_li_expectation[0:2]
# 4a （0-4hour，8-12hour，12-16hour）
weekend_expectation_time4a_am1 = weekend_li_expectation[0:8]
weekend_expectation_time4a_am3 = weekend_li_expectation[16:24]
weekend_expectation_time4a_pm1 = weekend_li_expectation[24:32]
# 4d （7-11hour，11-15hour，23-3hour）
weekend_expectation_time4d_am2 = weekend_li_expectation[14:22]
weekend_expectation_time4d_am3 = weekend_li_expectation[22:30]
weekend_expectation_time4d_pm3 = weekend_li_expectation[46:48] + weekend_li_expectation[0:6]

# 有效道德区间数据汇总
weekend_moral_group = [weekend_moral_time3a_am1+weekend_moral_time3a_am3+weekend_moral_time3a_pm1+
                       weekend_moral_time3a_pm4+weekend_moral_time3b_am1+weekend_moral_time3b_am4+
                       weekend_moral_time3b_pm2+weekend_moral_time3b_pm4+weekend_moral_time4a_am1+
                       weekend_moral_time4a_am3+weekend_moral_time4a_pm1+weekend_moral_time4d_am2+
                       weekend_moral_time4d_am3+weekend_moral_time4d_pm3]


# 有效情绪区间数据汇总（总情绪）
weekend_emo_group = [weekend_emo_time3a_am1+weekend_emo_time3a_am3+weekend_emo_time3a_pm1+
                       weekend_emo_time3a_pm4+weekend_emo_time3b_am1+weekend_emo_time3b_am4+
                       weekend_emo_time3b_pm2+weekend_emo_time3b_pm4+weekend_emo_time4a_am1+
                       weekend_emo_time4a_am3+weekend_emo_time4a_pm1+weekend_emo_time4d_am2+
                       weekend_emo_time4d_am3+weekend_emo_time4d_pm3]

# 有效情绪区间数据汇总（积极情绪positive）
weekend_positive_group = [weekend_positive_time3a_am1+weekend_positive_time3a_am3+weekend_positive_time3a_pm1+
                       weekend_positive_time3a_pm4+weekend_positive_time3b_am1+weekend_positive_time3b_am4+
                       weekend_positive_time3b_pm2+weekend_positive_time3b_pm4+weekend_positive_time4a_am1+
                       weekend_positive_time4a_am3+weekend_positive_time4a_pm1+weekend_positive_time4d_am2+
                       weekend_positive_time4d_am3+weekend_positive_time4d_pm3]

# 有效情绪区间数据汇总（消极情绪negative）
weekend_negative_group = [weekend_negative_time3a_am1+weekend_negative_time3a_am3+weekend_negative_time3a_pm1+
                       weekend_negative_time3a_pm4+weekend_negative_time3b_am1+weekend_negative_time3b_am4+
                       weekend_negative_time3b_pm2+weekend_negative_time3b_pm4+weekend_negative_time4a_am1+
                       weekend_negative_time4a_am3+weekend_negative_time4a_pm1+weekend_negative_time4d_am2+
                       weekend_negative_time4d_am3+weekend_negative_time4d_pm3]

# 有效anger情绪区间数据汇总
weekend_anger_group = [weekend_anger_time3a_am1+weekend_anger_time3a_am3+weekend_anger_time3a_pm1+
                       weekend_anger_time3a_pm4+weekend_anger_time3b_am1+weekend_anger_time3b_am4+
                       weekend_anger_time3b_pm2+weekend_anger_time3b_pm4+weekend_anger_time4a_am1+
                       weekend_anger_time4a_am3+weekend_anger_time4a_pm1+weekend_anger_time4d_am2+
                       weekend_anger_time4d_am3+weekend_anger_time4d_pm3]

# 有效anxi情绪区间数据汇总
weekend_anxi_group = [weekend_anxi_time3a_am1+weekend_anxi_time3a_am3+weekend_anxi_time3a_pm1+
                       weekend_anxi_time3a_pm4+weekend_anxi_time3b_am1+weekend_anxi_time3b_am4+
                       weekend_anxi_time3b_pm2+weekend_anxi_time3b_pm4+weekend_anxi_time4a_am1+
                       weekend_anxi_time4a_am3+weekend_anxi_time4a_pm1+weekend_anxi_time4d_am2+
                       weekend_anxi_time4d_am3+weekend_anxi_time4d_pm3]

# 有效hate情绪区间数据汇总
weekend_hate_group = [weekend_hate_time3a_am1+weekend_hate_time3a_am3+weekend_hate_time3a_pm1+
                       weekend_hate_time3a_pm4+weekend_hate_time3b_am1+weekend_hate_time3b_am4+
                       weekend_hate_time3b_pm2+weekend_hate_time3b_pm4+weekend_hate_time4a_am1+
                       weekend_hate_time4a_am3+weekend_hate_time4a_pm1+weekend_hate_time4d_am2+
                       weekend_hate_time4d_am3+weekend_hate_time4d_pm3]

# 有效sad情绪区间数据汇总
weekend_sad_group = [weekend_sad_time3a_am1+weekend_sad_time3a_am3+weekend_sad_time3a_pm1+
                       weekend_sad_time3a_pm4+weekend_sad_time3b_am1+weekend_sad_time3b_am4+
                       weekend_sad_time3b_pm2+weekend_sad_time3b_pm4+weekend_sad_time4a_am1+
                       weekend_sad_time4a_am3+weekend_sad_time4a_pm1+weekend_sad_time4d_am2+
                       weekend_sad_time4d_am3+weekend_sad_time4d_pm3]

# 有效surprise情绪区间数据汇总
weekend_surprise_group = [weekend_surprise_time3a_am1+weekend_surprise_time3a_am3+weekend_surprise_time3a_pm1+
                       weekend_surprise_time3a_pm4+weekend_surprise_time3b_am1+weekend_surprise_time3b_am4+
                       weekend_surprise_time3b_pm2+weekend_surprise_time3b_pm4+weekend_surprise_time4a_am1+
                       weekend_surprise_time4a_am3+weekend_surprise_time4a_pm1+weekend_surprise_time4d_am2+
                       weekend_surprise_time4d_am3+weekend_surprise_time4d_pm3]

# 有效love情绪区间数据汇总
weekend_love_group = [weekend_love_time3a_am1+weekend_love_time3a_am3+weekend_love_time3a_pm1+
                       weekend_love_time3a_pm4+weekend_love_time3b_am1+weekend_love_time3b_am4+
                       weekend_love_time3b_pm2+weekend_love_time3b_pm4+weekend_love_time4a_am1+
                       weekend_love_time4a_am3+weekend_love_time4a_pm1+weekend_love_time4d_am2+
                       weekend_love_time4d_am3+weekend_love_time4d_pm3]

# 有效joy情绪区间数据汇总
weekend_joy_group = [weekend_joy_time3a_am1+weekend_joy_time3a_am3+weekend_joy_time3a_pm1+
                       weekend_joy_time3a_pm4+weekend_joy_time3b_am1+weekend_joy_time3b_am4+
                       weekend_joy_time3b_pm2+weekend_joy_time3b_pm4+weekend_joy_time4a_am1+
                       weekend_joy_time4a_am3+weekend_joy_time4a_pm1+weekend_joy_time4d_am2+
                       weekend_joy_time4d_am3+weekend_joy_time4d_pm3]

# 有效expectation情绪区间数据汇总
weekend_expectation_group = [weekend_expectation_time3a_am1+weekend_expectation_time3a_am3+weekend_expectation_time3a_pm1+
                       weekend_expectation_time3a_pm4+weekend_expectation_time3b_am1+weekend_expectation_time3b_am4+
                       weekend_expectation_time3b_pm2+weekend_expectation_time3b_pm4+weekend_expectation_time4a_am1+
                       weekend_expectation_time4a_am3+weekend_expectation_time4a_pm1+weekend_expectation_time4d_am2+
                       weekend_expectation_time4d_am3+weekend_expectation_time4d_pm3]

# 有效时间标签构造（Morning=1,Afternoon=0,Night=-1）
# 注，需要在数据结构中改为category类型
weekend_time_group = [6*[-1]+6*[1]+6*[0]+6*[-1]+
                      6*[-1]+6*[1]+6*[0]+6*[-1]+
                      8*[-1]+8*[1]+8*[0]+
                      8*[1]+8*[1]+8*[-1]]

# 有效时区长度标签构造（三小时时区=3，四小时时区=4，六小时时区=6）
# 注，需要在数据结构中改为category类型
weekend_zone_group = [48*[3]+48*[4]]

In [13]:
# weekend中介效应测试结果
# 总体情绪——道德动机
weekend_emo_to_moral = test_media_process2(weekend_time_group[0],weekend_moral_group[0],
                                   weekend_emo_group[0],4,'Total_emotion(weekend)')
# 积极情绪——道德动机
weekend_positive_to_moral = test_media_process2(weekend_time_group[0],weekend_moral_group[0],
                                   weekend_positive_group[0],4,'Positive(weekend)')
# 消极情绪——道德动机
weekend_negative_to_moral = test_media_process2(weekend_time_group[0],weekend_moral_group[0],
                                   weekend_negative_group[0],4,'Negative(weekend)')

#*******************************************************************************
# anger——道德动机
weekend_anger_to_moral = test_media_process2(weekend_time_group[0],weekend_moral_group[0],
                                   weekend_anger_group[0],4,'Anger(weekend)')
# anxi——道德动机
weekend_anxi_to_moral = test_media_process2(weekend_time_group[0],weekend_moral_group[0],
                                   weekend_anxi_group[0],4,'Anxi(weekend)')
# hate——道德动机
weekend_hate_to_moral = test_media_process2(weekend_time_group[0],weekend_moral_group[0],
                                   weekend_hate_group[0],4,'Hate(weekend)')
# sad——道德动机
weekend_sad_to_moral = test_media_process2(weekend_time_group[0],weekend_moral_group[0],
                                   weekend_sad_group[0],4,'Sad(weekend)')
# surprise——道德动机
weekend_surprise_to_moral = test_media_process2(weekend_time_group[0],weekend_moral_group[0],
                                   weekend_surprise_group[0],4,'Surprise(weekend)')
#*******************************************************************************
# love——道德动机
weekend_love_to_moral = test_media_process2(weekend_time_group[0],weekend_moral_group[0],
                                   weekend_love_group[0],4,'Love(weekend)')
# joy——道德动机
weekend_joy_to_moral = test_media_process2(weekend_time_group[0],weekend_moral_group[0],
                                   weekend_joy_group[0],4,'Joy(weekend)')
# expectation——道德动机
weekend_expectation_to_moral = test_media_process2(weekend_time_group[0],weekend_moral_group[0],
                                   weekend_expectation_group[0],4,'Expectation(weekend)')


Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 4

Variables:
    Cons = Cons
    x = Time
    y = Moral
    m1 = Total_emotion(weekend)

Sample size:
96

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = Moral 
OLS Regression Summary

     R²  Adj. R²    MSE      F  df1  df2  p-value
 0.1648   0.1376 0.0568 9.1783    2   93   0.0002

Coefficients

                         coeff     se       t      p    LLCI    ULCI
Cons                    1.2105 0.0485 24.9677 0.0000  1.1155  1.3056
Time                   -0.0655 0.0280 -2.3437 0.0212 -0.1203 -0.0107
Total_emotion(weekend) -0.0026 0.0009 -3.0356 0.0031 -0.0043 -0.0009

-------------------------------------------------

Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 4

Variables:
    Cons = Cons
    x = Time
    y = Moral
    m1 = Hate(weekend)

Sample size:
96

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = Moral 
OLS Regression Summary

     R²  Adj. R²    MSE      F  df1  df2  p-value
 0.0998   0.0704 0.0613 5.1552    2   93   0.0075

Coefficients

                coeff     se       t      p    LLCI    ULCI
Cons           1.1099 0.0321 34.6182 0.0000  1.0471  1.1728
Time          -0.0742 0.0291 -2.5468 0.0125 -0.1313 -0.0171
Hate(weekend) -0.0421 0.0311 -1.3525 0.1795 -0.1030  0.0189

-------------------------------------------------------------------------

Outcome = Hate(week

Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 4

Variables:
    Cons = Cons
    x = Time
    y = Moral
    m1 = Expectation(weekend)

Sample size:
96

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = Moral 
OLS Regression Summary

     R²  Adj. R²    MSE      F  df1  df2  p-value
 0.1211   0.0925 0.0598 6.4079    2   93   0.0025

Coefficients

                       coeff     se       t      p    LLCI    ULCI
Cons                  1.1519 0.0420 27.4178 0.0000  1.0696  1.2342
Time                 -0.0863 0.0281 -3.0674 0.0028 -0.1415 -0.0312
Expectation(weekend) -0.0272 0.0134 -2.0320 0.0450 -0.0534 -0.0010

-----------------------------------------------------------

In [14]:
# 中介效应检测数据准备
# 四、holiday情境
# 有效道德区间
# （1）holiday_moral
# 3a （3-6hour，9-12hour，12-15hour，15-18hour，18-21hour，21-24hour）
holiday_moral_time3a_am2 = holiday_moral[15:30]
holiday_moral_time3a_am4 = holiday_moral[45:60]
holiday_moral_time3a_pm1 = holiday_moral[60:75]
holiday_moral_time3a_pm2 = holiday_moral[75:90]
holiday_moral_time3a_pm3 = holiday_moral[90:105]
holiday_moral_time3a_pm4 = holiday_moral[105:120]
# 3b （1-4hour，7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
holiday_moral_time3b_am1 = holiday_moral[5:20]
holiday_moral_time3b_am3 = holiday_moral[35:50]
holiday_moral_time3b_am4 = holiday_moral[50:65]
holiday_moral_time3b_pm1 = holiday_moral[65:80]
holiday_moral_time3b_pm2 = holiday_moral[80:95]
holiday_moral_time3b_pm3 = holiday_moral[95:110]
holiday_moral_time3b_pm4 = holiday_moral[110:120]+holiday_moral[0:5]
# 3c （2-5hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
holiday_moral_time3c_am1 = holiday_moral[10:25]
holiday_moral_time3c_am4 = holiday_moral[55:70]
holiday_moral_time3c_pm1 = holiday_moral[70:85]
holiday_moral_time3c_pm2 = holiday_moral[85:100]
holiday_moral_time3c_pm3 = holiday_moral[100:115]
holiday_moral_time3c_pm4 = holiday_moral[115:120]+holiday_moral[0:10]
# 4b （1-5hour，9-13hour，13-17hour，17-21hour，21-1hour）
holiday_moral_time4b_am1 = holiday_moral[5:25]
holiday_moral_time4b_am3 = holiday_moral[45:65]
holiday_moral_time4b_pm1 = holiday_moral[65:85]
holiday_moral_time4b_pm2 = holiday_moral[85:105]
holiday_moral_time4b_pm3 = holiday_moral[105:120] + holiday_moral[0:5]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
holiday_moral_time4c_am1 = holiday_moral[10:30]
holiday_moral_time4c_am3 = holiday_moral[50:70]
holiday_moral_time4c_pm1 = holiday_moral[70:90]
holiday_moral_time4c_pm2 = holiday_moral[90:110]
holiday_moral_time4c_pm3 = holiday_moral[110:120] + holiday_moral[0:10]
# 6a （0-6hour，12-18hour）
holiday_moral_time6a_am1 = holiday_moral[0:30]
holiday_moral_time6a_pm1 = holiday_moral[60:90]
# 6b （1-7hour，19-1hour）
holiday_moral_time6b_am1 = holiday_moral[5:35]
holiday_moral_time6b_pm2 = holiday_moral[95:120]+holiday_moral[0:5]
# 6c （2-8hour，8-14hour，14-20hour，20-2hour）
holiday_moral_time6c_am1 = holiday_moral[10:40]
holiday_moral_time6c_am2 = holiday_moral[40:70]
holiday_moral_time6c_pm1 = holiday_moral[70:100]
holiday_moral_time6c_pm2 = holiday_moral[100:120]+holiday_moral[0:10]
# 6d （3-9hour，9-15hour）
holiday_moral_time6d_am1 = holiday_moral[15:45]
holiday_moral_time6d_am2 = holiday_moral[45:75]
# 6e （4-10hour，10-16hour）
holiday_moral_time6e_am1 = holiday_moral[20:50]
holiday_moral_time6e_am2 = holiday_moral[50:80]


# （2）有效情绪区间（总情绪）
# 3a （3-6hour，9-12hour，12-15hour，15-18hour，18-21hour，21-24hour）
holiday_emo_time3a_am2 = holiday_total_emo[15:30]
holiday_emo_time3a_am4 = holiday_total_emo[45:60]
holiday_emo_time3a_pm1 = holiday_total_emo[60:75]
holiday_emo_time3a_pm2 = holiday_total_emo[75:90]
holiday_emo_time3a_pm3 = holiday_total_emo[90:105]
holiday_emo_time3a_pm4 = holiday_total_emo[105:120]
# 3b （1-4hour，7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
holiday_emo_time3b_am1 = holiday_total_emo[5:20]
holiday_emo_time3b_am3 = holiday_total_emo[35:50]
holiday_emo_time3b_am4 = holiday_total_emo[50:65]
holiday_emo_time3b_pm1 = holiday_total_emo[65:80]
holiday_emo_time3b_pm2 = holiday_total_emo[80:95]
holiday_emo_time3b_pm3 = holiday_total_emo[95:110]
holiday_emo_time3b_pm4 = holiday_total_emo[110:120]+holiday_total_emo[0:5]
# 3c （2-5hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
holiday_emo_time3c_am1 = holiday_total_emo[10:25]
holiday_emo_time3c_am4 = holiday_total_emo[55:70]
holiday_emo_time3c_pm1 = holiday_total_emo[70:85]
holiday_emo_time3c_pm2 = holiday_total_emo[85:100]
holiday_emo_time3c_pm3 = holiday_total_emo[100:115]
holiday_emo_time3c_pm4 = holiday_total_emo[115:120]+holiday_total_emo[0:10]
# 4b （1-5hour，9-13hour，13-17hour，17-21hour，21-1hour）
holiday_emo_time4b_am1 = holiday_total_emo[5:25]
holiday_emo_time4b_am3 = holiday_total_emo[45:65]
holiday_emo_time4b_pm1 = holiday_total_emo[65:85]
holiday_emo_time4b_pm2 = holiday_total_emo[85:105]
holiday_emo_time4b_pm3 = holiday_total_emo[105:120] + holiday_total_emo[0:5]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
holiday_emo_time4c_am1 = holiday_total_emo[10:30]
holiday_emo_time4c_am3 = holiday_total_emo[50:70]
holiday_emo_time4c_pm1 = holiday_total_emo[70:90]
holiday_emo_time4c_pm2 = holiday_total_emo[90:110]
holiday_emo_time4c_pm3 = holiday_total_emo[110:120] + holiday_total_emo[0:10]
# 6a （0-6hour，12-18hour）
holiday_emo_time6a_am1 = holiday_total_emo[0:30]
holiday_emo_time6a_pm1 = holiday_total_emo[60:90]
# 6b （1-7hour，19-1hour）
holiday_emo_time6b_am1 = holiday_total_emo[5:35]
holiday_emo_time6b_pm2 = holiday_total_emo[95:120]+holiday_total_emo[0:5]
# 6c （2-8hour，8-14hour，14-20hour，20-2hour）
holiday_emo_time6c_am1 = holiday_total_emo[10:40]
holiday_emo_time6c_am2 = holiday_total_emo[40:70]
holiday_emo_time6c_pm1 = holiday_total_emo[70:100]
holiday_emo_time6c_pm2 = holiday_total_emo[100:120]+holiday_total_emo[0:10]
# 6d （3-9hour，9-15hour）
holiday_emo_time6d_am1 = holiday_total_emo[15:45]
holiday_emo_time6d_am2 = holiday_total_emo[45:75]
# 6e （4-10hour，10-16hour）
holiday_emo_time6e_am1 = holiday_total_emo[20:50]
holiday_emo_time6e_am2 = holiday_total_emo[50:80]


# （3）有效积极情绪区间（positive）
# 3a （3-6hour，9-12hour，12-15hour，15-18hour，18-21hour，21-24hour）
holiday_positive_time3a_am2 = holiday_positive_emo[15:30]
holiday_positive_time3a_am4 = holiday_positive_emo[45:60]
holiday_positive_time3a_pm1 = holiday_positive_emo[60:75]
holiday_positive_time3a_pm2 = holiday_positive_emo[75:90]
holiday_positive_time3a_pm3 = holiday_positive_emo[90:105]
holiday_positive_time3a_pm4 = holiday_positive_emo[105:120]
# 3b （1-4hour，7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
holiday_positive_time3b_am1 = holiday_positive_emo[5:20]
holiday_positive_time3b_am3 = holiday_positive_emo[35:50]
holiday_positive_time3b_am4 = holiday_positive_emo[50:65]
holiday_positive_time3b_pm1 = holiday_positive_emo[65:80]
holiday_positive_time3b_pm2 = holiday_positive_emo[80:95]
holiday_positive_time3b_pm3 = holiday_positive_emo[95:110]
holiday_positive_time3b_pm4 = holiday_positive_emo[110:120]+holiday_positive_emo[0:5]
# 3c （2-5hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
holiday_positive_time3c_am1 = holiday_positive_emo[10:25]
holiday_positive_time3c_am4 = holiday_positive_emo[55:70]
holiday_positive_time3c_pm1 = holiday_positive_emo[70:85]
holiday_positive_time3c_pm2 = holiday_positive_emo[85:100]
holiday_positive_time3c_pm3 = holiday_positive_emo[100:115]
holiday_positive_time3c_pm4 = holiday_positive_emo[115:120]+holiday_positive_emo[0:10]
# 4b （1-5hour，9-13hour，13-17hour，17-21hour，21-1hour）
holiday_positive_time4b_am1 = holiday_positive_emo[5:25]
holiday_positive_time4b_am3 = holiday_positive_emo[45:65]
holiday_positive_time4b_pm1 = holiday_positive_emo[65:85]
holiday_positive_time4b_pm2 = holiday_positive_emo[85:105]
holiday_positive_time4b_pm3 = holiday_positive_emo[105:120] + holiday_positive_emo[0:5]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
holiday_positive_time4c_am1 = holiday_positive_emo[10:30]
holiday_positive_time4c_am3 = holiday_positive_emo[50:70]
holiday_positive_time4c_pm1 = holiday_positive_emo[70:90]
holiday_positive_time4c_pm2 = holiday_positive_emo[90:110]
holiday_positive_time4c_pm3 = holiday_positive_emo[110:120] + holiday_positive_emo[0:10]
# 6a （0-6hour，12-18hour）
holiday_positive_time6a_am1 = holiday_positive_emo[0:30]
holiday_positive_time6a_pm1 = holiday_positive_emo[60:90]
# 6b （1-7hour，19-1hour）
holiday_positive_time6b_am1 = holiday_positive_emo[5:35]
holiday_positive_time6b_pm2 = holiday_positive_emo[95:120]+holiday_positive_emo[0:5]
# 6c （2-8hour，8-14hour，14-20hour，20-2hour）
holiday_positive_time6c_am1 = holiday_positive_emo[10:40]
holiday_positive_time6c_am2 = holiday_positive_emo[40:70]
holiday_positive_time6c_pm1 = holiday_positive_emo[70:100]
holiday_positive_time6c_pm2 = holiday_positive_emo[100:120]+holiday_positive_emo[0:10]
# 6d （3-9hour，9-15hour）
holiday_positive_time6d_am1 = holiday_positive_emo[15:45]
holiday_positive_time6d_am2 = holiday_positive_emo[45:75]
# 6e （4-10hour，10-16hour）
holiday_positive_time6e_am1 = holiday_positive_emo[20:50]
holiday_positive_time6e_am2 = holiday_positive_emo[50:80]


# （4）有效消极情绪区间（negative）
# 3a （3-6hour，9-12hour，12-15hour，15-18hour，18-21hour，21-24hour）
holiday_negative_time3a_am2 = holiday_negative_emo[15:30]
holiday_negative_time3a_am4 = holiday_negative_emo[45:60]
holiday_negative_time3a_pm1 = holiday_negative_emo[60:75]
holiday_negative_time3a_pm2 = holiday_negative_emo[75:90]
holiday_negative_time3a_pm3 = holiday_negative_emo[90:105]
holiday_negative_time3a_pm4 = holiday_negative_emo[105:120]
# 3b （1-4hour，7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
holiday_negative_time3b_am1 = holiday_negative_emo[5:20]
holiday_negative_time3b_am3 = holiday_negative_emo[35:50]
holiday_negative_time3b_am4 = holiday_negative_emo[50:65]
holiday_negative_time3b_pm1 = holiday_negative_emo[65:80]
holiday_negative_time3b_pm2 = holiday_negative_emo[80:95]
holiday_negative_time3b_pm3 = holiday_negative_emo[95:110]
holiday_negative_time3b_pm4 = holiday_negative_emo[110:120]+holiday_negative_emo[0:5]
# 3c （2-5hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
holiday_negative_time3c_am1 = holiday_negative_emo[10:25]
holiday_negative_time3c_am4 = holiday_negative_emo[55:70]
holiday_negative_time3c_pm1 = holiday_negative_emo[70:85]
holiday_negative_time3c_pm2 = holiday_negative_emo[85:100]
holiday_negative_time3c_pm3 = holiday_negative_emo[100:115]
holiday_negative_time3c_pm4 = holiday_negative_emo[115:120]+holiday_negative_emo[0:10]
# 4b （1-5hour，9-13hour，13-17hour，17-21hour，21-1hour）
holiday_negative_time4b_am1 = holiday_negative_emo[5:25]
holiday_negative_time4b_am3 = holiday_negative_emo[45:65]
holiday_negative_time4b_pm1 = holiday_negative_emo[65:85]
holiday_negative_time4b_pm2 = holiday_negative_emo[85:105]
holiday_negative_time4b_pm3 = holiday_negative_emo[105:120] + holiday_negative_emo[0:5]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
holiday_negative_time4c_am1 = holiday_negative_emo[10:30]
holiday_negative_time4c_am3 = holiday_negative_emo[50:70]
holiday_negative_time4c_pm1 = holiday_negative_emo[70:90]
holiday_negative_time4c_pm2 = holiday_negative_emo[90:110]
holiday_negative_time4c_pm3 = holiday_negative_emo[110:120] + holiday_negative_emo[0:10]
# 6a （0-6hour，12-18hour）
holiday_negative_time6a_am1 = holiday_negative_emo[0:30]
holiday_negative_time6a_pm1 = holiday_negative_emo[60:90]
# 6b （1-7hour，19-1hour）
holiday_negative_time6b_am1 = holiday_negative_emo[5:35]
holiday_negative_time6b_pm2 = holiday_negative_emo[95:120]+holiday_negative_emo[0:5]
# 6c （2-8hour，8-14hour，14-20hour，20-2hour）
holiday_negative_time6c_am1 = holiday_negative_emo[10:40]
holiday_negative_time6c_am2 = holiday_negative_emo[40:70]
holiday_negative_time6c_pm1 = holiday_negative_emo[70:100]
holiday_negative_time6c_pm2 = holiday_negative_emo[100:120]+holiday_negative_emo[0:10]
# 6d （3-9hour，9-15hour）
holiday_negative_time6d_am1 = holiday_negative_emo[15:45]
holiday_negative_time6d_am2 = holiday_negative_emo[45:75]
# 6e （4-10hour，10-16hour）
holiday_negative_time6e_am1 = holiday_negative_emo[20:50]
holiday_negative_time6e_am2 = holiday_negative_emo[50:80]

# (5) 有效anger情绪区间
# 3a （3-6hour，9-12hour，12-15hour，15-18hour，18-21hour，21-24hour）
holiday_anger_time3a_am2 = holiday_li_anger[15:30]
holiday_anger_time3a_am4 = holiday_li_anger[45:60]
holiday_anger_time3a_pm1 = holiday_li_anger[60:75]
holiday_anger_time3a_pm2 = holiday_li_anger[75:90]
holiday_anger_time3a_pm3 = holiday_li_anger[90:105]
holiday_anger_time3a_pm4 = holiday_li_anger[105:120]
# 3b （1-4hour，7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
holiday_anger_time3b_am1 = holiday_li_anger[5:20]
holiday_anger_time3b_am3 = holiday_li_anger[35:50]
holiday_anger_time3b_am4 = holiday_li_anger[50:65]
holiday_anger_time3b_pm1 = holiday_li_anger[65:80]
holiday_anger_time3b_pm2 = holiday_li_anger[80:95]
holiday_anger_time3b_pm3 = holiday_li_anger[95:110]
holiday_anger_time3b_pm4 = holiday_li_anger[110:120]+holiday_li_anger[0:5]
# 3c （2-5hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
holiday_anger_time3c_am1 = holiday_li_anger[10:25]
holiday_anger_time3c_am4 = holiday_li_anger[55:70]
holiday_anger_time3c_pm1 = holiday_li_anger[70:85]
holiday_anger_time3c_pm2 = holiday_li_anger[85:100]
holiday_anger_time3c_pm3 = holiday_li_anger[100:115]
holiday_anger_time3c_pm4 = holiday_li_anger[115:120]+holiday_li_anger[0:10]
# 4b （1-5hour，9-13hour，13-17hour，17-21hour，21-1hour）
holiday_anger_time4b_am1 = holiday_li_anger[5:25]
holiday_anger_time4b_am3 = holiday_li_anger[45:65]
holiday_anger_time4b_pm1 = holiday_li_anger[65:85]
holiday_anger_time4b_pm2 = holiday_li_anger[85:105]
holiday_anger_time4b_pm3 = holiday_li_anger[105:120] + holiday_li_anger[0:5]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
holiday_anger_time4c_am1 = holiday_li_anger[10:30]
holiday_anger_time4c_am3 = holiday_li_anger[50:70]
holiday_anger_time4c_pm1 = holiday_li_anger[70:90]
holiday_anger_time4c_pm2 = holiday_li_anger[90:110]
holiday_anger_time4c_pm3 = holiday_li_anger[110:120] + holiday_li_anger[0:10]
# 6a （0-6hour，12-18hour）
holiday_anger_time6a_am1 = holiday_li_anger[0:30]
holiday_anger_time6a_pm1 = holiday_li_anger[60:90]
# 6b （1-7hour，19-1hour）
holiday_anger_time6b_am1 = holiday_li_anger[5:35]
holiday_anger_time6b_pm2 = holiday_li_anger[95:120]+holiday_li_anger[0:5]
# 6c （2-8hour，8-14hour，14-20hour，20-2hour）
holiday_anger_time6c_am1 = holiday_li_anger[10:40]
holiday_anger_time6c_am2 = holiday_li_anger[40:70]
holiday_anger_time6c_pm1 = holiday_li_anger[70:100]
holiday_anger_time6c_pm2 = holiday_li_anger[100:120]+holiday_li_anger[0:10]
# 6d （3-9hour，9-15hour）
holiday_anger_time6d_am1 = holiday_li_anger[15:45]
holiday_anger_time6d_am2 = holiday_li_anger[45:75]
# 6e （4-10hour，10-16hour）
holiday_anger_time6e_am1 = holiday_li_anger[20:50]
holiday_anger_time6e_am2 = holiday_li_anger[50:80]

# (6) 有效anxi情绪区间
# 3a （3-6hour，9-12hour，12-15hour，15-18hour，18-21hour，21-24hour）
holiday_anxi_time3a_am2 = holiday_li_anxi[15:30]
holiday_anxi_time3a_am4 = holiday_li_anxi[45:60]
holiday_anxi_time3a_pm1 = holiday_li_anxi[60:75]
holiday_anxi_time3a_pm2 = holiday_li_anxi[75:90]
holiday_anxi_time3a_pm3 = holiday_li_anxi[90:105]
holiday_anxi_time3a_pm4 = holiday_li_anxi[105:120]
# 3b （1-4hour，7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
holiday_anxi_time3b_am1 = holiday_li_anxi[5:20]
holiday_anxi_time3b_am3 = holiday_li_anxi[35:50]
holiday_anxi_time3b_am4 = holiday_li_anxi[50:65]
holiday_anxi_time3b_pm1 = holiday_li_anxi[65:80]
holiday_anxi_time3b_pm2 = holiday_li_anxi[80:95]
holiday_anxi_time3b_pm3 = holiday_li_anxi[95:110]
holiday_anxi_time3b_pm4 = holiday_li_anxi[110:120]+holiday_li_anxi[0:5]
# 3c （2-5hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
holiday_anxi_time3c_am1 = holiday_li_anxi[10:25]
holiday_anxi_time3c_am4 = holiday_li_anxi[55:70]
holiday_anxi_time3c_pm1 = holiday_li_anxi[70:85]
holiday_anxi_time3c_pm2 = holiday_li_anxi[85:100]
holiday_anxi_time3c_pm3 = holiday_li_anxi[100:115]
holiday_anxi_time3c_pm4 = holiday_li_anxi[115:120]+holiday_li_anxi[0:10]
# 4b （1-5hour，9-13hour，13-17hour，17-21hour，21-1hour）
holiday_anxi_time4b_am1 = holiday_li_anxi[5:25]
holiday_anxi_time4b_am3 = holiday_li_anxi[45:65]
holiday_anxi_time4b_pm1 = holiday_li_anxi[65:85]
holiday_anxi_time4b_pm2 = holiday_li_anxi[85:105]
holiday_anxi_time4b_pm3 = holiday_li_anxi[105:120] + holiday_li_anxi[0:5]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
holiday_anxi_time4c_am1 = holiday_li_anxi[10:30]
holiday_anxi_time4c_am3 = holiday_li_anxi[50:70]
holiday_anxi_time4c_pm1 = holiday_li_anxi[70:90]
holiday_anxi_time4c_pm2 = holiday_li_anxi[90:110]
holiday_anxi_time4c_pm3 = holiday_li_anxi[110:120] + holiday_li_anxi[0:10]
# 6a （0-6hour，12-18hour）
holiday_anxi_time6a_am1 = holiday_li_anxi[0:30]
holiday_anxi_time6a_pm1 = holiday_li_anxi[60:90]
# 6b （1-7hour，19-1hour）
holiday_anxi_time6b_am1 = holiday_li_anxi[5:35]
holiday_anxi_time6b_pm2 = holiday_li_anxi[95:120]+holiday_li_anxi[0:5]
# 6c （2-8hour，8-14hour，14-20hour，20-2hour）
holiday_anxi_time6c_am1 = holiday_li_anxi[10:40]
holiday_anxi_time6c_am2 = holiday_li_anxi[40:70]
holiday_anxi_time6c_pm1 = holiday_li_anxi[70:100]
holiday_anxi_time6c_pm2 = holiday_li_anxi[100:120]+holiday_li_anxi[0:10]
# 6d （3-9hour，9-15hour）
holiday_anxi_time6d_am1 = holiday_li_anxi[15:45]
holiday_anxi_time6d_am2 = holiday_li_anxi[45:75]
# 6e （4-10hour，10-16hour）
holiday_anxi_time6e_am1 = holiday_li_anxi[20:50]
holiday_anxi_time6e_am2 = holiday_li_anxi[50:80]

# (7) 有效hate情绪区间
# 3a （3-6hour，9-12hour，12-15hour，15-18hour，18-21hour，21-24hour）
holiday_hate_time3a_am2 = holiday_li_hate[15:30]
holiday_hate_time3a_am4 = holiday_li_hate[45:60]
holiday_hate_time3a_pm1 = holiday_li_hate[60:75]
holiday_hate_time3a_pm2 = holiday_li_hate[75:90]
holiday_hate_time3a_pm3 = holiday_li_hate[90:105]
holiday_hate_time3a_pm4 = holiday_li_hate[105:120]
# 3b （1-4hour，7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
holiday_hate_time3b_am1 = holiday_li_hate[5:20]
holiday_hate_time3b_am3 = holiday_li_hate[35:50]
holiday_hate_time3b_am4 = holiday_li_hate[50:65]
holiday_hate_time3b_pm1 = holiday_li_hate[65:80]
holiday_hate_time3b_pm2 = holiday_li_hate[80:95]
holiday_hate_time3b_pm3 = holiday_li_hate[95:110]
holiday_hate_time3b_pm4 = holiday_li_hate[110:120]+holiday_li_hate[0:5]
# 3c （2-5hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
holiday_hate_time3c_am1 = holiday_li_hate[10:25]
holiday_hate_time3c_am4 = holiday_li_hate[55:70]
holiday_hate_time3c_pm1 = holiday_li_hate[70:85]
holiday_hate_time3c_pm2 = holiday_li_hate[85:100]
holiday_hate_time3c_pm3 = holiday_li_hate[100:115]
holiday_hate_time3c_pm4 = holiday_li_hate[115:120]+holiday_li_hate[0:10]
# 4b （1-5hour，9-13hour，13-17hour，17-21hour，21-1hour）
holiday_hate_time4b_am1 = holiday_li_hate[5:25]
holiday_hate_time4b_am3 = holiday_li_hate[45:65]
holiday_hate_time4b_pm1 = holiday_li_hate[65:85]
holiday_hate_time4b_pm2 = holiday_li_hate[85:105]
holiday_hate_time4b_pm3 = holiday_li_hate[105:120] + holiday_li_hate[0:5]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
holiday_hate_time4c_am1 = holiday_li_hate[10:30]
holiday_hate_time4c_am3 = holiday_li_hate[50:70]
holiday_hate_time4c_pm1 = holiday_li_hate[70:90]
holiday_hate_time4c_pm2 = holiday_li_hate[90:110]
holiday_hate_time4c_pm3 = holiday_li_hate[110:120] + holiday_li_hate[0:10]
# 6a （0-6hour，12-18hour）
holiday_hate_time6a_am1 = holiday_li_hate[0:30]
holiday_hate_time6a_pm1 = holiday_li_hate[60:90]
# 6b （1-7hour，19-1hour）
holiday_hate_time6b_am1 = holiday_li_hate[5:35]
holiday_hate_time6b_pm2 = holiday_li_hate[95:120]+holiday_li_hate[0:5]
# 6c （2-8hour，8-14hour，14-20hour，20-2hour）
holiday_hate_time6c_am1 = holiday_li_hate[10:40]
holiday_hate_time6c_am2 = holiday_li_hate[40:70]
holiday_hate_time6c_pm1 = holiday_li_hate[70:100]
holiday_hate_time6c_pm2 = holiday_li_hate[100:120]+holiday_li_hate[0:10]
# 6d （3-9hour，9-15hour）
holiday_hate_time6d_am1 = holiday_li_hate[15:45]
holiday_hate_time6d_am2 = holiday_li_hate[45:75]
# 6e （4-10hour，10-16hour）
holiday_hate_time6e_am1 = holiday_li_hate[20:50]
holiday_hate_time6e_am2 = holiday_li_hate[50:80]

# (8) 有效sad情绪区间
# 3a （3-6hour，9-12hour，12-15hour，15-18hour，18-21hour，21-24hour）
holiday_sad_time3a_am2 = holiday_li_sad[15:30]
holiday_sad_time3a_am4 = holiday_li_sad[45:60]
holiday_sad_time3a_pm1 = holiday_li_sad[60:75]
holiday_sad_time3a_pm2 = holiday_li_sad[75:90]
holiday_sad_time3a_pm3 = holiday_li_sad[90:105]
holiday_sad_time3a_pm4 = holiday_li_sad[105:120]
# 3b （1-4hour，7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
holiday_sad_time3b_am1 = holiday_li_sad[5:20]
holiday_sad_time3b_am3 = holiday_li_sad[35:50]
holiday_sad_time3b_am4 = holiday_li_sad[50:65]
holiday_sad_time3b_pm1 = holiday_li_sad[65:80]
holiday_sad_time3b_pm2 = holiday_li_sad[80:95]
holiday_sad_time3b_pm3 = holiday_li_sad[95:110]
holiday_sad_time3b_pm4 = holiday_li_sad[110:120]+holiday_li_sad[0:5]
# 3c （2-5hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
holiday_sad_time3c_am1 = holiday_li_sad[10:25]
holiday_sad_time3c_am4 = holiday_li_sad[55:70]
holiday_sad_time3c_pm1 = holiday_li_sad[70:85]
holiday_sad_time3c_pm2 = holiday_li_sad[85:100]
holiday_sad_time3c_pm3 = holiday_li_sad[100:115]
holiday_sad_time3c_pm4 = holiday_li_sad[115:120]+holiday_li_sad[0:10]
# 4b （1-5hour，9-13hour，13-17hour，17-21hour，21-1hour）
holiday_sad_time4b_am1 = holiday_li_sad[5:25]
holiday_sad_time4b_am3 = holiday_li_sad[45:65]
holiday_sad_time4b_pm1 = holiday_li_sad[65:85]
holiday_sad_time4b_pm2 = holiday_li_sad[85:105]
holiday_sad_time4b_pm3 = holiday_li_sad[105:120] + holiday_li_sad[0:5]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
holiday_sad_time4c_am1 = holiday_li_sad[10:30]
holiday_sad_time4c_am3 = holiday_li_sad[50:70]
holiday_sad_time4c_pm1 = holiday_li_sad[70:90]
holiday_sad_time4c_pm2 = holiday_li_sad[90:110]
holiday_sad_time4c_pm3 = holiday_li_sad[110:120] + holiday_li_sad[0:10]
# 6a （0-6hour，12-18hour）
holiday_sad_time6a_am1 = holiday_li_sad[0:30]
holiday_sad_time6a_pm1 = holiday_li_sad[60:90]
# 6b （1-7hour，19-1hour）
holiday_sad_time6b_am1 = holiday_li_sad[5:35]
holiday_sad_time6b_pm2 = holiday_li_sad[95:120]+holiday_li_sad[0:5]
# 6c （2-8hour，8-14hour，14-20hour，20-2hour）
holiday_sad_time6c_am1 = holiday_li_sad[10:40]
holiday_sad_time6c_am2 = holiday_li_sad[40:70]
holiday_sad_time6c_pm1 = holiday_li_sad[70:100]
holiday_sad_time6c_pm2 = holiday_li_sad[100:120]+holiday_li_sad[0:10]
# 6d （3-9hour，9-15hour）
holiday_sad_time6d_am1 = holiday_li_sad[15:45]
holiday_sad_time6d_am2 = holiday_li_sad[45:75]
# 6e （4-10hour，10-16hour）
holiday_sad_time6e_am1 = holiday_li_sad[20:50]
holiday_sad_time6e_am2 = holiday_li_sad[50:80]

# (9) 有效surprise情绪区间
# 3a （3-6hour，9-12hour，12-15hour，15-18hour，18-21hour，21-24hour）
holiday_surprise_time3a_am2 = holiday_li_surprise[15:30]
holiday_surprise_time3a_am4 = holiday_li_surprise[45:60]
holiday_surprise_time3a_pm1 = holiday_li_surprise[60:75]
holiday_surprise_time3a_pm2 = holiday_li_surprise[75:90]
holiday_surprise_time3a_pm3 = holiday_li_surprise[90:105]
holiday_surprise_time3a_pm4 = holiday_li_surprise[105:120]
# 3b （1-4hour，7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
holiday_surprise_time3b_am1 = holiday_li_surprise[5:20]
holiday_surprise_time3b_am3 = holiday_li_surprise[35:50]
holiday_surprise_time3b_am4 = holiday_li_surprise[50:65]
holiday_surprise_time3b_pm1 = holiday_li_surprise[65:80]
holiday_surprise_time3b_pm2 = holiday_li_surprise[80:95]
holiday_surprise_time3b_pm3 = holiday_li_surprise[95:110]
holiday_surprise_time3b_pm4 = holiday_li_surprise[110:120]+holiday_li_surprise[0:5]
# 3c （2-5hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
holiday_surprise_time3c_am1 = holiday_li_surprise[10:25]
holiday_surprise_time3c_am4 = holiday_li_surprise[55:70]
holiday_surprise_time3c_pm1 = holiday_li_surprise[70:85]
holiday_surprise_time3c_pm2 = holiday_li_surprise[85:100]
holiday_surprise_time3c_pm3 = holiday_li_surprise[100:115]
holiday_surprise_time3c_pm4 = holiday_li_surprise[115:120]+holiday_li_surprise[0:10]
# 4b （1-5hour，9-13hour，13-17hour，17-21hour，21-1hour）
holiday_surprise_time4b_am1 = holiday_li_surprise[5:25]
holiday_surprise_time4b_am3 = holiday_li_surprise[45:65]
holiday_surprise_time4b_pm1 = holiday_li_surprise[65:85]
holiday_surprise_time4b_pm2 = holiday_li_surprise[85:105]
holiday_surprise_time4b_pm3 = holiday_li_surprise[105:120] + holiday_li_surprise[0:5]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
holiday_surprise_time4c_am1 = holiday_li_surprise[10:30]
holiday_surprise_time4c_am3 = holiday_li_surprise[50:70]
holiday_surprise_time4c_pm1 = holiday_li_surprise[70:90]
holiday_surprise_time4c_pm2 = holiday_li_surprise[90:110]
holiday_surprise_time4c_pm3 = holiday_li_surprise[110:120] + holiday_li_surprise[0:10]
# 6a （0-6hour，12-18hour）
holiday_surprise_time6a_am1 = holiday_li_surprise[0:30]
holiday_surprise_time6a_pm1 = holiday_li_surprise[60:90]
# 6b （1-7hour，19-1hour）
holiday_surprise_time6b_am1 = holiday_li_surprise[5:35]
holiday_surprise_time6b_pm2 = holiday_li_surprise[95:120]+holiday_li_surprise[0:5]
# 6c （2-8hour，8-14hour，14-20hour，20-2hour）
holiday_surprise_time6c_am1 = holiday_li_surprise[10:40]
holiday_surprise_time6c_am2 = holiday_li_surprise[40:70]
holiday_surprise_time6c_pm1 = holiday_li_surprise[70:100]
holiday_surprise_time6c_pm2 = holiday_li_surprise[100:120]+holiday_li_surprise[0:10]
# 6d （3-9hour，9-15hour）
holiday_surprise_time6d_am1 = holiday_li_surprise[15:45]
holiday_surprise_time6d_am2 = holiday_li_surprise[45:75]
# 6e （4-10hour，10-16hour）
holiday_surprise_time6e_am1 = holiday_li_surprise[20:50]
holiday_surprise_time6e_am2 = holiday_li_surprise[50:80]

# (10) 有效love情绪区间
# 3a （3-6hour，9-12hour，12-15hour，15-18hour，18-21hour，21-24hour）
holiday_love_time3a_am2 = holiday_li_love[15:30]
holiday_love_time3a_am4 = holiday_li_love[45:60]
holiday_love_time3a_pm1 = holiday_li_love[60:75]
holiday_love_time3a_pm2 = holiday_li_love[75:90]
holiday_love_time3a_pm3 = holiday_li_love[90:105]
holiday_love_time3a_pm4 = holiday_li_love[105:120]
# 3b （1-4hour，7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
holiday_love_time3b_am1 = holiday_li_love[5:20]
holiday_love_time3b_am3 = holiday_li_love[35:50]
holiday_love_time3b_am4 = holiday_li_love[50:65]
holiday_love_time3b_pm1 = holiday_li_love[65:80]
holiday_love_time3b_pm2 = holiday_li_love[80:95]
holiday_love_time3b_pm3 = holiday_li_love[95:110]
holiday_love_time3b_pm4 = holiday_li_love[110:120]+holiday_li_love[0:5]
# 3c （2-5hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
holiday_love_time3c_am1 = holiday_li_love[10:25]
holiday_love_time3c_am4 = holiday_li_love[55:70]
holiday_love_time3c_pm1 = holiday_li_love[70:85]
holiday_love_time3c_pm2 = holiday_li_love[85:100]
holiday_love_time3c_pm3 = holiday_li_love[100:115]
holiday_love_time3c_pm4 = holiday_li_love[115:120]+holiday_li_love[0:10]
# 4b （1-5hour，9-13hour，13-17hour，17-21hour，21-1hour）
holiday_love_time4b_am1 = holiday_li_love[5:25]
holiday_love_time4b_am3 = holiday_li_love[45:65]
holiday_love_time4b_pm1 = holiday_li_love[65:85]
holiday_love_time4b_pm2 = holiday_li_love[85:105]
holiday_love_time4b_pm3 = holiday_li_love[105:120] + holiday_li_love[0:5]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
holiday_love_time4c_am1 = holiday_li_love[10:30]
holiday_love_time4c_am3 = holiday_li_love[50:70]
holiday_love_time4c_pm1 = holiday_li_love[70:90]
holiday_love_time4c_pm2 = holiday_li_love[90:110]
holiday_love_time4c_pm3 = holiday_li_love[110:120] + holiday_li_love[0:10]
# 6a （0-6hour，12-18hour）
holiday_love_time6a_am1 = holiday_li_love[0:30]
holiday_love_time6a_pm1 = holiday_li_love[60:90]
# 6b （1-7hour，19-1hour）
holiday_love_time6b_am1 = holiday_li_love[5:35]
holiday_love_time6b_pm2 = holiday_li_love[95:120]+holiday_li_love[0:5]
# 6c （2-8hour，8-14hour，14-20hour，20-2hour）
holiday_love_time6c_am1 = holiday_li_love[10:40]
holiday_love_time6c_am2 = holiday_li_love[40:70]
holiday_love_time6c_pm1 = holiday_li_love[70:100]
holiday_love_time6c_pm2 = holiday_li_love[100:120]+holiday_li_love[0:10]
# 6d （3-9hour，9-15hour）
holiday_love_time6d_am1 = holiday_li_love[15:45]
holiday_love_time6d_am2 = holiday_li_love[45:75]
# 6e （4-10hour，10-16hour）
holiday_love_time6e_am1 = holiday_li_love[20:50]
holiday_love_time6e_am2 = holiday_li_love[50:80]

# (11) 有效joy情绪区间
# 3a （3-6hour，9-12hour，12-15hour，15-18hour，18-21hour，21-24hour）
holiday_joy_time3a_am2 = holiday_li_joy[15:30]
holiday_joy_time3a_am4 = holiday_li_joy[45:60]
holiday_joy_time3a_pm1 = holiday_li_joy[60:75]
holiday_joy_time3a_pm2 = holiday_li_joy[75:90]
holiday_joy_time3a_pm3 = holiday_li_joy[90:105]
holiday_joy_time3a_pm4 = holiday_li_joy[105:120]
# 3b （1-4hour，7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
holiday_joy_time3b_am1 = holiday_li_joy[5:20]
holiday_joy_time3b_am3 = holiday_li_joy[35:50]
holiday_joy_time3b_am4 = holiday_li_joy[50:65]
holiday_joy_time3b_pm1 = holiday_li_joy[65:80]
holiday_joy_time3b_pm2 = holiday_li_joy[80:95]
holiday_joy_time3b_pm3 = holiday_li_joy[95:110]
holiday_joy_time3b_pm4 = holiday_li_joy[110:120]+holiday_li_joy[0:5]
# 3c （2-5hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
holiday_joy_time3c_am1 = holiday_li_joy[10:25]
holiday_joy_time3c_am4 = holiday_li_joy[55:70]
holiday_joy_time3c_pm1 = holiday_li_joy[70:85]
holiday_joy_time3c_pm2 = holiday_li_joy[85:100]
holiday_joy_time3c_pm3 = holiday_li_joy[100:115]
holiday_joy_time3c_pm4 = holiday_li_joy[115:120]+holiday_li_joy[0:10]
# 4b （1-5hour，9-13hour，13-17hour，17-21hour，21-1hour）
holiday_joy_time4b_am1 = holiday_li_joy[5:25]
holiday_joy_time4b_am3 = holiday_li_joy[45:65]
holiday_joy_time4b_pm1 = holiday_li_joy[65:85]
holiday_joy_time4b_pm2 = holiday_li_joy[85:105]
holiday_joy_time4b_pm3 = holiday_li_joy[105:120] + holiday_li_joy[0:5]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
holiday_joy_time4c_am1 = holiday_li_joy[10:30]
holiday_joy_time4c_am3 = holiday_li_joy[50:70]
holiday_joy_time4c_pm1 = holiday_li_joy[70:90]
holiday_joy_time4c_pm2 = holiday_li_joy[90:110]
holiday_joy_time4c_pm3 = holiday_li_joy[110:120] + holiday_li_joy[0:10]
# 6a （0-6hour，12-18hour）
holiday_joy_time6a_am1 = holiday_li_joy[0:30]
holiday_joy_time6a_pm1 = holiday_li_joy[60:90]
# 6b （1-7hour，19-1hour）
holiday_joy_time6b_am1 = holiday_li_joy[5:35]
holiday_joy_time6b_pm2 = holiday_li_joy[95:120]+holiday_li_joy[0:5]
# 6c （2-8hour，8-14hour，14-20hour，20-2hour）
holiday_joy_time6c_am1 = holiday_li_joy[10:40]
holiday_joy_time6c_am2 = holiday_li_joy[40:70]
holiday_joy_time6c_pm1 = holiday_li_joy[70:100]
holiday_joy_time6c_pm2 = holiday_li_joy[100:120]+holiday_li_joy[0:10]
# 6d （3-9hour，9-15hour）
holiday_joy_time6d_am1 = holiday_li_joy[15:45]
holiday_joy_time6d_am2 = holiday_li_joy[45:75]
# 6e （4-10hour，10-16hour）
holiday_joy_time6e_am1 = holiday_li_joy[20:50]
holiday_joy_time6e_am2 = holiday_li_joy[50:80]

# (12) 有效expectation情绪区间
# 3a （3-6hour，9-12hour，12-15hour，15-18hour，18-21hour，21-24hour）
holiday_expectation_time3a_am2 = holiday_li_expectation[15:30]
holiday_expectation_time3a_am4 = holiday_li_expectation[45:60]
holiday_expectation_time3a_pm1 = holiday_li_expectation[60:75]
holiday_expectation_time3a_pm2 = holiday_li_expectation[75:90]
holiday_expectation_time3a_pm3 = holiday_li_expectation[90:105]
holiday_expectation_time3a_pm4 = holiday_li_expectation[105:120]
# 3b （1-4hour，7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
holiday_expectation_time3b_am1 = holiday_li_expectation[5:20]
holiday_expectation_time3b_am3 = holiday_li_expectation[35:50]
holiday_expectation_time3b_am4 = holiday_li_expectation[50:65]
holiday_expectation_time3b_pm1 = holiday_li_expectation[65:80]
holiday_expectation_time3b_pm2 = holiday_li_expectation[80:95]
holiday_expectation_time3b_pm3 = holiday_li_expectation[95:110]
holiday_expectation_time3b_pm4 = holiday_li_expectation[110:120]+holiday_li_expectation[0:5]
# 3c （2-5hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
holiday_expectation_time3c_am1 = holiday_li_expectation[10:25]
holiday_expectation_time3c_am4 = holiday_li_expectation[55:70]
holiday_expectation_time3c_pm1 = holiday_li_expectation[70:85]
holiday_expectation_time3c_pm2 = holiday_li_expectation[85:100]
holiday_expectation_time3c_pm3 = holiday_li_expectation[100:115]
holiday_expectation_time3c_pm4 = holiday_li_expectation[115:120]+holiday_li_expectation[0:10]
# 4b （1-5hour，9-13hour，13-17hour，17-21hour，21-1hour）
holiday_expectation_time4b_am1 = holiday_li_expectation[5:25]
holiday_expectation_time4b_am3 = holiday_li_expectation[45:65]
holiday_expectation_time4b_pm1 = holiday_li_expectation[65:85]
holiday_expectation_time4b_pm2 = holiday_li_expectation[85:105]
holiday_expectation_time4b_pm3 = holiday_li_expectation[105:120] + holiday_li_expectation[0:5]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
holiday_expectation_time4c_am1 = holiday_li_expectation[10:30]
holiday_expectation_time4c_am3 = holiday_li_expectation[50:70]
holiday_expectation_time4c_pm1 = holiday_li_expectation[70:90]
holiday_expectation_time4c_pm2 = holiday_li_expectation[90:110]
holiday_expectation_time4c_pm3 = holiday_li_expectation[110:120] + holiday_li_expectation[0:10]
# 6a （0-6hour，12-18hour）
holiday_expectation_time6a_am1 = holiday_li_expectation[0:30]
holiday_expectation_time6a_pm1 = holiday_li_expectation[60:90]
# 6b （1-7hour，19-1hour）
holiday_expectation_time6b_am1 = holiday_li_expectation[5:35]
holiday_expectation_time6b_pm2 = holiday_li_expectation[95:120]+holiday_li_expectation[0:5]
# 6c （2-8hour，8-14hour，14-20hour，20-2hour）
holiday_expectation_time6c_am1 = holiday_li_expectation[10:40]
holiday_expectation_time6c_am2 = holiday_li_expectation[40:70]
holiday_expectation_time6c_pm1 = holiday_li_expectation[70:100]
holiday_expectation_time6c_pm2 = holiday_li_expectation[100:120]+holiday_li_expectation[0:10]
# 6d （3-9hour，9-15hour）
holiday_expectation_time6d_am1 = holiday_li_expectation[15:45]
holiday_expectation_time6d_am2 = holiday_li_expectation[45:75]
# 6e （4-10hour，10-16hour）
holiday_expectation_time6e_am1 = holiday_li_expectation[20:50]
holiday_expectation_time6e_am2 = holiday_li_expectation[50:80]


# 有效道德区间数据汇总
holiday_moral_group = [holiday_moral_time3a_am2+holiday_moral_time3a_am4+holiday_moral_time3a_pm1+
                       holiday_moral_time3a_pm2+holiday_moral_time3a_pm3+holiday_moral_time3a_pm4+
                       holiday_moral_time3b_am1+holiday_moral_time3b_am3+holiday_moral_time3b_am4+
                       holiday_moral_time3b_pm1+holiday_moral_time3b_pm2+holiday_moral_time3b_pm3+
                       holiday_moral_time3b_pm4+holiday_moral_time3c_am1+holiday_moral_time3c_am4+
                       holiday_moral_time3c_pm1+holiday_moral_time3c_pm2+holiday_moral_time3c_pm3+
                       holiday_moral_time3c_pm4+holiday_moral_time4b_am1+holiday_moral_time4b_am3+
                       holiday_moral_time4b_pm1+holiday_moral_time4b_pm2+holiday_moral_time4b_pm3+
                       holiday_moral_time4c_am1+holiday_moral_time4c_am3+holiday_moral_time4c_pm1+
                       holiday_moral_time4c_pm2+holiday_moral_time4c_pm3+holiday_moral_time6a_am1+
                       holiday_moral_time6a_pm1+holiday_moral_time6b_am1+holiday_moral_time6b_pm2+
                       holiday_moral_time6c_am1+holiday_moral_time6c_am2+holiday_moral_time6c_pm1+
                       holiday_moral_time6c_pm2+holiday_moral_time6d_am1+holiday_moral_time6d_am2+
                       holiday_moral_time6e_am1+holiday_moral_time6e_am2]

# 有效情绪区间数据汇总（总情绪）
holiday_emo_group = [holiday_emo_time3a_am2+holiday_emo_time3a_am4+holiday_emo_time3a_pm1+
                       holiday_emo_time3a_pm2+holiday_emo_time3a_pm3+holiday_emo_time3a_pm4+
                       holiday_emo_time3b_am1+holiday_emo_time3b_am3+holiday_emo_time3b_am4+
                       holiday_emo_time3b_pm1+holiday_emo_time3b_pm2+holiday_emo_time3b_pm3+
                       holiday_emo_time3b_pm4+holiday_emo_time3c_am1+holiday_emo_time3c_am4+
                       holiday_emo_time3c_pm1+holiday_emo_time3c_pm2+holiday_emo_time3c_pm3+
                       holiday_emo_time3c_pm4+holiday_emo_time4b_am1+holiday_emo_time4b_am3+
                       holiday_emo_time4b_pm1+holiday_emo_time4b_pm2+holiday_emo_time4b_pm3+
                       holiday_emo_time4c_am1+holiday_emo_time4c_am3+holiday_emo_time4c_pm1+
                       holiday_emo_time4c_pm2+holiday_emo_time4c_pm3+holiday_emo_time6a_am1+
                       holiday_emo_time6a_pm1+holiday_emo_time6b_am1+holiday_emo_time6b_pm2+
                       holiday_emo_time6c_am1+holiday_emo_time6c_am2+holiday_emo_time6c_pm1+
                       holiday_emo_time6c_pm2+holiday_emo_time6d_am1+holiday_emo_time6d_am2+
                       holiday_emo_time6e_am1+holiday_emo_time6e_am2]


# 有效情绪区间数据汇总（积极情绪positive）
holiday_positive_group = [holiday_positive_time3a_am2+holiday_positive_time3a_am4+holiday_positive_time3a_pm1+
                       holiday_positive_time3a_pm2+holiday_positive_time3a_pm3+holiday_positive_time3a_pm4+
                       holiday_positive_time3b_am1+holiday_positive_time3b_am3+holiday_positive_time3b_am4+
                       holiday_positive_time3b_pm1+holiday_positive_time3b_pm2+holiday_positive_time3b_pm3+
                       holiday_positive_time3b_pm4+holiday_positive_time3c_am1+holiday_positive_time3c_am4+
                       holiday_positive_time3c_pm1+holiday_positive_time3c_pm2+holiday_positive_time3c_pm3+
                       holiday_positive_time3c_pm4+holiday_positive_time4b_am1+holiday_positive_time4b_am3+
                       holiday_positive_time4b_pm1+holiday_positive_time4b_pm2+holiday_positive_time4b_pm3+
                       holiday_positive_time4c_am1+holiday_positive_time4c_am3+holiday_positive_time4c_pm1+
                       holiday_positive_time4c_pm2+holiday_positive_time4c_pm3+holiday_positive_time6a_am1+
                       holiday_positive_time6a_pm1+holiday_positive_time6b_am1+holiday_positive_time6b_pm2+
                       holiday_positive_time6c_am1+holiday_positive_time6c_am2+holiday_positive_time6c_pm1+
                       holiday_positive_time6c_pm2+holiday_positive_time6d_am1+holiday_positive_time6d_am2+
                       holiday_positive_time6e_am1+holiday_positive_time6e_am2]

# 有效情绪区间数据汇总（消极情绪negative）
holiday_negative_group = [holiday_negative_time3a_am2+holiday_negative_time3a_am4+holiday_negative_time3a_pm1+
                       holiday_negative_time3a_pm2+holiday_negative_time3a_pm3+holiday_negative_time3a_pm4+
                       holiday_negative_time3b_am1+holiday_negative_time3b_am3+holiday_negative_time3b_am4+
                       holiday_negative_time3b_pm1+holiday_negative_time3b_pm2+holiday_negative_time3b_pm3+
                       holiday_negative_time3b_pm4+holiday_negative_time3c_am1+holiday_negative_time3c_am4+
                       holiday_negative_time3c_pm1+holiday_negative_time3c_pm2+holiday_negative_time3c_pm3+
                       holiday_negative_time3c_pm4+holiday_negative_time4b_am1+holiday_negative_time4b_am3+
                       holiday_negative_time4b_pm1+holiday_negative_time4b_pm2+holiday_negative_time4b_pm3+
                       holiday_negative_time4c_am1+holiday_negative_time4c_am3+holiday_negative_time4c_pm1+
                       holiday_negative_time4c_pm2+holiday_negative_time4c_pm3+holiday_negative_time6a_am1+
                       holiday_negative_time6a_pm1+holiday_negative_time6b_am1+holiday_negative_time6b_pm2+
                       holiday_negative_time6c_am1+holiday_negative_time6c_am2+holiday_negative_time6c_pm1+
                       holiday_negative_time6c_pm2+holiday_negative_time6d_am1+holiday_negative_time6d_am2+
                       holiday_negative_time6e_am1+holiday_negative_time6e_am2]

# 有效anger情绪区间数据汇总
holiday_anger_group = [holiday_anger_time3a_am2+holiday_anger_time3a_am4+holiday_anger_time3a_pm1+
                       holiday_anger_time3a_pm2+holiday_anger_time3a_pm3+holiday_anger_time3a_pm4+
                       holiday_anger_time3b_am1+holiday_anger_time3b_am3+holiday_anger_time3b_am4+
                       holiday_anger_time3b_pm1+holiday_anger_time3b_pm2+holiday_anger_time3b_pm3+
                       holiday_anger_time3b_pm4+holiday_anger_time3c_am1+holiday_anger_time3c_am4+
                       holiday_anger_time3c_pm1+holiday_anger_time3c_pm2+holiday_anger_time3c_pm3+
                       holiday_anger_time3c_pm4+holiday_anger_time4b_am1+holiday_anger_time4b_am3+
                       holiday_anger_time4b_pm1+holiday_anger_time4b_pm2+holiday_anger_time4b_pm3+
                       holiday_anger_time4c_am1+holiday_anger_time4c_am3+holiday_anger_time4c_pm1+
                       holiday_anger_time4c_pm2+holiday_anger_time4c_pm3+holiday_anger_time6a_am1+
                       holiday_anger_time6a_pm1+holiday_anger_time6b_am1+holiday_anger_time6b_pm2+
                       holiday_anger_time6c_am1+holiday_anger_time6c_am2+holiday_anger_time6c_pm1+
                       holiday_anger_time6c_pm2+holiday_anger_time6d_am1+holiday_anger_time6d_am2+
                       holiday_anger_time6e_am1+holiday_anger_time6e_am2]

# 有效anxi情绪区间数据汇总
holiday_anxi_group = [holiday_anxi_time3a_am2+holiday_anxi_time3a_am4+holiday_anxi_time3a_pm1+
                       holiday_anxi_time3a_pm2+holiday_anxi_time3a_pm3+holiday_anxi_time3a_pm4+
                       holiday_anxi_time3b_am1+holiday_anxi_time3b_am3+holiday_anxi_time3b_am4+
                       holiday_anxi_time3b_pm1+holiday_anxi_time3b_pm2+holiday_anxi_time3b_pm3+
                       holiday_anxi_time3b_pm4+holiday_anxi_time3c_am1+holiday_anxi_time3c_am4+
                       holiday_anxi_time3c_pm1+holiday_anxi_time3c_pm2+holiday_anxi_time3c_pm3+
                       holiday_anxi_time3c_pm4+holiday_anxi_time4b_am1+holiday_anxi_time4b_am3+
                       holiday_anxi_time4b_pm1+holiday_anxi_time4b_pm2+holiday_anxi_time4b_pm3+
                       holiday_anxi_time4c_am1+holiday_anxi_time4c_am3+holiday_anxi_time4c_pm1+
                       holiday_anxi_time4c_pm2+holiday_anxi_time4c_pm3+holiday_anxi_time6a_am1+
                       holiday_anxi_time6a_pm1+holiday_anxi_time6b_am1+holiday_anxi_time6b_pm2+
                       holiday_anxi_time6c_am1+holiday_anxi_time6c_am2+holiday_anxi_time6c_pm1+
                       holiday_anxi_time6c_pm2+holiday_anxi_time6d_am1+holiday_anxi_time6d_am2+
                       holiday_anxi_time6e_am1+holiday_anxi_time6e_am2]

# 有效hate情绪区间数据汇总
holiday_hate_group = [holiday_hate_time3a_am2+holiday_hate_time3a_am4+holiday_hate_time3a_pm1+
                       holiday_hate_time3a_pm2+holiday_hate_time3a_pm3+holiday_hate_time3a_pm4+
                       holiday_hate_time3b_am1+holiday_hate_time3b_am3+holiday_hate_time3b_am4+
                       holiday_hate_time3b_pm1+holiday_hate_time3b_pm2+holiday_hate_time3b_pm3+
                       holiday_hate_time3b_pm4+holiday_hate_time3c_am1+holiday_hate_time3c_am4+
                       holiday_hate_time3c_pm1+holiday_hate_time3c_pm2+holiday_hate_time3c_pm3+
                       holiday_hate_time3c_pm4+holiday_hate_time4b_am1+holiday_hate_time4b_am3+
                       holiday_hate_time4b_pm1+holiday_hate_time4b_pm2+holiday_hate_time4b_pm3+
                       holiday_hate_time4c_am1+holiday_hate_time4c_am3+holiday_hate_time4c_pm1+
                       holiday_hate_time4c_pm2+holiday_hate_time4c_pm3+holiday_hate_time6a_am1+
                       holiday_hate_time6a_pm1+holiday_hate_time6b_am1+holiday_hate_time6b_pm2+
                       holiday_hate_time6c_am1+holiday_hate_time6c_am2+holiday_hate_time6c_pm1+
                       holiday_hate_time6c_pm2+holiday_hate_time6d_am1+holiday_hate_time6d_am2+
                       holiday_hate_time6e_am1+holiday_hate_time6e_am2]

# 有效sad情绪区间数据汇总
holiday_sad_group = [holiday_sad_time3a_am2+holiday_sad_time3a_am4+holiday_sad_time3a_pm1+
                       holiday_sad_time3a_pm2+holiday_sad_time3a_pm3+holiday_sad_time3a_pm4+
                       holiday_sad_time3b_am1+holiday_sad_time3b_am3+holiday_sad_time3b_am4+
                       holiday_sad_time3b_pm1+holiday_sad_time3b_pm2+holiday_sad_time3b_pm3+
                       holiday_sad_time3b_pm4+holiday_sad_time3c_am1+holiday_sad_time3c_am4+
                       holiday_sad_time3c_pm1+holiday_sad_time3c_pm2+holiday_sad_time3c_pm3+
                       holiday_sad_time3c_pm4+holiday_sad_time4b_am1+holiday_sad_time4b_am3+
                       holiday_sad_time4b_pm1+holiday_sad_time4b_pm2+holiday_sad_time4b_pm3+
                       holiday_sad_time4c_am1+holiday_sad_time4c_am3+holiday_sad_time4c_pm1+
                       holiday_sad_time4c_pm2+holiday_sad_time4c_pm3+holiday_sad_time6a_am1+
                       holiday_sad_time6a_pm1+holiday_sad_time6b_am1+holiday_sad_time6b_pm2+
                       holiday_sad_time6c_am1+holiday_sad_time6c_am2+holiday_sad_time6c_pm1+
                       holiday_sad_time6c_pm2+holiday_sad_time6d_am1+holiday_sad_time6d_am2+
                       holiday_sad_time6e_am1+holiday_sad_time6e_am2]

# 有效surprise情绪区间数据汇总
holiday_surprise_group = [holiday_surprise_time3a_am2+holiday_surprise_time3a_am4+holiday_surprise_time3a_pm1+
                       holiday_surprise_time3a_pm2+holiday_surprise_time3a_pm3+holiday_surprise_time3a_pm4+
                       holiday_surprise_time3b_am1+holiday_surprise_time3b_am3+holiday_surprise_time3b_am4+
                       holiday_surprise_time3b_pm1+holiday_surprise_time3b_pm2+holiday_surprise_time3b_pm3+
                       holiday_surprise_time3b_pm4+holiday_surprise_time3c_am1+holiday_surprise_time3c_am4+
                       holiday_surprise_time3c_pm1+holiday_surprise_time3c_pm2+holiday_surprise_time3c_pm3+
                       holiday_surprise_time3c_pm4+holiday_surprise_time4b_am1+holiday_surprise_time4b_am3+
                       holiday_surprise_time4b_pm1+holiday_surprise_time4b_pm2+holiday_surprise_time4b_pm3+
                       holiday_surprise_time4c_am1+holiday_surprise_time4c_am3+holiday_surprise_time4c_pm1+
                       holiday_surprise_time4c_pm2+holiday_surprise_time4c_pm3+holiday_surprise_time6a_am1+
                       holiday_surprise_time6a_pm1+holiday_surprise_time6b_am1+holiday_surprise_time6b_pm2+
                       holiday_surprise_time6c_am1+holiday_surprise_time6c_am2+holiday_surprise_time6c_pm1+
                       holiday_surprise_time6c_pm2+holiday_surprise_time6d_am1+holiday_surprise_time6d_am2+
                       holiday_surprise_time6e_am1+holiday_surprise_time6e_am2]

# 有效love情绪区间数据汇总
holiday_love_group = [holiday_love_time3a_am2+holiday_love_time3a_am4+holiday_love_time3a_pm1+
                       holiday_love_time3a_pm2+holiday_love_time3a_pm3+holiday_love_time3a_pm4+
                       holiday_love_time3b_am1+holiday_love_time3b_am3+holiday_love_time3b_am4+
                       holiday_love_time3b_pm1+holiday_love_time3b_pm2+holiday_love_time3b_pm3+
                       holiday_love_time3b_pm4+holiday_love_time3c_am1+holiday_love_time3c_am4+
                       holiday_love_time3c_pm1+holiday_love_time3c_pm2+holiday_love_time3c_pm3+
                       holiday_love_time3c_pm4+holiday_love_time4b_am1+holiday_love_time4b_am3+
                       holiday_love_time4b_pm1+holiday_love_time4b_pm2+holiday_love_time4b_pm3+
                       holiday_love_time4c_am1+holiday_love_time4c_am3+holiday_love_time4c_pm1+
                       holiday_love_time4c_pm2+holiday_love_time4c_pm3+holiday_love_time6a_am1+
                       holiday_love_time6a_pm1+holiday_love_time6b_am1+holiday_love_time6b_pm2+
                       holiday_love_time6c_am1+holiday_love_time6c_am2+holiday_love_time6c_pm1+
                       holiday_love_time6c_pm2+holiday_love_time6d_am1+holiday_love_time6d_am2+
                       holiday_love_time6e_am1+holiday_love_time6e_am2]

# 有效joy情绪区间数据汇总
holiday_joy_group = [holiday_joy_time3a_am2+holiday_joy_time3a_am4+holiday_joy_time3a_pm1+
                       holiday_joy_time3a_pm2+holiday_joy_time3a_pm3+holiday_joy_time3a_pm4+
                       holiday_joy_time3b_am1+holiday_joy_time3b_am3+holiday_joy_time3b_am4+
                       holiday_joy_time3b_pm1+holiday_joy_time3b_pm2+holiday_joy_time3b_pm3+
                       holiday_joy_time3b_pm4+holiday_joy_time3c_am1+holiday_joy_time3c_am4+
                       holiday_joy_time3c_pm1+holiday_joy_time3c_pm2+holiday_joy_time3c_pm3+
                       holiday_joy_time3c_pm4+holiday_joy_time4b_am1+holiday_joy_time4b_am3+
                       holiday_joy_time4b_pm1+holiday_joy_time4b_pm2+holiday_joy_time4b_pm3+
                       holiday_joy_time4c_am1+holiday_joy_time4c_am3+holiday_joy_time4c_pm1+
                       holiday_joy_time4c_pm2+holiday_joy_time4c_pm3+holiday_joy_time6a_am1+
                       holiday_joy_time6a_pm1+holiday_joy_time6b_am1+holiday_joy_time6b_pm2+
                       holiday_joy_time6c_am1+holiday_joy_time6c_am2+holiday_joy_time6c_pm1+
                       holiday_joy_time6c_pm2+holiday_joy_time6d_am1+holiday_joy_time6d_am2+
                       holiday_joy_time6e_am1+holiday_joy_time6e_am2]

# 有效expectation情绪区间数据汇总
holiday_expectation_group = [holiday_expectation_time3a_am2+holiday_expectation_time3a_am4+holiday_expectation_time3a_pm1+
                       holiday_expectation_time3a_pm2+holiday_expectation_time3a_pm3+holiday_expectation_time3a_pm4+
                       holiday_expectation_time3b_am1+holiday_expectation_time3b_am3+holiday_expectation_time3b_am4+
                       holiday_expectation_time3b_pm1+holiday_expectation_time3b_pm2+holiday_expectation_time3b_pm3+
                       holiday_expectation_time3b_pm4+holiday_expectation_time3c_am1+holiday_expectation_time3c_am4+
                       holiday_expectation_time3c_pm1+holiday_expectation_time3c_pm2+holiday_expectation_time3c_pm3+
                       holiday_expectation_time3c_pm4+holiday_expectation_time4b_am1+holiday_expectation_time4b_am3+
                       holiday_expectation_time4b_pm1+holiday_expectation_time4b_pm2+holiday_expectation_time4b_pm3+
                       holiday_expectation_time4c_am1+holiday_expectation_time4c_am3+holiday_expectation_time4c_pm1+
                       holiday_expectation_time4c_pm2+holiday_expectation_time4c_pm3+holiday_expectation_time6a_am1+
                       holiday_expectation_time6a_pm1+holiday_expectation_time6b_am1+holiday_expectation_time6b_pm2+
                       holiday_expectation_time6c_am1+holiday_expectation_time6c_am2+holiday_expectation_time6c_pm1+
                       holiday_expectation_time6c_pm2+holiday_expectation_time6d_am1+holiday_expectation_time6d_am2+
                       holiday_expectation_time6e_am1+holiday_expectation_time6e_am2]


# 有效时间标签构造（Morning=1,Afternoon=0,Night=-1）
# 注，需要在数据结构中改为category类型
holiday_time_group = [15*[-1]+15*[1]+15*[0]+15*[0]+15*[0]+15*[-1]+
                      15*[-1]+15*[1]+15*[1]+15*[0]+15*[0]+15*[0]+15*[-1]+
                      15*[-1]+15*[1]+15*[0]+15*[0]+15*[-1]+15*[-1]+
                      20*[-1]+20*[1]+20*[0]+20*[0]+20*[-1]+
                      20*[-1]+20*[1]+20*[0]+20*[0]+20*[-1]+
                      30*[-1]+30*[0]+
                      30*[-1]+30*[0]+
                      30*[-1]+30*[1]+30*[0]+30*[-1]+
                      30*[-1]+30*[1]+
                      30*[1]+30*[1]]

# 有效时区长度标签构造（三小时时区=3，四小时时区=4，六小时时区=6）
# 注，需要在数据结构中改为category类型
holiday_zone_group = [285*[3]+200*[4]+360*[6]]



In [15]:
# holiday中介效应测试结果
# 总体情绪——道德动机
holiday_emo_to_moral = test_media_process2(holiday_time_group[0],holiday_moral_group[0],
                                   holiday_emo_group[0],4,'Total_emotion(holiday)')
# 积极情绪——道德动机
holiday_positive_to_moral = test_media_process2(holiday_time_group[0],holiday_moral_group[0],
                                   holiday_positive_group[0],4,'Positive(holiday)')

# 消极情绪——道德动机
holiday_negative_to_moral = test_media_process2(holiday_time_group[0],holiday_moral_group[0],
                                   holiday_negative_group[0],4,'Negative(holiday)')

#*******************************************************************************
# anger——道德动机
holiday_anger_to_moral = test_media_process2(holiday_time_group[0],holiday_moral_group[0],
                                   holiday_anger_group[0],4,'Anger(holiday)')
# anxi——道德动机
holiday_anxi_to_moral = test_media_process2(holiday_time_group[0],holiday_moral_group[0],
                                   holiday_anxi_group[0],4,'Anxi(holiday)')
# hate——道德动机
holiday_hate_to_moral = test_media_process2(holiday_time_group[0],holiday_moral_group[0],
                                   holiday_hate_group[0],4,'Hate(holiday)')
# sad——道德动机
holiday_sad_to_moral = test_media_process2(holiday_time_group[0],holiday_moral_group[0],
                                   holiday_sad_group[0],4,'Sad(holiday)')
# surprise——道德动机
holiday_surprise_to_moral = test_media_process2(holiday_time_group[0],holiday_moral_group[0],
                                   holiday_surprise_group[0],4,'Surprise(holiday)')
#*******************************************************************************
# love——道德动机
holiday_love_to_moral = test_media_process2(holiday_time_group[0],holiday_moral_group[0],
                                   holiday_love_group[0],4,'Love(holiday)')
# joy——道德动机
holiday_joy_to_moral = test_media_process2(holiday_time_group[0],holiday_moral_group[0],
                                   holiday_joy_group[0],4,'Joy(holiday)')
# expectation——道德动机
holiday_expectation_to_moral = test_media_process2(holiday_time_group[0],holiday_moral_group[0],
                                   holiday_expectation_group[0],4,'Expectation(holiday)')


Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 4

Variables:
    Cons = Cons
    x = Time
    y = Moral
    m1 = Total_emotion(holiday)

Sample size:
845

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = Moral 
OLS Regression Summary

     R²  Adj. R²    MSE       F  df1  df2  p-value
 0.0388   0.0353 0.9392 16.9764    2  842   0.0000

Coefficients

                        coeff     se       t      p   LLCI   ULCI
Cons                   0.8261 0.0385 21.4388 0.0000 0.7506 0.9016
Time                   0.2139 0.0417  5.1261 0.0000 0.1321 0.2957
Total_emotion(holiday) 0.0140 0.0051  2.7130 0.0068 0.0039 0.0241

----------------------------------------------------------

Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 4

Variables:
    Cons = Cons
    x = Time
    y = Moral
    m1 = Hate(holiday)

Sample size:
845

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = Moral 
OLS Regression Summary

     R²  Adj. R²    MSE       F  df1  df2  p-value
 0.0400   0.0365 0.9380 17.5200    2  842   0.0000

Coefficients

               coeff     se       t      p   LLCI   ULCI
Cons          0.8534 0.0347 24.5963 0.0000 0.7854 0.9214
Time          0.2303 0.0420  5.4792 0.0000 0.1479 0.3126
Hate(holiday) 0.3330 0.1148  2.9008 0.0038 0.1080 0.5579

-------------------------------------------------------------------------

Outcome = Hate(holiday) 
OLS

Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 4

Variables:
    Cons = Cons
    x = Time
    y = Moral
    m1 = Expectation(holiday)

Sample size:
845

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = Moral 
OLS Regression Summary

     R²  Adj. R²    MSE       F  df1  df2  p-value
 0.0343   0.0308 0.9436 14.9457    2  842   0.0000

Coefficients

                      coeff     se       t      p    LLCI   ULCI
Cons                 0.8479 0.0372 22.8035 0.0000  0.7751 0.9208
Time                 0.2076 0.0420  4.9382 0.0000  0.1252 0.2899
Expectation(holiday) 0.1412 0.0763  1.8499 0.0647 -0.0084 0.2908

----------------------------------------------------------------

In [16]:
# 中介效应检测数据准备
# 五、weekend_holiday情境
# 有效道德区间
# （1）weekend_holiday_moral
# 3a （3-6hour，21-24hour）
weekend_holiday_moral_time3a_am2 = weekend_holiday_moral[21:42]
weekend_holiday_moral_time3a_pm4 = weekend_holiday_moral[147:168]
# 3b （1-4hour，7-10hour，10-13hour，19-22hour，22-1hour）
weekend_holiday_moral_time3b_am1 = weekend_holiday_moral[7:28]
weekend_holiday_moral_time3b_am3 = weekend_holiday_moral[49:70]
weekend_holiday_moral_time3b_am4 = weekend_holiday_moral[70:91]
weekend_holiday_moral_time3b_pm3 = weekend_holiday_moral[133:154]
weekend_holiday_moral_time3b_pm4 = weekend_holiday_moral[154:168]+weekend_holiday_moral[0:7]
# 3c （2-5hour，11-14hour，23-2hour）
weekend_holiday_moral_time3c_am1 = weekend_holiday_moral[14:35]
weekend_holiday_moral_time3c_am4 = weekend_holiday_moral[77:98]
weekend_holiday_moral_time3c_pm4 = weekend_holiday_moral[161:168]+weekend_holiday_moral[0:14]
# 4b （1-5hour，9-13hour，21-1hour）
weekend_holiday_moral_time4b_am1 = weekend_holiday_moral[7:35]
weekend_holiday_moral_time4b_am3 = weekend_holiday_moral[63:91]
weekend_holiday_moral_time4b_pm3 = weekend_holiday_moral[147:168] + weekend_holiday_moral[0:7]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
weekend_holiday_moral_time4c_am1 = weekend_holiday_moral[14:42]
weekend_holiday_moral_time4c_am3 = weekend_holiday_moral[70:98]
weekend_holiday_moral_time4c_pm1 = weekend_holiday_moral[98:126]
weekend_holiday_moral_time4c_pm2 = weekend_holiday_moral[126:154]
weekend_holiday_moral_time4c_pm3 = weekend_holiday_moral[154:168] + weekend_holiday_moral[0:14]
# 6b （1-7hour，19-1hour）
weekend_holiday_moral_time6b_am1 = weekend_holiday_moral[7:49]
weekend_holiday_moral_time6b_pm2 = weekend_holiday_moral[133:168]+weekend_holiday_moral[0:7]


# （2）有效情绪区间（总情绪）
# 3a （3-6hour，21-24hour）
weekend_holiday_emo_time3a_am2 = weekend_holiday_total_emo[21:42]
weekend_holiday_emo_time3a_pm4 = weekend_holiday_total_emo[147:168]
# 3b （1-4hour，7-10hour，10-13hour，19-22hour，22-1hour）
weekend_holiday_emo_time3b_am1 = weekend_holiday_total_emo[7:28]
weekend_holiday_emo_time3b_am3 = weekend_holiday_total_emo[49:70]
weekend_holiday_emo_time3b_am4 = weekend_holiday_total_emo[70:91]
weekend_holiday_emo_time3b_pm3 = weekend_holiday_total_emo[133:154]
weekend_holiday_emo_time3b_pm4 = weekend_holiday_total_emo[154:168]+weekend_holiday_total_emo[0:7]
# 3c （2-5hour，11-14hour，23-2hour）
weekend_holiday_emo_time3c_am1 = weekend_holiday_total_emo[14:35]
weekend_holiday_emo_time3c_am4 = weekend_holiday_total_emo[77:98]
weekend_holiday_emo_time3c_pm4 = weekend_holiday_total_emo[161:168]+weekend_holiday_total_emo[0:14]
# 4b （1-5hour，9-13hour，21-1hour）
weekend_holiday_emo_time4b_am1 = weekend_holiday_total_emo[7:35]
weekend_holiday_emo_time4b_am3 = weekend_holiday_total_emo[63:91]
weekend_holiday_emo_time4b_pm3 = weekend_holiday_total_emo[147:168] + weekend_holiday_total_emo[0:7]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
weekend_holiday_emo_time4c_am1 = weekend_holiday_total_emo[14:42]
weekend_holiday_emo_time4c_am3 = weekend_holiday_total_emo[70:98]
weekend_holiday_emo_time4c_pm1 = weekend_holiday_total_emo[98:126]
weekend_holiday_emo_time4c_pm2 = weekend_holiday_total_emo[126:154]
weekend_holiday_emo_time4c_pm3 = weekend_holiday_total_emo[154:168] + weekend_holiday_total_emo[0:14]
# 6b （1-7hour，19-1hour）
weekend_holiday_emo_time6b_am1 = weekend_holiday_total_emo[7:49]
weekend_holiday_emo_time6b_pm2 = weekend_holiday_total_emo[133:168]+weekend_holiday_total_emo[0:7]


# （3）有效积极情绪区间（positive）
# 3a （3-6hour，21-24hour）
weekend_holiday_positive_time3a_am2 = weekend_holiday_positive_emo[21:42]
weekend_holiday_positive_time3a_pm4 = weekend_holiday_positive_emo[147:168]
# 3b （1-4hour，7-10hour，10-13hour，19-22hour，22-1hour）
weekend_holiday_positive_time3b_am1 = weekend_holiday_positive_emo[7:28]
weekend_holiday_positive_time3b_am3 = weekend_holiday_positive_emo[49:70]
weekend_holiday_positive_time3b_am4 = weekend_holiday_positive_emo[70:91]
weekend_holiday_positive_time3b_pm3 = weekend_holiday_positive_emo[133:154]
weekend_holiday_positive_time3b_pm4 = weekend_holiday_positive_emo[154:168]+weekend_holiday_positive_emo[0:7]
# 3c （2-5hour，11-14hour，23-2hour）
weekend_holiday_positive_time3c_am1 = weekend_holiday_positive_emo[14:35]
weekend_holiday_positive_time3c_am4 = weekend_holiday_positive_emo[77:98]
weekend_holiday_positive_time3c_pm4 = weekend_holiday_positive_emo[161:168]+weekend_holiday_positive_emo[0:14]
# 4b （1-5hour，9-13hour，21-1hour）
weekend_holiday_positive_time4b_am1 = weekend_holiday_positive_emo[7:35]
weekend_holiday_positive_time4b_am3 = weekend_holiday_positive_emo[63:91]
weekend_holiday_positive_time4b_pm3 = weekend_holiday_positive_emo[147:168] + weekend_holiday_positive_emo[0:7]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
weekend_holiday_positive_time4c_am1 = weekend_holiday_positive_emo[14:42]
weekend_holiday_positive_time4c_am3 = weekend_holiday_positive_emo[70:98]
weekend_holiday_positive_time4c_pm1 = weekend_holiday_positive_emo[98:126]
weekend_holiday_positive_time4c_pm2 = weekend_holiday_positive_emo[126:154]
weekend_holiday_positive_time4c_pm3 = weekend_holiday_positive_emo[154:168] + weekend_holiday_positive_emo[0:14]
# 6b （1-7hour，19-1hour）
weekend_holiday_positive_time6b_am1 = weekend_holiday_positive_emo[7:49]
weekend_holiday_positive_time6b_pm2 = weekend_holiday_positive_emo[133:168]+weekend_holiday_positive_emo[0:7]

# （4）有效消极情绪区间（negative）
# 3a （3-6hour，21-24hour）
weekend_holiday_negative_time3a_am2 = weekend_holiday_negative_emo[21:42]
weekend_holiday_negative_time3a_pm4 = weekend_holiday_negative_emo[147:168]
# 3b （1-4hour，7-10hour，10-13hour，19-22hour，22-1hour）
weekend_holiday_negative_time3b_am1 = weekend_holiday_negative_emo[7:28]
weekend_holiday_negative_time3b_am3 = weekend_holiday_negative_emo[49:70]
weekend_holiday_negative_time3b_am4 = weekend_holiday_negative_emo[70:91]
weekend_holiday_negative_time3b_pm3 = weekend_holiday_negative_emo[133:154]
weekend_holiday_negative_time3b_pm4 = weekend_holiday_negative_emo[154:168]+weekend_holiday_negative_emo[0:7]
# 3c （2-5hour，11-14hour，23-2hour）
weekend_holiday_negative_time3c_am1 = weekend_holiday_negative_emo[14:35]
weekend_holiday_negative_time3c_am4 = weekend_holiday_negative_emo[77:98]
weekend_holiday_negative_time3c_pm4 = weekend_holiday_negative_emo[161:168]+weekend_holiday_negative_emo[0:14]
# 4b （1-5hour，9-13hour，21-1hour）
weekend_holiday_negative_time4b_am1 = weekend_holiday_negative_emo[7:35]
weekend_holiday_negative_time4b_am3 = weekend_holiday_negative_emo[63:91]
weekend_holiday_negative_time4b_pm3 = weekend_holiday_negative_emo[147:168] + weekend_holiday_negative_emo[0:7]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
weekend_holiday_negative_time4c_am1 = weekend_holiday_negative_emo[14:42]
weekend_holiday_negative_time4c_am3 = weekend_holiday_negative_emo[70:98]
weekend_holiday_negative_time4c_pm1 = weekend_holiday_negative_emo[98:126]
weekend_holiday_negative_time4c_pm2 = weekend_holiday_negative_emo[126:154]
weekend_holiday_negative_time4c_pm3 = weekend_holiday_negative_emo[154:168] + weekend_holiday_negative_emo[0:14]
# 6b （1-7hour，19-1hour）
weekend_holiday_negative_time6b_am1 = weekend_holiday_negative_emo[7:49]
weekend_holiday_negative_time6b_pm2 = weekend_holiday_negative_emo[133:168]+weekend_holiday_negative_emo[0:7]

# (5) 有效anger情绪区间
# 3a （3-6hour，21-24hour）
weekend_holiday_anger_time3a_am2 = weekend_holiday_li_anger[21:42]
weekend_holiday_anger_time3a_pm4 = weekend_holiday_li_anger[147:168]
# 3b （1-4hour，7-10hour，10-13hour，19-22hour，22-1hour）
weekend_holiday_anger_time3b_am1 = weekend_holiday_li_anger[7:28]
weekend_holiday_anger_time3b_am3 = weekend_holiday_li_anger[49:70]
weekend_holiday_anger_time3b_am4 = weekend_holiday_li_anger[70:91]
weekend_holiday_anger_time3b_pm3 = weekend_holiday_li_anger[133:154]
weekend_holiday_anger_time3b_pm4 = weekend_holiday_li_anger[154:168]+weekend_holiday_li_anger[0:7]
# 3c （2-5hour，11-14hour，23-2hour）
weekend_holiday_anger_time3c_am1 = weekend_holiday_li_anger[14:35]
weekend_holiday_anger_time3c_am4 = weekend_holiday_li_anger[77:98]
weekend_holiday_anger_time3c_pm4 = weekend_holiday_li_anger[161:168]+weekend_holiday_li_anger[0:14]
# 4b （1-5hour，9-13hour，21-1hour）
weekend_holiday_anger_time4b_am1 = weekend_holiday_li_anger[7:35]
weekend_holiday_anger_time4b_am3 = weekend_holiday_li_anger[63:91]
weekend_holiday_anger_time4b_pm3 = weekend_holiday_li_anger[147:168] + weekend_holiday_li_anger[0:7]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
weekend_holiday_anger_time4c_am1 = weekend_holiday_li_anger[14:42]
weekend_holiday_anger_time4c_am3 = weekend_holiday_li_anger[70:98]
weekend_holiday_anger_time4c_pm1 = weekend_holiday_li_anger[98:126]
weekend_holiday_anger_time4c_pm2 = weekend_holiday_li_anger[126:154]
weekend_holiday_anger_time4c_pm3 = weekend_holiday_li_anger[154:168] + weekend_holiday_li_anger[0:14]
# 6b （1-7hour，19-1hour）
weekend_holiday_anger_time6b_am1 = weekend_holiday_li_anger[7:49]
weekend_holiday_anger_time6b_pm2 = weekend_holiday_li_anger[133:168]+weekend_holiday_li_anger[0:7]

# (6) 有效anxi情绪区间
# 3a （3-6hour，21-24hour）
weekend_holiday_anxi_time3a_am2 = weekend_holiday_li_anxi[21:42]
weekend_holiday_anxi_time3a_pm4 = weekend_holiday_li_anxi[147:168]
# 3b （1-4hour，7-10hour，10-13hour，19-22hour，22-1hour）
weekend_holiday_anxi_time3b_am1 = weekend_holiday_li_anxi[7:28]
weekend_holiday_anxi_time3b_am3 = weekend_holiday_li_anxi[49:70]
weekend_holiday_anxi_time3b_am4 = weekend_holiday_li_anxi[70:91]
weekend_holiday_anxi_time3b_pm3 = weekend_holiday_li_anxi[133:154]
weekend_holiday_anxi_time3b_pm4 = weekend_holiday_li_anxi[154:168]+weekend_holiday_li_anxi[0:7]
# 3c （2-5hour，11-14hour，23-2hour）
weekend_holiday_anxi_time3c_am1 = weekend_holiday_li_anxi[14:35]
weekend_holiday_anxi_time3c_am4 = weekend_holiday_li_anxi[77:98]
weekend_holiday_anxi_time3c_pm4 = weekend_holiday_li_anxi[161:168]+weekend_holiday_li_anxi[0:14]
# 4b （1-5hour，9-13hour，21-1hour）
weekend_holiday_anxi_time4b_am1 = weekend_holiday_li_anxi[7:35]
weekend_holiday_anxi_time4b_am3 = weekend_holiday_li_anxi[63:91]
weekend_holiday_anxi_time4b_pm3 = weekend_holiday_li_anxi[147:168] + weekend_holiday_li_anxi[0:7]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
weekend_holiday_anxi_time4c_am1 = weekend_holiday_li_anxi[14:42]
weekend_holiday_anxi_time4c_am3 = weekend_holiday_li_anxi[70:98]
weekend_holiday_anxi_time4c_pm1 = weekend_holiday_li_anxi[98:126]
weekend_holiday_anxi_time4c_pm2 = weekend_holiday_li_anxi[126:154]
weekend_holiday_anxi_time4c_pm3 = weekend_holiday_li_anxi[154:168] + weekend_holiday_li_anxi[0:14]
# 6b （1-7hour，19-1hour）
weekend_holiday_anxi_time6b_am1 = weekend_holiday_li_anxi[7:49]
weekend_holiday_anxi_time6b_pm2 = weekend_holiday_li_anxi[133:168]+weekend_holiday_li_anxi[0:7]

# (7) 有效hate情绪区间
# 3a （3-6hour，21-24hour）
weekend_holiday_hate_time3a_am2 = weekend_holiday_li_hate[21:42]
weekend_holiday_hate_time3a_pm4 = weekend_holiday_li_hate[147:168]
# 3b （1-4hour，7-10hour，10-13hour，19-22hour，22-1hour）
weekend_holiday_hate_time3b_am1 = weekend_holiday_li_hate[7:28]
weekend_holiday_hate_time3b_am3 = weekend_holiday_li_hate[49:70]
weekend_holiday_hate_time3b_am4 = weekend_holiday_li_hate[70:91]
weekend_holiday_hate_time3b_pm3 = weekend_holiday_li_hate[133:154]
weekend_holiday_hate_time3b_pm4 = weekend_holiday_li_hate[154:168]+weekend_holiday_li_hate[0:7]
# 3c （2-5hour，11-14hour，23-2hour）
weekend_holiday_hate_time3c_am1 = weekend_holiday_li_hate[14:35]
weekend_holiday_hate_time3c_am4 = weekend_holiday_li_hate[77:98]
weekend_holiday_hate_time3c_pm4 = weekend_holiday_li_hate[161:168]+weekend_holiday_li_hate[0:14]
# 4b （1-5hour，9-13hour，21-1hour）
weekend_holiday_hate_time4b_am1 = weekend_holiday_li_hate[7:35]
weekend_holiday_hate_time4b_am3 = weekend_holiday_li_hate[63:91]
weekend_holiday_hate_time4b_pm3 = weekend_holiday_li_hate[147:168] + weekend_holiday_li_hate[0:7]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
weekend_holiday_hate_time4c_am1 = weekend_holiday_li_hate[14:42]
weekend_holiday_hate_time4c_am3 = weekend_holiday_li_hate[70:98]
weekend_holiday_hate_time4c_pm1 = weekend_holiday_li_hate[98:126]
weekend_holiday_hate_time4c_pm2 = weekend_holiday_li_hate[126:154]
weekend_holiday_hate_time4c_pm3 = weekend_holiday_li_hate[154:168] + weekend_holiday_li_hate[0:14]
# 6b （1-7hour，19-1hour）
weekend_holiday_hate_time6b_am1 = weekend_holiday_li_hate[7:49]
weekend_holiday_hate_time6b_pm2 = weekend_holiday_li_hate[133:168]+weekend_holiday_li_hate[0:7]

# (8) 有效sad情绪区间
# 3a （3-6hour，21-24hour）
weekend_holiday_sad_time3a_am2 = weekend_holiday_li_sad[21:42]
weekend_holiday_sad_time3a_pm4 = weekend_holiday_li_sad[147:168]
# 3b （1-4hour，7-10hour，10-13hour，19-22hour，22-1hour）
weekend_holiday_sad_time3b_am1 = weekend_holiday_li_sad[7:28]
weekend_holiday_sad_time3b_am3 = weekend_holiday_li_sad[49:70]
weekend_holiday_sad_time3b_am4 = weekend_holiday_li_sad[70:91]
weekend_holiday_sad_time3b_pm3 = weekend_holiday_li_sad[133:154]
weekend_holiday_sad_time3b_pm4 = weekend_holiday_li_sad[154:168]+weekend_holiday_li_sad[0:7]
# 3c （2-5hour，11-14hour，23-2hour）
weekend_holiday_sad_time3c_am1 = weekend_holiday_li_sad[14:35]
weekend_holiday_sad_time3c_am4 = weekend_holiday_li_sad[77:98]
weekend_holiday_sad_time3c_pm4 = weekend_holiday_li_sad[161:168]+weekend_holiday_li_sad[0:14]
# 4b （1-5hour，9-13hour，21-1hour）
weekend_holiday_sad_time4b_am1 = weekend_holiday_li_sad[7:35]
weekend_holiday_sad_time4b_am3 = weekend_holiday_li_sad[63:91]
weekend_holiday_sad_time4b_pm3 = weekend_holiday_li_sad[147:168] + weekend_holiday_li_sad[0:7]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
weekend_holiday_sad_time4c_am1 = weekend_holiday_li_sad[14:42]
weekend_holiday_sad_time4c_am3 = weekend_holiday_li_sad[70:98]
weekend_holiday_sad_time4c_pm1 = weekend_holiday_li_sad[98:126]
weekend_holiday_sad_time4c_pm2 = weekend_holiday_li_sad[126:154]
weekend_holiday_sad_time4c_pm3 = weekend_holiday_li_sad[154:168] + weekend_holiday_li_sad[0:14]
# 6b （1-7hour，19-1hour）
weekend_holiday_sad_time6b_am1 = weekend_holiday_li_sad[7:49]
weekend_holiday_sad_time6b_pm2 = weekend_holiday_li_sad[133:168]+weekend_holiday_li_sad[0:7]

# (9) 有效surprise情绪区间
# 3a （3-6hour，21-24hour）
weekend_holiday_surprise_time3a_am2 = weekend_holiday_li_surprise[21:42]
weekend_holiday_surprise_time3a_pm4 = weekend_holiday_li_surprise[147:168]
# 3b （1-4hour，7-10hour，10-13hour，19-22hour，22-1hour）
weekend_holiday_surprise_time3b_am1 = weekend_holiday_li_surprise[7:28]
weekend_holiday_surprise_time3b_am3 = weekend_holiday_li_surprise[49:70]
weekend_holiday_surprise_time3b_am4 = weekend_holiday_li_surprise[70:91]
weekend_holiday_surprise_time3b_pm3 = weekend_holiday_li_surprise[133:154]
weekend_holiday_surprise_time3b_pm4 = weekend_holiday_li_surprise[154:168]+weekend_holiday_li_surprise[0:7]
# 3c （2-5hour，11-14hour，23-2hour）
weekend_holiday_surprise_time3c_am1 = weekend_holiday_li_surprise[14:35]
weekend_holiday_surprise_time3c_am4 = weekend_holiday_li_surprise[77:98]
weekend_holiday_surprise_time3c_pm4 = weekend_holiday_li_surprise[161:168]+weekend_holiday_li_surprise[0:14]
# 4b （1-5hour，9-13hour，21-1hour）
weekend_holiday_surprise_time4b_am1 = weekend_holiday_li_surprise[7:35]
weekend_holiday_surprise_time4b_am3 = weekend_holiday_li_surprise[63:91]
weekend_holiday_surprise_time4b_pm3 = weekend_holiday_li_surprise[147:168] + weekend_holiday_li_surprise[0:7]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
weekend_holiday_surprise_time4c_am1 = weekend_holiday_li_surprise[14:42]
weekend_holiday_surprise_time4c_am3 = weekend_holiday_li_surprise[70:98]
weekend_holiday_surprise_time4c_pm1 = weekend_holiday_li_surprise[98:126]
weekend_holiday_surprise_time4c_pm2 = weekend_holiday_li_surprise[126:154]
weekend_holiday_surprise_time4c_pm3 = weekend_holiday_li_surprise[154:168] + weekend_holiday_li_surprise[0:14]
# 6b （1-7hour，19-1hour）
weekend_holiday_surprise_time6b_am1 = weekend_holiday_li_surprise[7:49]
weekend_holiday_surprise_time6b_pm2 = weekend_holiday_li_surprise[133:168]+weekend_holiday_li_surprise[0:7]

# (10) 有效love情绪区间
# 3a （3-6hour，21-24hour）
weekend_holiday_love_time3a_am2 = weekend_holiday_li_love[21:42]
weekend_holiday_love_time3a_pm4 = weekend_holiday_li_love[147:168]
# 3b （1-4hour，7-10hour，10-13hour，19-22hour，22-1hour）
weekend_holiday_love_time3b_am1 = weekend_holiday_li_love[7:28]
weekend_holiday_love_time3b_am3 = weekend_holiday_li_love[49:70]
weekend_holiday_love_time3b_am4 = weekend_holiday_li_love[70:91]
weekend_holiday_love_time3b_pm3 = weekend_holiday_li_love[133:154]
weekend_holiday_love_time3b_pm4 = weekend_holiday_li_love[154:168]+weekend_holiday_li_love[0:7]
# 3c （2-5hour，11-14hour，23-2hour）
weekend_holiday_love_time3c_am1 = weekend_holiday_li_love[14:35]
weekend_holiday_love_time3c_am4 = weekend_holiday_li_love[77:98]
weekend_holiday_love_time3c_pm4 = weekend_holiday_li_love[161:168]+weekend_holiday_li_love[0:14]
# 4b （1-5hour，9-13hour，21-1hour）
weekend_holiday_love_time4b_am1 = weekend_holiday_li_love[7:35]
weekend_holiday_love_time4b_am3 = weekend_holiday_li_love[63:91]
weekend_holiday_love_time4b_pm3 = weekend_holiday_li_love[147:168] + weekend_holiday_li_love[0:7]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
weekend_holiday_love_time4c_am1 = weekend_holiday_li_love[14:42]
weekend_holiday_love_time4c_am3 = weekend_holiday_li_love[70:98]
weekend_holiday_love_time4c_pm1 = weekend_holiday_li_love[98:126]
weekend_holiday_love_time4c_pm2 = weekend_holiday_li_love[126:154]
weekend_holiday_love_time4c_pm3 = weekend_holiday_li_love[154:168] + weekend_holiday_li_love[0:14]
# 6b （1-7hour，19-1hour）
weekend_holiday_love_time6b_am1 = weekend_holiday_li_love[7:49]
weekend_holiday_love_time6b_pm2 = weekend_holiday_li_love[133:168]+weekend_holiday_li_love[0:7]

# (11) 有效joy情绪区间
# 3a （3-6hour，21-24hour）
weekend_holiday_joy_time3a_am2 = weekend_holiday_li_joy[21:42]
weekend_holiday_joy_time3a_pm4 = weekend_holiday_li_joy[147:168]
# 3b （1-4hour，7-10hour，10-13hour，19-22hour，22-1hour）
weekend_holiday_joy_time3b_am1 = weekend_holiday_li_joy[7:28]
weekend_holiday_joy_time3b_am3 = weekend_holiday_li_joy[49:70]
weekend_holiday_joy_time3b_am4 = weekend_holiday_li_joy[70:91]
weekend_holiday_joy_time3b_pm3 = weekend_holiday_li_joy[133:154]
weekend_holiday_joy_time3b_pm4 = weekend_holiday_li_joy[154:168]+weekend_holiday_li_joy[0:7]
# 3c （2-5hour，11-14hour，23-2hour）
weekend_holiday_joy_time3c_am1 = weekend_holiday_li_joy[14:35]
weekend_holiday_joy_time3c_am4 = weekend_holiday_li_joy[77:98]
weekend_holiday_joy_time3c_pm4 = weekend_holiday_li_joy[161:168]+weekend_holiday_li_joy[0:14]
# 4b （1-5hour，9-13hour，21-1hour）
weekend_holiday_joy_time4b_am1 = weekend_holiday_li_joy[7:35]
weekend_holiday_joy_time4b_am3 = weekend_holiday_li_joy[63:91]
weekend_holiday_joy_time4b_pm3 = weekend_holiday_li_joy[147:168] + weekend_holiday_li_joy[0:7]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
weekend_holiday_joy_time4c_am1 = weekend_holiday_li_joy[14:42]
weekend_holiday_joy_time4c_am3 = weekend_holiday_li_joy[70:98]
weekend_holiday_joy_time4c_pm1 = weekend_holiday_li_joy[98:126]
weekend_holiday_joy_time4c_pm2 = weekend_holiday_li_joy[126:154]
weekend_holiday_joy_time4c_pm3 = weekend_holiday_li_joy[154:168] + weekend_holiday_li_joy[0:14]
# 6b （1-7hour，19-1hour）
weekend_holiday_joy_time6b_am1 = weekend_holiday_li_joy[7:49]
weekend_holiday_joy_time6b_pm2 = weekend_holiday_li_joy[133:168]+weekend_holiday_li_joy[0:7]

# (12) 有效expectation情绪区间
# 3a （3-6hour，21-24hour）
weekend_holiday_expectation_time3a_am2 = weekend_holiday_li_expectation[21:42]
weekend_holiday_expectation_time3a_pm4 = weekend_holiday_li_expectation[147:168]
# 3b （1-4hour，7-10hour，10-13hour，19-22hour，22-1hour）
weekend_holiday_expectation_time3b_am1 = weekend_holiday_li_expectation[7:28]
weekend_holiday_expectation_time3b_am3 = weekend_holiday_li_expectation[49:70]
weekend_holiday_expectation_time3b_am4 = weekend_holiday_li_expectation[70:91]
weekend_holiday_expectation_time3b_pm3 = weekend_holiday_li_expectation[133:154]
weekend_holiday_expectation_time3b_pm4 = weekend_holiday_li_expectation[154:168]+weekend_holiday_li_expectation[0:7]
# 3c （2-5hour，11-14hour，23-2hour）
weekend_holiday_expectation_time3c_am1 = weekend_holiday_li_expectation[14:35]
weekend_holiday_expectation_time3c_am4 = weekend_holiday_li_expectation[77:98]
weekend_holiday_expectation_time3c_pm4 = weekend_holiday_li_expectation[161:168]+weekend_holiday_li_expectation[0:14]
# 4b （1-5hour，9-13hour，21-1hour）
weekend_holiday_expectation_time4b_am1 = weekend_holiday_li_expectation[7:35]
weekend_holiday_expectation_time4b_am3 = weekend_holiday_li_expectation[63:91]
weekend_holiday_expectation_time4b_pm3 = weekend_holiday_li_expectation[147:168] + weekend_holiday_li_expectation[0:7]
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
weekend_holiday_expectation_time4c_am1 = weekend_holiday_li_expectation[14:42]
weekend_holiday_expectation_time4c_am3 = weekend_holiday_li_expectation[70:98]
weekend_holiday_expectation_time4c_pm1 = weekend_holiday_li_expectation[98:126]
weekend_holiday_expectation_time4c_pm2 = weekend_holiday_li_expectation[126:154]
weekend_holiday_expectation_time4c_pm3 = weekend_holiday_li_expectation[154:168] + weekend_holiday_li_expectation[0:14]
# 6b （1-7hour，19-1hour）
weekend_holiday_expectation_time6b_am1 = weekend_holiday_li_expectation[7:49]
weekend_holiday_expectation_time6b_pm2 = weekend_holiday_li_expectation[133:168]+weekend_holiday_li_expectation[0:7]


# 有效道德区间数据汇总
weekend_holiday_moral_group = [weekend_holiday_moral_time3a_am2+weekend_holiday_moral_time3a_pm4+
                               weekend_holiday_moral_time3b_am1+weekend_holiday_moral_time3b_am3+
                               weekend_holiday_moral_time3b_am4+weekend_holiday_moral_time3b_pm3+
                               weekend_holiday_moral_time3b_pm4+weekend_holiday_moral_time3c_am1+
                               weekend_holiday_moral_time3c_am4+weekend_holiday_moral_time3c_pm4+
                               weekend_holiday_moral_time4b_am1+weekend_holiday_moral_time4b_am3+
                               weekend_holiday_moral_time4b_pm3+weekend_holiday_moral_time4c_am1+
                               weekend_holiday_moral_time4c_am3+weekend_holiday_moral_time4c_pm1+
                               weekend_holiday_moral_time4c_pm2+weekend_holiday_moral_time4c_pm3+
                               weekend_holiday_moral_time6b_am1+weekend_holiday_moral_time6b_pm2]

# 有效情绪区间数据汇总（总情绪）
weekend_holiday_emo_group = [weekend_holiday_emo_time3a_am2+weekend_holiday_emo_time3a_pm4+
                               weekend_holiday_emo_time3b_am1+weekend_holiday_emo_time3b_am3+
                               weekend_holiday_emo_time3b_am4+weekend_holiday_emo_time3b_pm3+
                               weekend_holiday_emo_time3b_pm4+weekend_holiday_emo_time3c_am1+
                               weekend_holiday_emo_time3c_am4+weekend_holiday_emo_time3c_pm4+
                               weekend_holiday_emo_time4b_am1+weekend_holiday_emo_time4b_am3+
                               weekend_holiday_emo_time4b_pm3+weekend_holiday_emo_time4c_am1+
                               weekend_holiday_emo_time4c_am3+weekend_holiday_emo_time4c_pm1+
                               weekend_holiday_emo_time4c_pm2+weekend_holiday_emo_time4c_pm3+
                               weekend_holiday_emo_time6b_am1+weekend_holiday_emo_time6b_pm2]

# 有效情绪区间数据汇总（积极情绪positive）
weekend_holiday_positive_group = [weekend_holiday_positive_time3a_am2+weekend_holiday_positive_time3a_pm4+
                               weekend_holiday_positive_time3b_am1+weekend_holiday_positive_time3b_am3+
                               weekend_holiday_positive_time3b_am4+weekend_holiday_positive_time3b_pm3+
                               weekend_holiday_positive_time3b_pm4+weekend_holiday_positive_time3c_am1+
                               weekend_holiday_positive_time3c_am4+weekend_holiday_positive_time3c_pm4+
                               weekend_holiday_positive_time4b_am1+weekend_holiday_positive_time4b_am3+
                               weekend_holiday_positive_time4b_pm3+weekend_holiday_positive_time4c_am1+
                               weekend_holiday_positive_time4c_am3+weekend_holiday_positive_time4c_pm1+
                               weekend_holiday_positive_time4c_pm2+weekend_holiday_positive_time4c_pm3+
                               weekend_holiday_positive_time6b_am1+weekend_holiday_positive_time6b_pm2]

# 有效情绪区间数据汇总（消极情绪negative）
weekend_holiday_negative_group = [weekend_holiday_negative_time3a_am2+weekend_holiday_negative_time3a_pm4+
                               weekend_holiday_negative_time3b_am1+weekend_holiday_negative_time3b_am3+
                               weekend_holiday_negative_time3b_am4+weekend_holiday_negative_time3b_pm3+
                               weekend_holiday_negative_time3b_pm4+weekend_holiday_negative_time3c_am1+
                               weekend_holiday_negative_time3c_am4+weekend_holiday_negative_time3c_pm4+
                               weekend_holiday_negative_time4b_am1+weekend_holiday_negative_time4b_am3+
                               weekend_holiday_negative_time4b_pm3+weekend_holiday_negative_time4c_am1+
                               weekend_holiday_negative_time4c_am3+weekend_holiday_negative_time4c_pm1+
                               weekend_holiday_negative_time4c_pm2+weekend_holiday_negative_time4c_pm3+
                               weekend_holiday_negative_time6b_am1+weekend_holiday_negative_time6b_pm2]

# 有效anger情绪区间数据汇总
weekend_holiday_anger_group = [weekend_holiday_anger_time3a_am2+weekend_holiday_anger_time3a_pm4+
                               weekend_holiday_anger_time3b_am1+weekend_holiday_anger_time3b_am3+
                               weekend_holiday_anger_time3b_am4+weekend_holiday_anger_time3b_pm3+
                               weekend_holiday_anger_time3b_pm4+weekend_holiday_anger_time3c_am1+
                               weekend_holiday_anger_time3c_am4+weekend_holiday_anger_time3c_pm4+
                               weekend_holiday_anger_time4b_am1+weekend_holiday_anger_time4b_am3+
                               weekend_holiday_anger_time4b_pm3+weekend_holiday_anger_time4c_am1+
                               weekend_holiday_anger_time4c_am3+weekend_holiday_anger_time4c_pm1+
                               weekend_holiday_anger_time4c_pm2+weekend_holiday_anger_time4c_pm3+
                               weekend_holiday_anger_time6b_am1+weekend_holiday_anger_time6b_pm2]

# 有效anxi情绪区间数据汇总
weekend_holiday_anxi_group = [weekend_holiday_anxi_time3a_am2+weekend_holiday_anxi_time3a_pm4+
                               weekend_holiday_anxi_time3b_am1+weekend_holiday_anxi_time3b_am3+
                               weekend_holiday_anxi_time3b_am4+weekend_holiday_anxi_time3b_pm3+
                               weekend_holiday_anxi_time3b_pm4+weekend_holiday_anxi_time3c_am1+
                               weekend_holiday_anxi_time3c_am4+weekend_holiday_anxi_time3c_pm4+
                               weekend_holiday_anxi_time4b_am1+weekend_holiday_anxi_time4b_am3+
                               weekend_holiday_anxi_time4b_pm3+weekend_holiday_anxi_time4c_am1+
                               weekend_holiday_anxi_time4c_am3+weekend_holiday_anxi_time4c_pm1+
                               weekend_holiday_anxi_time4c_pm2+weekend_holiday_anxi_time4c_pm3+
                               weekend_holiday_anxi_time6b_am1+weekend_holiday_anxi_time6b_pm2]

# 有效hate情绪区间数据汇总
weekend_holiday_hate_group = [weekend_holiday_hate_time3a_am2+weekend_holiday_hate_time3a_pm4+
                               weekend_holiday_hate_time3b_am1+weekend_holiday_hate_time3b_am3+
                               weekend_holiday_hate_time3b_am4+weekend_holiday_hate_time3b_pm3+
                               weekend_holiday_hate_time3b_pm4+weekend_holiday_hate_time3c_am1+
                               weekend_holiday_hate_time3c_am4+weekend_holiday_hate_time3c_pm4+
                               weekend_holiday_hate_time4b_am1+weekend_holiday_hate_time4b_am3+
                               weekend_holiday_hate_time4b_pm3+weekend_holiday_hate_time4c_am1+
                               weekend_holiday_hate_time4c_am3+weekend_holiday_hate_time4c_pm1+
                               weekend_holiday_hate_time4c_pm2+weekend_holiday_hate_time4c_pm3+
                               weekend_holiday_hate_time6b_am1+weekend_holiday_hate_time6b_pm2]

# 有效sad情绪区间数据汇总
weekend_holiday_sad_group = [weekend_holiday_sad_time3a_am2+weekend_holiday_sad_time3a_pm4+
                               weekend_holiday_sad_time3b_am1+weekend_holiday_sad_time3b_am3+
                               weekend_holiday_sad_time3b_am4+weekend_holiday_sad_time3b_pm3+
                               weekend_holiday_sad_time3b_pm4+weekend_holiday_sad_time3c_am1+
                               weekend_holiday_sad_time3c_am4+weekend_holiday_sad_time3c_pm4+
                               weekend_holiday_sad_time4b_am1+weekend_holiday_sad_time4b_am3+
                               weekend_holiday_sad_time4b_pm3+weekend_holiday_sad_time4c_am1+
                               weekend_holiday_sad_time4c_am3+weekend_holiday_sad_time4c_pm1+
                               weekend_holiday_sad_time4c_pm2+weekend_holiday_sad_time4c_pm3+
                               weekend_holiday_sad_time6b_am1+weekend_holiday_sad_time6b_pm2]

# 有效surprise情绪区间数据汇总
weekend_holiday_surprise_group = [weekend_holiday_surprise_time3a_am2+weekend_holiday_surprise_time3a_pm4+
                               weekend_holiday_surprise_time3b_am1+weekend_holiday_surprise_time3b_am3+
                               weekend_holiday_surprise_time3b_am4+weekend_holiday_surprise_time3b_pm3+
                               weekend_holiday_surprise_time3b_pm4+weekend_holiday_surprise_time3c_am1+
                               weekend_holiday_surprise_time3c_am4+weekend_holiday_surprise_time3c_pm4+
                               weekend_holiday_surprise_time4b_am1+weekend_holiday_surprise_time4b_am3+
                               weekend_holiday_surprise_time4b_pm3+weekend_holiday_surprise_time4c_am1+
                               weekend_holiday_surprise_time4c_am3+weekend_holiday_surprise_time4c_pm1+
                               weekend_holiday_surprise_time4c_pm2+weekend_holiday_surprise_time4c_pm3+
                               weekend_holiday_surprise_time6b_am1+weekend_holiday_surprise_time6b_pm2]

# 有效love情绪区间数据汇总
weekend_holiday_love_group = [weekend_holiday_love_time3a_am2+weekend_holiday_love_time3a_pm4+
                               weekend_holiday_love_time3b_am1+weekend_holiday_love_time3b_am3+
                               weekend_holiday_love_time3b_am4+weekend_holiday_love_time3b_pm3+
                               weekend_holiday_love_time3b_pm4+weekend_holiday_love_time3c_am1+
                               weekend_holiday_love_time3c_am4+weekend_holiday_love_time3c_pm4+
                               weekend_holiday_love_time4b_am1+weekend_holiday_love_time4b_am3+
                               weekend_holiday_love_time4b_pm3+weekend_holiday_love_time4c_am1+
                               weekend_holiday_love_time4c_am3+weekend_holiday_love_time4c_pm1+
                               weekend_holiday_love_time4c_pm2+weekend_holiday_love_time4c_pm3+
                               weekend_holiday_love_time6b_am1+weekend_holiday_love_time6b_pm2]

# 有效joy情绪区间数据汇总
weekend_holiday_joy_group = [weekend_holiday_joy_time3a_am2+weekend_holiday_joy_time3a_pm4+
                               weekend_holiday_joy_time3b_am1+weekend_holiday_joy_time3b_am3+
                               weekend_holiday_joy_time3b_am4+weekend_holiday_joy_time3b_pm3+
                               weekend_holiday_joy_time3b_pm4+weekend_holiday_joy_time3c_am1+
                               weekend_holiday_joy_time3c_am4+weekend_holiday_joy_time3c_pm4+
                               weekend_holiday_joy_time4b_am1+weekend_holiday_joy_time4b_am3+
                               weekend_holiday_joy_time4b_pm3+weekend_holiday_joy_time4c_am1+
                               weekend_holiday_joy_time4c_am3+weekend_holiday_joy_time4c_pm1+
                               weekend_holiday_joy_time4c_pm2+weekend_holiday_joy_time4c_pm3+
                               weekend_holiday_joy_time6b_am1+weekend_holiday_joy_time6b_pm2]

# 有效expectation情绪区间数据汇总
weekend_holiday_expectation_group = [weekend_holiday_expectation_time3a_am2+weekend_holiday_expectation_time3a_pm4+
                               weekend_holiday_expectation_time3b_am1+weekend_holiday_expectation_time3b_am3+
                               weekend_holiday_expectation_time3b_am4+weekend_holiday_expectation_time3b_pm3+
                               weekend_holiday_expectation_time3b_pm4+weekend_holiday_expectation_time3c_am1+
                               weekend_holiday_expectation_time3c_am4+weekend_holiday_expectation_time3c_pm4+
                               weekend_holiday_expectation_time4b_am1+weekend_holiday_expectation_time4b_am3+
                               weekend_holiday_expectation_time4b_pm3+weekend_holiday_expectation_time4c_am1+
                               weekend_holiday_expectation_time4c_am3+weekend_holiday_expectation_time4c_pm1+
                               weekend_holiday_expectation_time4c_pm2+weekend_holiday_expectation_time4c_pm3+
                               weekend_holiday_expectation_time6b_am1+weekend_holiday_expectation_time6b_pm2]


# 有效时间标签构造（Morning=1,Afternoon=0,Night=-1）
# 注，需要在数据结构中改为category类型
weekend_holiday_time_group = [21*[-1]+21*[-1]+
                              21*[-1]+21*[1]+21*[1]+21*[0]+21*[-1]+
                              21*[-1]+21*[1]+21*[-1]+
                              28*[-1]+28*[1]+28*[-1]+
                              28*[-1]+28*[1]+28*[0]+28*[0]+28*[-1]+
                              42*[-1]+42*[0]]

# 有效时区长度标签构造（三小时时区=3，四小时时区=4，六小时时区=6）
# 注，需要在数据结构中改为category类型
weekend_holiday_zone_group = [210*[3]+224*[4]+84*[6]]

In [17]:
# weekend_holiday中介效应测试结果
# 总体情绪——道德动机
weekend_holiday_emo_to_moral = test_media_process2(weekend_holiday_time_group[0],
                                                   weekend_holiday_moral_group[0],
                                                   weekend_holiday_emo_group[0],4,
                                                   'Total_emotion(weekend_holiday)')
# 积极情绪——道德动机
weekend_holiday_positive_to_moral = test_media_process2(weekend_holiday_time_group[0],
                                                        weekend_holiday_moral_group[0],
                                                        weekend_holiday_positive_group[0],4,
                                                        'Positive(weekend_holiday)')
# 消极情绪——道德动机
weekend_holiday_negative_to_moral = test_media_process2(weekend_holiday_time_group[0],
                                                        weekend_holiday_moral_group[0],
                                                        weekend_holiday_negative_group[0],4,
                                                        'Negative(weekend_holiday)')

#*******************************************************************************
# anger——道德动机
weekend_holiday_anger_to_moral = test_media_process2(weekend_holiday_time_group[0],weekend_holiday_moral_group[0],
                                   weekend_holiday_anger_group[0],4,'Anger(weekend_holiday)')
# anxi——道德动机
weekend_holiday_anxi_to_moral = test_media_process2(weekend_holiday_time_group[0],weekend_holiday_moral_group[0],
                                   weekend_holiday_anxi_group[0],4,'Anxi(weekend_holiday)')
# hate——道德动机
weekend_holiday_hate_to_moral = test_media_process2(weekend_holiday_time_group[0],weekend_holiday_moral_group[0],
                                   weekend_holiday_hate_group[0],4,'Hate(weekend_holiday)')
# sad——道德动机
weekend_holiday_sad_to_moral = test_media_process2(weekend_holiday_time_group[0],weekend_holiday_moral_group[0],
                                   weekend_holiday_sad_group[0],4,'Sad(weekend_holiday)')
# surprise——道德动机
weekend_holiday_surprise_to_moral = test_media_process2(weekend_holiday_time_group[0],weekend_holiday_moral_group[0],
                                   weekend_holiday_surprise_group[0],4,'Surprise(weekend_holiday)')
#*******************************************************************************
# love——道德动机
weekend_holiday_love_to_moral = test_media_process2(weekend_holiday_time_group[0],weekend_holiday_moral_group[0],
                                   weekend_holiday_love_group[0],4,'Love(weekend_holiday)')
# joy——道德动机
weekend_holiday_joy_to_moral = test_media_process2(weekend_holiday_time_group[0],weekend_holiday_moral_group[0],
                                   weekend_holiday_joy_group[0],4,'Joy(weekend_holiday)')
# expectation——道德动机
weekend_holiday_expectation_to_moral = test_media_process2(weekend_holiday_time_group[0],weekend_holiday_moral_group[0],
                                   weekend_holiday_expectation_group[0],4,'Expectation(weekend_holiday)')


Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 4

Variables:
    Cons = Cons
    x = Time
    y = Moral
    m1 = Total_emotion(weekend_holiday)

Sample size:
518

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = Moral 
OLS Regression Summary

     R²  Adj. R²    MSE      F  df1  df2  p-value
 0.0264   0.0207 0.7513 6.9743    2  515   0.0010

Coefficients

                                coeff     se       t      p    LLCI   ULCI
Cons                           0.9259 0.0474 19.5131 0.0000  0.8329 1.0189
Time                           0.1383 0.0469  2.9510 0.0033  0.0464 0.2301
Total_emotion(weekend_holiday) 0.0025 0.0013  1.8549 0.0642 -0.0001 0.0051

----------------

Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 4

Variables:
    Cons = Cons
    x = Time
    y = Moral
    m1 = Hate(weekend_holiday)

Sample size:
518

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = Moral 
OLS Regression Summary

     R²  Adj. R²    MSE      F  df1  df2  p-value
 0.0270   0.0214 0.7508 7.1586    2  515   0.0009

Coefficients

                       coeff     se       t      p    LLCI   ULCI
Cons                  0.9412 0.0435 21.6400 0.0000  0.8559 1.0264
Time                  0.1503 0.0464  3.2404 0.0013  0.0594 0.2412
Hate(weekend_holiday) 0.1444 0.0741  1.9499 0.0517 -0.0007 0.2895

-------------------------------------------------------------

Process successfully initialized.
Based on the Process Macro by Andrew F. Hayes, Ph.D. (www.afhayes.com)


****************************** SPECIFICATION ****************************

Model = 4

Variables:
    Cons = Cons
    x = Time
    y = Moral
    m1 = Expectation(weekend_holiday)

Sample size:
518

Bootstrapping information for indirect effects:
Final number of bootstrap samples: 5000
Number of samples discarded due to convergence issues: 0

***************************** OUTCOME MODELS ****************************

Outcome = Moral 
OLS Regression Summary

     R²  Adj. R²    MSE      F  df1  df2  p-value
 0.0286   0.0229 0.7496 7.5793    2  515   0.0006

Coefficients

                              coeff     se       t      p   LLCI   ULCI
Cons                         0.9256 0.0458 20.1911 0.0000 0.8357 1.0154
Time                         0.1450 0.0464  3.1246 0.0019 0.0541 0.2360
Expectation(weekend_holiday) 0.0512 0.0238  2.1510 0.0319 0.0045 0.0979

------------------------------

In [19]:
# （一）week情境中介效应检测模型（非有效区间测试）

# 非有效道德区间

# 非有效情绪区间

# 非有效时间标签

#（1）上午下午夜晚——>道德动机（中介：总体情绪，条件：时区长度）
dict_test = {'Time':week_time_group5[0],'Moral':week_moral_group1[0],
             'Emotion':week_emo_group2[0]}
test_data = pd.DataFrame.from_dict(dict_test)
test_data['Time']=test_data['Time'].astype('category')
#test_data['Time_zone']=test_data['Time_zone'].astype('category')
p = Process(data=test_data,model=4,x='Time',y='Moral',m=['Emotion'])
p.summary()


NameError: name 'week_time_group5' is not defined

In [ ]:
# 有效道德区间备注
# （1）week
# 3a （9-12hour，15-18hour，18-21hour，21-24hour）
# 3b （10-13hour，16-19hour，22-1hour）
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour）
# 4a （8-12hour，16-20hour，20-24hour）
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
# 4c （10-14hour，14-18hour，22-2hour）
# 4d （11-15hour，15-19hour）
# 6b （7-13hour，13-19hour，19-1hour）
# 6c （8-14hour，14-20hour，20-2hour）
# 6d （9-15hour，15-21hour）

# （2）workdays
# 3a （9-12hour，15-18hour，21-24hour）
# 3b （7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
# 3c （5-8hour，8-11hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
# 4a （8-12hour，16-20hour，20-24hour）
# 4b （5-9hour，9-13hour，13-17hour，17-21hour，21-1hour）
# 4c （6-10hour，10-14hour，14-18hour，22-2hour）
# 4d （7-11hour，11-15hour，15-19hour，19-23hour）
# 6a （6-12hour，12-18hour）
# 6b （7-13hour，13-19hour，19-1hour）
# 6c （8-14hour，14-20hour，20-2hour）
# 6e （4-10hour，16-22hour）
# 6f （5-11hour，11-17hour，17-23hour）

# （3）weekend
# 3a （0-3hour，6-9hour，12-15hour，21-24hour）
# 3b （1-4hour，10-13hour，16-19hour，22-1hour）
# 4a （0-4hour，8-12hour，12-16hour）
# 4d （7-11hour，11-15hour，23-3hour）

# （4）holiday
# 3a （3-6hour，9-12hour，12-15hour，15-18hour，18-21hour，21-24hour）
# 3b （1-4hour，7-10hour，10-13hour，13-16hour，16-19hour，19-22hour，22-1hour）
# 3c （2-5hour，11-14hour，14-17hour，17-20hour，20-23hour，23-2hour）
# 4b （1-5hour，9-13hour，13-17hour，17-21hour，21-1hour）
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
# 6a （0-6hour，12-18hour）
# 6b （1-7hour，19-1hour）
# 6c （2-8hour，8-14hour，14-20hour，20-2hour）
# 6d （3-9hour，9-15hour）
# 6e （4-10hour，10-16hour）

# （5）weekend & holiday
# 3a （3-6hour，21-24hour）
# 3b （1-4hour，7-10hour，10-13hour，19-22hour，22-1hour）
# 3c （2-5hour，11-14hour，23-2hour）
# 4b （1-5hour，9-13hour，21-1hour）
# 4c （2-6hour，10-14hour，14-18hour，18-22hour，22-2hour）
# 6b （1-7hour，19-1hour）

In [27]:
# 道德与情绪平行坐标绘图
import numpy as np
import pandas as pd
import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)
# 获取数据
paral_path1 = r'F:/baidu_moraldata/parallel coordinates/test.txt'
paral_path2 = r'F:/baidu_moraldata/parallel coordinates/indirect.txt'
paral_path3 = r'F:/baidu_moraldata/parallel coordinates/total.txt'
paral_direct = pd.read_table(paral_path1)
paral_indirect = pd.read_table(paral_path2)
pparal_total = pd.read_table(paral_path3)

paral_direct['Scene_id']=paral_direct['Scene'].map({'week':1,'workdays':2,'weekend':3,
                                                    'holiday':4,'weekend&holiday':5})

py.offline.iplot({
    "data":[go.Parcoords(
        line = dict(color = paral_direct['Scene_id'],
                    colorscale = [[1,'yellow'],[1,'blue'],[1,'orangered'],[1,'deepskyblue'],[1,'skyblue']]),
        dimensions = list([
            dict(range=[0,2],
                 tickvals=[1],
                 values=paral_direct['X'],
                 ticktext=['Time']),
            dict(range=[0,2],
                 tickvals=[1],
                 values=paral_direct['Effect Type'],
                 ticktext=['Direct effect']),
            dict(range= [-0.1,0.25],
                 label='Direct Effect value',values=paral_direct['D_Effect']),
            dict(range=[0.01,0.05],
                 label='SE',values=paral_direct['SE']),
            dict(range=[-4,6],
                 label='T',values=paral_direct['T']),
                 #ticktext=['text1','text2','text3','text4']),
            dict(range=[0,0.2],
                 label='P',values=paral_direct['P']),
            dict(range=[-0.2,0.2],
                 label='LLCI',values=paral_direct['LLCI']),
            dict(range=[-0.1,0.4],
                 label='ULCI',values=paral_direct['ULCI']),
            dict(range=[0,2],
                 tickvals=[1],
                 values=paral_direct['Y'],
                 ticktext=['Moral'])
        ])
      )],"layout":go.Layout(title='My first parallel coordinates')
})


In [34]:
# 道德与情绪平行坐标绘图
import numpy as np
import pandas as pd
import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)
# 获取数据
paral_path1 = r'F:/baidu_moraldata/parallel coordinates/test2.txt'
paral_path2 = r'F:/baidu_moraldata/parallel coordinates/indirect.txt'
paral_path3 = r'F:/baidu_moraldata/parallel coordinates/total.txt'
paral_direct = pd.read_table(paral_path1)
paral_indirect = pd.read_table(paral_path2)
pparal_total = pd.read_table(paral_path3)

paral_direct['Scene_id']=paral_direct['Scene'].map({'week':1,'workdays':2,'weekend':3,
                                                    'holiday':4,'weekend&holiday':5})

py.offline.iplot({
    "data":[go.Parcoords(
        line = dict(color = paral_direct['Scene_id'],
                    colorscale = [[1,'yellow'],[1,'blue'],[1,'orangered'],[1,'deepskyblue'],[1,'skyblue']]),
        dimensions = list([
            dict(range=[0,2],
                 tickvals=[1],
                 values=paral_direct['X'],
                 ticktext=['Time']),
            dict(range=[0,2],
                 tickvals=[1],
                 values=paral_direct['Effect Type'],
                 ticktext=['InDirect effect']),
            dict(range= [-0.03,0.02],
                 label='InDirect Effect value',values=paral_direct['InD_Effect']),
            dict(range=[0,0.015],
                 label='BootSE',values=paral_direct['BootSE']),
            dict(range=[-0.06,0.01],
                 label='BootLLCI',values=paral_direct['BootLLCI']),
                 #ticktext=['text1','text2','text3','text4']),
            dict(range=[-0.02,0.03],
                 label='BootULCI',values=paral_direct['BootULCI']),
            dict(range=[0,4],
                 tickvals=[1,2,3],
                 label='Mediating',
                 values=paral_direct['Mediating effect'],
                 ticktext=['Full mediation','Mediation','No Mediation']),
            dict(range=[0,2],
                 tickvals=[1],
                 values=paral_direct['Y'],
                 ticktext=['Moral'])
        ])
      )],"layout":go.Layout(title='My first parallel coordinates')
})

In [37]:
import pyecharts.options as opts
from pyecharts.charts import Parallel

r = np.random.rand(12)
data = [[i,round(j,2),round(m,2),n] for i,j,m,n in zip(range(12),r,np.random.randint(1,23,12)+r,np.random.choice(['A','B'],12))]
print(data)

schema = [
    {'dim':0,'name':'月份'},
    {'dim':1,'name':'指标1'},
    {'dim':2,'name':'指标2'},
    {'dim':3,'name':'类别','type':'category','data':['A','B']}
]
p = Parallel(init_opts=opts.InitOpts(height='350px',width='850px'))
p.add_schema(schema=schema)
p.add('2015',data,linestyle_opts=opts.LineStyleOpts(color='#CD0000'))
r = np.random.rand(12)
data = [[i,round(j,2),round(m,2),n] for i,j,m,n in zip(range(12),r,np.random.randint(1,23,12)+r,np.random.choice(['A','B'],12))]

p.add('2016',data,linestyle_opts=opts.LineStyleOpts(color='#5CACEE'))
p.render_notebook()

[[0, 0.02, 17.02, 'A'], [1, 0.18, 5.18, 'A'], [2, 0.98, 17.98, 'A'], [3, 0.69, 11.69, 'B'], [4, 0.5, 17.5, 'B'], [5, 0.58, 11.58, 'A'], [6, 0.27, 11.27, 'A'], [7, 0.66, 20.66, 'A'], [8, 0.54, 12.54, 'B'], [9, 0.21, 15.21, 'A'], [10, 0.84, 10.84, 'B'], [11, 0.27, 18.27, 'B']]


In [1]:
import numpy as np
import pandas as pd
import pyecharts.options as opts
from pyecharts.charts import Parallel

# 数据读取路径
# （一）以星期为颜色进行分配
week_d_path = r'F:/baidu_moraldata/parallel coordinates/pyechart/week_direct.txt'
workdays_d_path = r'F:/baidu_moraldata/parallel coordinates/pyechart/workdays_direct.txt'
weekend_d_path = r'F:/baidu_moraldata/parallel coordinates/pyechart/weekend_direct.txt'
holiday_d_path = r'F:/baidu_moraldata/parallel coordinates/pyechart/holiday_direct.txt'
weekend_holiday_d_path = r'F:/baidu_moraldata/parallel coordinates/pyechart/weekend&holiday_direct.txt'
week_i_path = r'F:/baidu_moraldata/parallel coordinates/pyechart/week_indirect.txt'
workdays_i_path = r'F:/baidu_moraldata/parallel coordinates/pyechart/workdays_indirect.txt'
weekend_i_path = r'F:/baidu_moraldata/parallel coordinates/pyechart/weekend_indirect.txt'
holiday_i_path = r'F:/baidu_moraldata/parallel coordinates/pyechart/holiday_indirect.txt'
weekend_holiday_i_path = r'F:/baidu_moraldata/parallel coordinates/pyechart/weekend&holiday_indirect.txt'

# 数据读取dataframe
week_d = pd.read_table(week_d_path)
week_i = pd.read_table(week_i_path)
workdays_d = pd.read_table(workdays_d_path)
workdays_i = pd.read_table(workdays_i_path)
weekend_d = pd.read_table(weekend_d_path)
weekend_i = pd.read_table(weekend_i_path)
holiday_d = pd.read_table(holiday_d_path)
holiday_i = pd.read_table(holiday_i_path)
weekend_holiday_d = pd.read_table(weekend_holiday_d_path)
weekend_holiday_i = pd.read_table(weekend_holiday_i_path)

#数据转换函数
def get_data_d(data_d):
    final_data = [[a,b,c,d,e,f,g,h,i,j] for a,b,c,d,e,f,g,h,i,j in zip(data_d['X'],
                                                                        data_d['Effect Type'],
                                                                        data_d['Emotion'],
                                                                        data_d['D_Effect'],
                                                                        data_d['SE'],
                                                                        data_d['T'],
                                                                        data_d['P'],
                                                                        data_d['LLCI'],
                                                                        data_d['ULCI'],
                                                                        data_d['Y'])]
    return final_data

def get_data_i(data_i):
    final_data = [[a,b,c,d,e,f,g,h,i] for a,b,c,d,e,f,g,h,i in zip(data_i['X'],
                                                                       data_i['Effect Type'],
                                                                       data_i['Emotion'],
                                                                       data_i['InD_Effect'],
                                                                       data_i['BootSE'],
                                                                       data_i['BootLLCI'],
                                                                       data_i['BootULCI'],
                                                                       data_i['Mediating effect'],
                                                                       data_i['Y'])]
    return final_data

def get_emotion_d(data_d):
    final_data = [[a,b,c,d,e,f,g,h,i,j,k] for a,b,c,d,e,f,g,h,i,j,k in zip(data_d['X'],
                                                                           data_d['Effect Type'],
                                                                           data_d['D_Effect'],
                                                                           data_d['SE'],
                                                                           data_d['T'],
                                                                           data_d['P'],
                                                                           data_d['LLCI'],
                                                                           data_d['ULCI'],
                                                                           data_d['Scene'],
                                                                           data_d['Statistical significance'],
                                                                           data_d['Y'])]
    return final_data

def get_emotion_i(data_i):
    final_data = [[a,b,c,d,e,f,g,h,i] for a,b,c,d,e,f,g,h,i in zip(data_i['X'],
                                                                   data_i['Effect Type'],
                                                                   data_i['InD_Effect'],
                                                                   data_i['BootSE'],
                                                                   data_i['BootLLCI'],
                                                                   data_i['BootULCI'],
                                                                   data_i['Scene'],
                                                                   data_i['Mediating effect'],
                                                                   data_i['Y'])]
    return final_data

# 数据准备
week_d_data = get_data_d(week_d)
week_i_data = get_data_i(week_i)
workdays_d_data = get_data_d(workdays_d)
workdays_i_data = get_data_i(workdays_i)
weekend_d_data = get_data_d(weekend_d)
weekend_i_data = get_data_i(weekend_i)
holiday_d_data = get_data_d(holiday_d)
holiday_i_data = get_data_i(holiday_i)
weekend_holiday_d_data = get_data_d(weekend_holiday_d)
weekend_holiday_i_data = get_data_i(weekend_holiday_i)


# （二）以情绪为颜色进行分配
# 数据读取路径

total_emotion_d_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_d/Total_emotion_d.txt'
positive_d_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_d/Positive_d.txt'
negative_d_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_d/Negative_d.txt'
anger_d_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_d/Anger_d.txt'
anxi_d_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_d/Anxious_d.txt'
hate_d_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_d/Hate_d.txt'
sad_d_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_d/Sad_d.txt'
surprise_d_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_d/Surprise_d.txt'
love_d_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_d/Love_d.txt'
joy_d_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_d/Joy_d.txt'
expectation_d_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_d/Expectation_d.txt'


total_emotion_i_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_i/Total_emotion_i.txt'
positive_i_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_i/Positive_i.txt'
negative_i_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_i/Negative_i.txt'
anger_i_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_i/Anger_i.txt'
anxi_i_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_i/Anxious_i.txt'
hate_i_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_i/Hate_i.txt'
sad_i_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_i/Sad_i.txt'
surprise_i_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_i/Surprise_i.txt'
love_i_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_i/Love_i.txt'
joy_i_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_i/Joy_i.txt'
expectation_i_path=r'F:/baidu_moraldata/parallel coordinates/pyechart_i/Expectation_i.txt'

# 数据读取dataframe
total_emotion_d = pd.read_table(total_emotion_d_path)
positive_d = pd.read_table(positive_d_path)
negative_d = pd.read_table(negative_d_path)
anger_d = pd.read_table(anger_d_path)
anxi_d = pd.read_table(anxi_d_path)
hate_d = pd.read_table(hate_d_path)
sad_d = pd.read_table(sad_d_path)
surprise_d = pd.read_table(surprise_d_path)
love_d = pd.read_table(love_d_path)
joy_d = pd.read_table(joy_d_path)
expectation_d = pd.read_table(expectation_d_path)

total_emotion_i = pd.read_table(total_emotion_i_path)
positive_i = pd.read_table(positive_i_path)
negative_i = pd.read_table(negative_i_path)
anger_i = pd.read_table(anger_i_path)
anxi_i = pd.read_table(anxi_i_path)
hate_i = pd.read_table(hate_i_path)
sad_i = pd.read_table(sad_i_path)
surprise_i = pd.read_table(surprise_i_path)
love_i = pd.read_table(love_i_path)
joy_i = pd.read_table(joy_i_path)
expectation_i = pd.read_table(expectation_i_path)

# 数据准备
total_emotion_d_data = get_emotion_d(total_emotion_d)
positive_d_data = get_emotion_d(positive_d)
negative_d_data = get_emotion_d(negative_d)
anger_d_data = get_emotion_d(anger_d)
anxi_d_data = get_emotion_d(anxi_d)
hate_d_data = get_emotion_d(hate_d)
sad_d_data = get_emotion_d(sad_d)
surprise_d_data = get_emotion_d(surprise_d)
love_d_data = get_emotion_d(love_d)
joy_d_data = get_emotion_d(joy_d)
expectation_d_data = get_emotion_d(expectation_d)

total_emotion_i_data = get_emotion_i(total_emotion_i)
positive_i_data = get_emotion_i(positive_i)
negative_i_data = get_emotion_i(negative_i)
anger_i_data = get_emotion_i(anger_i)
anxi_i_data = get_emotion_i(anxi_i)
hate_i_data = get_emotion_i(hate_i)
sad_i_data = get_emotion_i(sad_i)
surprise_i_data = get_emotion_i(surprise_i)
love_i_data = get_emotion_i(love_i)
joy_i_data = get_emotion_i(joy_i)
expectation_i_data = get_emotion_i(expectation_i)



In [3]:
# 情绪为颜色
schema = [{'dim':0,'name':'','type':'category','data':['Time']},
          {'dim':1,'name':'','type':'category','data':['Direct effect']},
          {'dim':2,'name':'Effect Value'},
          {'dim':3,'name':'SE'},{'dim':4,'name':'T'},{'dim':5,'name':'P'},
          {'dim':6,'name':'LLCI'},{'dim':7,'name':'ULCI'},
          {'dim':8,'name':'Scene','type':'category','data':['week','workdays','weekend','holiday','weekend&holiday']},
          {'dim':9,'name':'Statistical significance','type':'category','data':['***','**','*','none']},
          {'dim':10,'name':'','type':'category','data':['Moral']}]
parallelopts = [{'pos_left':'50%'}]
p = Parallel(init_opts=opts.InitOpts(height='450px',width='850px'))
p.add_schema(schema=schema,parallel_opts=parallelopts)
p.add('Total Emotion',total_emotion_d_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='red',
                                                                                            opacity=0.5,width=3))
p.add('Positive',positive_d_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='gold',
                                                                                  opacity=0.5,width=1))
p.add('Negative',negative_d_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='blue',
                                                                                  opacity=0.5,width=1))
p.add('Anger',anger_d_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='deepskyblue',
                                                                            opacity=0.5,width=1))
p.add('Anxious',anxi_d_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='lightskyblue',
                                                                             opacity=0.5,width=1))
p.add('Hate',hate_d_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='lightsteelblue',
                                                                          opacity=0.5,width=1))
p.add('Sad',sad_d_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='lightblue',
                                                                        opacity=0.5,width=1))
p.add('Surprise',surprise_d_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='gray',
                                                                                  opacity=0.5,width=1))
p.add('Love',love_d_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='yellow',
                                                                          opacity=0.5,width=1))
p.add('Joy',joy_d_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='khaki',
                                                                        opacity=0.5,width=1))
p.add('Expectation',expectation_d_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='beige',
                                                                                        opacity=0.5,width=1))
p.render_notebook()



In [12]:
schema = [{'dim':0,'name':'','type':'category','data':['Time']},
          {'dim':1,'name':'','type':'category','data':['InDirect effect']},
          {'dim':2,'name':'Effect Value'},
          {'dim':3,'name':'BootSE'},{'dim':4,'name':'BootLLCI'},{'dim':5,'name':'BootULCI'},
          {'dime':6,'name':'Scene','type':'category','data':['week','workdays','weekend',
                                                             'holiday','weekend & holiday']},
          {'dim':7,'name':'Mediating','type':'category','data':['full mediation','mediation',
                                                                'no mediation']},
          {'dim':8,'name':'','type':'category','data':['Moral']}]

parallelopts = [{'pos_left':'50%'}]
p = Parallel(init_opts=opts.InitOpts(height='450px',width='850px'))
p.add_schema(schema=schema,parallel_opts=parallelopts)
p.add('Total Emotion',total_emotion_i_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='red'))
p.add('Positive',positive_i_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='pink'))
p.add('Negative',negative_i_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='violet'))
p.add('Anger',anger_i_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='coral'))
p.add('Anxious',anxi_i_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='yellow'))
p.add('Hate',hate_i_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='lime'))
p.add('Sad',sad_i_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='lightblue'))
p.add('Surprise',surprise_i_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='gray'))
p.add('Love',love_i_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='aqua'))
p.add('Joy',joy_i_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='gold'))
p.add('Expectation',expectation_i_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='salmon'))
p.render_notebook()

In [2]:
schema = [{'dim':0,'name':'','type':'category','data':['Time']},
          {'dim':1,'name':'','type':'category','data':['Direct Effect']},
          {'dim':2,'name':'Emotion','type':'category','data':['Total emotion','Positive','Negative',
                                                              'Anger','Anxious','Hate','Sad','Surprise',
                                                              'Love','Joy','Expectation']},
          {'dim':3,'name':'Effect value'},
          {'dim':4,'name':'SE'},{'dim':5,'name':'T'},{'dim':6,'name':'P'},
          {'dime':7,'name':'LLCI'},{'dime':8,'name':'ULCI'},
          {'dim':9,'name':'','type':'category','data':['Moral']}]
parallelopts = [{'pos_left':'50%','layout':'vertical'}]
p = Parallel(init_opts=opts.InitOpts(height='850px',width='850px'))
p.add_schema(schema=schema,parallel_opts=parallelopts)
p.add('Week',week_d_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='red'))
p.add('workdays',workdays_d_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='blue'))
p.add('weekend',weekend_d_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='yellow'))
p.add('holiday',holiday_d_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='purple'))
p.add('weekend & holiday',weekend_holiday_d_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='green'))
p.render_notebook()

In [14]:
schema = [{'dim':0,'name':'','type':'category','data':['Time']},
          {'dim':1,'name':'','type':'category','data':['InDirect Effect']},
          {'dim':2,'name':'Emotion','type':'category','data':['Total emotion','Positive','Negative',
                                                              'Anger','Anxious','Hate','Sad','Surprise',
                                                              'Love','Joy','Expectation']},
          {'dim':3,'name':'Effect value'},
          {'dim':4,'name':'BootSE'},{'dim':5,'name':'BootLLCI'},{'dim':6,'name':'BootULCI'},
          {'dime':7,'name':'Mediating','type':'category','data':['mediation','no mediation','full mediation']},
          {'dim':9,'name':'','type':'category','data':['Moral']}]
parallelopts = [{'pos_left':'50%','layout':'vertical'}]
p = Parallel(init_opts=opts.InitOpts(height='850px',width='850px'))
p.add_schema(schema=schema,parallel_opts=parallelopts)
p.add('Week',week_i_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='red',opacity=0.5,width=3,
                                                                          type_='*'))
p.add('workdays',workdays_i_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='blue',opacity=0.5,
                                                                                  type_='dotted'))
p.add('weekend',weekend_i_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='yellow',opacity=0.5,))
p.add('holiday',holiday_i_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='purple',opacity=0.5))
p.add('weekend & holiday',weekend_holiday_i_data,is_smooth=True,linestyle_opts=opts.LineStyleOpts(color='green',opacity=0.5))
p.render_notebook()